<a href="https://colab.research.google.com/github/emerenan/Algorithms/blob/master/scrap_fleury.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:

#!apt-get update # to update ubuntu to correctly run apt install
!pip install scrapy
!pip install requests
!pip install selenium
#!pip install pangres


     |████████████████████████████████| 245kB 7.8MB/s 
     |████████████████████████████████| 3.2MB 16.0MB/s 
     |████████████████████████████████| 61kB 6.7MB/s 
     |████████████████████████████████| 3.1MB 50.3MB/s 
     |████████████████████████████████| 245kB 35.1MB/s 
     |████████████████████████████████| 2.6MB 53.1MB/s 
     |████████████████████████████████| 61kB 8.0MB/s 
     |████████████████████████████████| 51kB 6.5MB/s 
  Created wheel for protego: filename=Protego-0.1.16-cp36-none-any.whl size=7765 sha256=71c5062e893ce617b3980ce794e732b6b10dd4a31bfb8845f80cef3676f52aac
  Stored in directory: /root/.cache/pip/wheels/51/01/d1/4a2286a976dccd025ba679acacfe37320540df0f2283ecab12
  Created wheel for PyDispatcher: filename=PyDispatcher-2.0.5-cp36-none-any.whl size=11517 sha256=327055e0d57a974ee0f01665dd4226e50c4ed63d980044a7ce9e6eb509375002
  Stored in directory: /root/.cache/pip/wheels/88/99/96/cfef6665f9cb1522ee6757ae5955feedf2fe25f1737f91fa7f
Successfully built protego Py

In [ ]:
import time
import requests
from selenium import webdriver
from selenium.webdriver.support.ui import Select
from bs4 import BeautifulSoup
from scrapy import Selector
import pandas as pd
from pandas import json_normalize

def valores(lista):
    valores = ''
    for value in lista:
        valores += value +'\n'
    return valores
  

options = webdriver.ChromeOptions()
options.add_argument('headless')
options.add_argument('window-size=540x480')
driver = webdriver.Chrome(executable_path='C:/Users/emere/Documents/chromedriver_win32/chromedriver.exe')
driver.get('https://www.fleury.com.br/exames')

time.sleep(5)

# clicando no botão todos os exames
driver.find_element_by_xpath('//*[@id="button-all-exams"]').click()

time.sleep(10) # precisa de um tempo maior para carregar pagina
# selecionando mostrar 100 exames
select =  Select(driver.find_element_by_xpath('//select[@class="selectcomponentstyle__SelectStyled-sc-8aqv0u-1 gaSZLv"]'))
select.select_by_visible_text('100')
select.select_by_value('100')

time.sleep(5)

# clicando no botão proximo
proximo = driver.find_element_by_xpath('//*[@class="paginationcomponentstyle__PaginationButtonStyled-sc-1nv42op-1 iyLynz"]')

lista_hrefs = []

# Salvando todos os links de cada exame
page_num = 1
while page_num <= 27:
    page_num += 1
    time.sleep(5)
    hrefs = driver.find_elements_by_css_selector(".iYigXx [href]")
    links = [elem.get_attribute('href') for elem in hrefs]
    lista_hrefs.append([link for link in links])
    try:
        proximo.click()
    except:
        pass   

# lendo todos os exames
exames = []
for listas in lista_hrefs:
    for exame in listas:
        exames.append(exame)

# Salvando a descrição de cada exame
exames = {}
paginas = {'Não encontrada' : 0}

for url in lista:
    new_html = requests.get(url).content
    seletor = Selector (text = new_html)
    titulo_exame = seletor.xpath('//h1[@class = "typographycomponentstyle__H1-sc-1oox73n-1 lomAMa"]/text()').extract_first()
    if titulo_exame == 'Página não encontrada':
        print('Pagina Não Encontrada')
        paginas['Não encontrada'] += 1
        pass
    else:
        outros_nomes = seletor.xpath('//div[@class = "sc-htpNat gridcomponent__ColStyled-sc-8zg10d-2 dsZazA"]/p/text()').extract()
        outros_nomes_ext = [outros for outros in outros_nomes]
        exames[titulo_exame] =  valores(outros_nomes)


print(paginas)
lista_exames = pd.json_normalize(exames)
lista_exames.to_excel('/content/excel.xlsx', index=False)


In [4]:
lista = ['https://www.fleury.com.br/exames/12-dihidroxi-4-n-acetilcistenil-butano-urina',
'https://www.fleury.com.br/exames/1011-epoxido-carbamazepina-soro',
'https://www.fleury.com.br/exames/11-desoxicorticosterona-soro',
'https://www.fleury.com.br/exames/11-desoxicortisol-apos-estimulo-com-acth-soro',
'https://www.fleury.com.br/exames/11-desoxicortisol-soro',
'https://www.fleury.com.br/exames/17-alfa-hidroxi-progesterona-soro',
'https://www.fleury.com.br/exames/17-alfa-hidroxi-progesterona-apos-estimulo-com-acth-soro',
'https://www.fleury.com.br/exames/17-hidroxi-pregnenolona-apos-estimulo-com-acth-soro',
'https://www.fleury.com.br/exames/17-hidroxi-pregnenolona-soro',
'https://www.fleury.com.br/exames/18-hidroxi-corticosterona-soro',
'https://www.fleury.com.br/exames/25-hexanodiona-urina',
'https://www.fleury.com.br/exames/21-hidroxilase-enzima-anticorpo-soro',
'https://www.fleury.com.br/exames/21-desoxicortisol-soro',
'https://www.fleury.com.br/exames/5-nucleotidase-soro',
'https://www.fleury.com.br/exames/acth-plasma',
'https://www.fleury.com.br/exames/adamts13-atividade-e-inibidor-plasma',
'https://www.fleury.com.br/exames/alfa-glicosidade-papel-de-filtro',
'https://www.fleury.com.br/exames/analise-de-amiloidose-por-proteomica-bloco-de-parafina',
'https://www.fleury.com.br/exames/abatacept-orencia-1000mg-medicamento-e-aplicacao-endovenoso',
'https://www.fleury.com.br/exames/abatacept-orencia-125mg-medicamento-e-aplicacao-subcutaneo',
'https://www.fleury.com.br/exames/abatacept-orencia-500mg-medicamento-e-aplicacao-endovenoso',
'https://www.fleury.com.br/exames/abatacept-orencia-750mg-medicamento-e-aplicacao-endovenoso',
'https://www.fleury.com.br/exames/abatacept-orencia-apenas-aplicacao-subcutaneo',
'https://www.fleury.com.br/exames/abatacept-orencia-apenas-aplicacao-endovenoso',
'https://www.fleury.com.br/exames/acanthamoeba-pesquisa-varios-materiais',
'https://www.fleury.com.br/exames/acetato-de-leuprorrelina-lectrum-375mg-med-e-aplic-intramuscular',
'https://www.fleury.com.br/exames/acetato-de-leuprorrelina-lectrum-75mg-med-e-aplic-intramuscular',
'https://www.fleury.com.br/exames/acetato-de-leuprorrelina-lupron-depot-1125mg-med-e-aplic-intramuscular',
'https://www.fleury.com.br/exames/acetato-de-leuprorrelina-lupron-depot-15mg-med-e-aplic-intramuscular',
'https://www.fleury.com.br/exames/acetato-de-leuprorrelina-lupron-depot-375mg-med-e-aplic-intramuscular',
'https://www.fleury.com.br/exames/acetato-de-leuprorrelina-lupron-depot-75mg-med-e-aplic-intramuscular',
'https://www.fleury.com.br/exames/acetato-de-leuprorrelina-lectrum-apenas-aplicacao-intramuscular',
'https://www.fleury.com.br/exames/acetato-de-leuprorrelina-lupron-depot-apenas-aplicacao-intramuscular',
'https://www.fleury.com.br/exames/acetato-de-gosserelina-zoladex-36mg-medicamento-e-aplicacao-subcutaneo',
'https://www.fleury.com.br/exames/acetato-de-gosserelina-zoladex-108mg-medicamento-e-aplicacao-subcutaneo',
'https://www.fleury.com.br/exames/acetato-de-gosserelina-zoladex-apenas-aplicacao-subcutaneo',
'https://www.fleury.com.br/exames/acetato-de-octreotida-sandostatin-lar-10mg-med-e-aplicacao-intramuscular',
'https://www.fleury.com.br/exames/acetato-de-octreotida-sandostatin-lar-20mg-med-e-aplicacao-intramuscular',
'https://www.fleury.com.br/exames/acetato-de-octreotida-sandostatin-lar-30mg-med-e-aplicacao-intramuscular',
'https://www.fleury.com.br/exames/acetato-de-octreotida-sandostatin-lar-apenas-aplicacao-intramuscular',
'https://www.fleury.com.br/exames/acetilcolina-anticorpos-moduladores-do-receptor-soro',
'https://www.fleury.com.br/exames/acetilcolina-ligadores-do-receptor-anticorpos-soro',
'https://www.fleury.com.br/exames/acetilcolina-anticorpos-bloqueadores-do-receptor-soro',
'https://www.fleury.com.br/exames/acetona-sangue-total',
'https://www.fleury.com.br/exames/acetona-urina',
'https://www.fleury.com.br/exames/acidemia-matilmalonica-tipo-c-analise-molecular-sangue-total',
'https://www.fleury.com.br/exames/acidificacao-urinaria-prova-de-sangue-total-e-urina',
'https://www.fleury.com.br/exames/acido-zoledronico-aclasta-endovenoso',
'https://www.fleury.com.br/exames/acido-zoledronico-aclasta-endovenoso-somente-aplicacao',
'https://www.fleury.com.br/exames/acilcarnitinas-perfil-de-plasma',
'https://www.fleury.com.br/exames/acompanhamento-paciente-autismo',
'https://www.fleury.com.br/exames/acufenometria',
'https://www.fleury.com.br/exames/acuidade-visual-a-laser-olho-direito',
'https://www.fleury.com.br/exames/acuidade-visual-a-laser-olho-esquerdo',
'https://www.fleury.com.br/exames/acuidade-visual-a-laser-olhos-ambos',
'https://www.fleury.com.br/exames/acuidade-visual-teste-com-cartoes-de-teller',
'https://www.fleury.com.br/exames/acuidade-visual-teste-de',
'https://www.fleury.com.br/exames/adalimumabe-160mg-humira-subcutaneo',
'https://www.fleury.com.br/exames/adalimumabe-40mg-humira-subcutaneo',
'https://www.fleury.com.br/exames/adalimumabe-80mg-humira-subcutaneo',
'https://www.fleury.com.br/exames/adalimumabe-humira-20mg-medicamento-e-aplicacao-subcutaneo',
'https://www.fleury.com.br/exames/adalimumabe-humira-subcutaneo',
'https://www.fleury.com.br/exames/adalimumabe-humira-somente-aplicacao-subcutaneo',
'https://www.fleury.com.br/exames/adenosina-deaminase-atividade-liquor',
'https://www.fleury.com.br/exames/adenosina-deaminase-varios-materiais',
'https://www.fleury.com.br/exames/adenovirus-anticorpos-igg-e-igm-soro',
'https://www.fleury.com.br/exames/adenovirus-deteccao-quantitativa-do-dna-por-pcr-varios-materiais',
'https://www.fleury.com.br/exames/adiponectina-dosagem-soro',
'https://www.fleury.com.br/exames/administracao-de-medicamentos-ambulatoriais-1',
'https://www.fleury.com.br/exames/administracao-de-medicamentos-ambulatoriais-2',
'https://www.fleury.com.br/exames/adrenoleucodistrofia-pesquisa-varios-materiais',
'https://www.fleury.com.br/exames/aglutininas-anti-rh-soro',
'https://www.fleury.com.br/exames/agregacao-plaquetaria-avaliacao-sangue-total',
'https://www.fleury.com.br/exames/agulhamento-por-mamografia-ou-ultrassonografia',
'https://www.fleury.com.br/exames/albumina-liquor',
'https://www.fleury.com.br/exames/albumina-soro',
'https://www.fleury.com.br/exames/albuminuria-amostra-isolada-urina',
'https://www.fleury.com.br/exames/albuminuria-urina',
'https://www.fleury.com.br/exames/alcoolizacao-de-cisto-renal-feminino-guiada-por-ultrassom',
'https://www.fleury.com.br/exames/alcoolizacao-de-cisto-renal-masculino-guiada-por-ultrassom',
'https://www.fleury.com.br/exames/alcoolizacao-puncao-cistica',
'https://www.fleury.com.br/exames/aldolase-soro',
'https://www.fleury.com.br/exames/aldosterona-apos-estimulo-de-captopril-soro',
'https://www.fleury.com.br/exames/aldosterona-apos-infusao-de-salina-soro',
'https://www.fleury.com.br/exames/aldosterona-para-prova-de-postura',
'https://www.fleury.com.br/exames/aldosterona-soro',
'https://www.fleury.com.br/exames/aldosterona-urina',
'https://www.fleury.com.br/exames/alfa-1-glicoproteina-acida-soro',
'https://www.fleury.com.br/exames/alfa-fetoproteina-liquor',
'https://www.fleury.com.br/exames/alfa-fetoproteina-soro',
'https://www.fleury.com.br/exames/alfa-1-antitripsina-fenotipagem-soro',
'https://www.fleury.com.br/exames/alfa-1-antitripsina-fezes',
'https://www.fleury.com.br/exames/alfa-1-antitripsina-soro',
'https://www.fleury.com.br/exames/alfa-2-antiplasmina-plasma',
'https://www.fleury.com.br/exames/alfa-2-macroglobulina-soro',
'https://www.fleury.com.br/exames/alfa-galactosidase-papel-de-filtro',
'https://www.fleury.com.br/exames/alfa-l-iduronase-papel-de-filtro',
'https://www.fleury.com.br/exames/alfa-talassemia-analise-molecular-sangue-total',
'https://www.fleury.com.br/exames/aluminio-dosagem-pre-e-pos-administracao-de-quelante-plasma',
'https://www.fleury.com.br/exames/aluminio-dosagem-plasma',
'https://www.fleury.com.br/exames/aluminio-urina',
'https://www.fleury.com.br/exames/amicacina-soro',
'https://www.fleury.com.br/exames/amilase-depuracao-soro-e-urina',
'https://www.fleury.com.br/exames/amilase-isoenzimas-soro',
'https://www.fleury.com.br/exames/amilase-amostra-isolada-urina',
'https://www.fleury.com.br/exames/amilase-soro',
'https://www.fleury.com.br/exames/amilase-urina',
'https://www.fleury.com.br/exames/amiloidose-familiar-analise-molecular-varios-materiais',
'https://www.fleury.com.br/exames/aminoacidos-quantitativo-liquor',
'https://www.fleury.com.br/exames/aminoacidos-quantitativo-plasma',
'https://www.fleury.com.br/exames/aminoacidos-quantitativo-urina',
'https://www.fleury.com.br/exames/amiodarona-soro',
'https://www.fleury.com.br/exames/amitriptilina-dosagem-soro',
'https://www.fleury.com.br/exames/amigdalas-cultura-de-varios-materiais',
'https://www.fleury.com.br/exames/amonia-plasma',
'https://www.fleury.com.br/exames/anal-swab-regiao-peri-anal',
'https://www.fleury.com.br/exames/analise-do-exon-12-do-jak2-varios-materiais',
'https://www.fleury.com.br/exames/anatomopatologico-peca-pequena-varios-materiais',
'https://www.fleury.com.br/exames/anatomopatologico-peca-radical',
'https://www.fleury.com.br/exames/androstenediona-soro',
'https://www.fleury.com.br/exames/anestesia-para-procedimentos',
'https://www.fleury.com.br/exames/anestesia-para-procedimentos-de-otorrino',
'https://www.fleury.com.br/exames/anestesia-para-procedimentos-de-tempo-curto',
'https://www.fleury.com.br/exames/anestesia-para-procedimentos-de-tempo-medio',
'https://www.fleury.com.br/exames/anestesia-para-procedimentos-de-tempo-prolongado-1',
'https://www.fleury.com.br/exames/anestesia-para-procedimentos-de-tempo-prolongado-2',
'https://www.fleury.com.br/exames/anfetaminas-pesquisa-urina',
'https://www.fleury.com.br/exames/angiofluoresceinografia-olho-direito',
'https://www.fleury.com.br/exames/angiofluoresceinografia-olho-esquerdo',
'https://www.fleury.com.br/exames/angiofluoresceinografia-olhos-ambos',
'https://www.fleury.com.br/exames/angiografia-com-indocianina-verde-olho-direito',
'https://www.fleury.com.br/exames/angiografia-com-indocianina-verde-olho-esquerdo',
'https://www.fleury.com.br/exames/angiografia-com-indocianina-verde-olhos-ambos',
'https://www.fleury.com.br/exames/angiorressonancia-magnetica-abdome',
'https://www.fleury.com.br/exames/angiorressonancia-magnetica-arterial-mesenterica-superior',
'https://www.fleury.com.br/exames/angiorressonancia-magnetica-arterias-renais',
'https://www.fleury.com.br/exames/angiorressonancia-magnetica-cerebral',
'https://www.fleury.com.br/exames/angiorressonancia-magnetica-veia-porta',
'https://www.fleury.com.br/exames/angiotensina-ii-plasma',
'https://www.fleury.com.br/exames/angiotensina-enzima-conversora-soro',
'https://www.fleury.com.br/exames/angiotomografia-cerebral',
'https://www.fleury.com.br/exames/angiotomografia-arterial-de-vasos-cervicais',
'https://www.fleury.com.br/exames/angiotomografia-das-arterias-coronarias',
'https://www.fleury.com.br/exames/angiotomografia-de-coerencia-optica-olho-direito',
'https://www.fleury.com.br/exames/angiotomografia-de-coerencia-optica-olho-esquerdo',
'https://www.fleury.com.br/exames/angiotomografia-de-coerencia-optica-olhos-ambos',
'https://www.fleury.com.br/exames/angiotomografia-venosa-do-abdome-e-pelve',
'https://www.fleury.com.br/exames/angiotomografia-venosa-cranio',
'https://www.fleury.com.br/exames/angiotomografia-venosa-torax',
'https://www.fleury.com.br/exames/anidroglucitol-1-5-soro',
'https://www.fleury.com.br/exames/anti-c1q-anticorpos-soro',
'https://www.fleury.com.br/exames/anti-mulleriano-hormonio-soro',
'https://www.fleury.com.br/exames/anti-xa-para-heparina-nao-fracionada-hnf-plasma',
'https://www.fleury.com.br/exames/anti-xa-atividade-para-heparina-de-baixo-peso-molecular-plasma',
'https://www.fleury.com.br/exames/antibiograma-para-fungos-quantitativo-varios-materiais',
'https://www.fleury.com.br/exames/antibiograma-varios-materiais',
'https://www.fleury.com.br/exames/antibiograma-incluido-varios-materiais',
'https://www.fleury.com.br/exames/antibiograma-para-fungos-varios-materiais',
'https://www.fleury.com.br/exames/antibiograma-para-mycoplasma-varios-materiais',
'https://www.fleury.com.br/exames/antibiograma-para-microrganismos-anaerobios-varios-materiais',
'https://www.fleury.com.br/exames/anticoagulante-lupico-plasma',
'https://www.fleury.com.br/exames/anticorpo-gangliosideo-gq1b-igg-soro',
'https://www.fleury.com.br/exames/anticorpo-anti-iga-soro',
'https://www.fleury.com.br/exames/anticorpos-anti-ma1-e-anti-ma2-soro',
'https://www.fleury.com.br/exames/anticorpos-anti-receptor-de-nmda-soro',
'https://www.fleury.com.br/exames/anticorpos-anti-znt8-soro',
'https://www.fleury.com.br/exames/anticorpos-igg-anti-ige-soro',
'https://www.fleury.com.br/exames/anticorpos-anti-heparina-fator-plaquetario-4-soro',
'https://www.fleury.com.br/exames/anticorpos-anti-receptor-da-fosfolipade-a2-soro',
'https://www.fleury.com.br/exames/anticorpos-anti-ss2-glicoproteina-i-soro',
'https://www.fleury.com.br/exames/anticorpos-antifosfatidilserina/-protrombina-ps/pt-soro',
'https://www.fleury.com.br/exames/anticorpos-anticenpb-soro',
'https://www.fleury.com.br/exames/antiestreptolisina-o-aslo-soro',
'https://www.fleury.com.br/exames/antinucleossomo-anticorpos-soro',
'https://www.fleury.com.br/exames/antipeptideos-carbamilados-anticorpos-soro',
'https://www.fleury.com.br/exames/antitrombina-iii-plasma',
'https://www.fleury.com.br/exames/antigeno-carcinoembrionico-em-liquidos-biologicos-varios-materiais',
'https://www.fleury.com.br/exames/antigeno-carcinoembrionico-liquor',
'https://www.fleury.com.br/exames/antigeno-carcinoembrionico-soro',
'https://www.fleury.com.br/exames/antigeno-hepatico-soluvel-sla-autoanticorpos-soro',
'https://www.fleury.com.br/exames/antigeno-prostatico-especifico-soro',
'https://www.fleury.com.br/exames/antigeno-do-inibidor-do-ativador-de-plasminogenio-plasma',
'https://www.fleury.com.br/exames/antigenos-inalatorios-anticorpos-soro',
'https://www.fleury.com.br/exames/antigenos-nucleares-extraiveis-anticorpos-soro',
'https://www.fleury.com.br/exames/anuscopia',
'https://www.fleury.com.br/exames/analise-cito-histologica-de-biopsia-dirigida-por-procedimento-de-imagem',
'https://www.fleury.com.br/exames/analise-de-delecoes-associadas-a-distrofia-facio-escapulo-umeral-sangue-total',
'https://www.fleury.com.br/exames/analise-de-delecao/duplicacao-do-ids-mucopolissacaridose-tipo-ii-sangue-total',
'https://www.fleury.com.br/exames/analise-de-delecao/duplicacao-do-gene-abcd1adrenoleucodistro-sangue-total',
'https://www.fleury.com.br/exames/analise-de-hipermutacao-somatica-do-igh-para-llc-varios-materiais',
'https://www.fleury.com.br/exames/analise-de-mutacoes-do-gene-c-kit-varios-materiais',
'https://www.fleury.com.br/exames/analise-delecao/duplicacao-gene-ids-mucopolissacaridose-tipo-ii-sangue-total',
'https://www.fleury.com.br/exames/apolipoproteina-a-1-soro',
'https://www.fleury.com.br/exames/apolipoproteina-b-soro',
'https://www.fleury.com.br/exames/aquaporina-4-aqp4-anticorpo-igg-soro',
'https://www.fleury.com.br/exames/arsenico-sangue-total',
'https://www.fleury.com.br/exames/arsenico-urina',
'https://www.fleury.com.br/exames/artro-rm',
'https://www.fleury.com.br/exames/artrografia',
'https://www.fleury.com.br/exames/artrografia-por-ressonancia-magnetica',
'https://www.fleury.com.br/exames/artroscopia-do-joelho-com-sutura-meniscal',
'https://www.fleury.com.br/exames/artrotomografia-computadorizada',
'https://www.fleury.com.br/exames/aspergillus-anticorpos-totais-soro',
'https://www.fleury.com.br/exames/aspergillus-pesquisa-qualitativa-do-dna-varios-materiais',
'https://www.fleury.com.br/exames/aspergillus-pesquisa-do-antigeno-galactomanana-varios-materiais',
'https://www.fleury.com.br/exames/ataxia-de-friedreich-teste-molecular-sangue-total',
'https://www.fleury.com.br/exames/ataxias-espinocerebelares-sangue-total',
'https://www.fleury.com.br/exames/ataxias-hereditarias-sangue-total',
'https://www.fleury.com.br/exames/atividade-antixa-para-o-medicamento-rivaroxaban-plasma',
'https://www.fleury.com.br/exames/atividade-plasmatica-de-renina-apos-captopril-plasma',
'https://www.fleury.com.br/exames/atividade-plasmatica-de-renina-apos-estimulo-com-furosemida-plasma',
'https://www.fleury.com.br/exames/atrofia-dentato-rubro-palido-luysiana-drpla-sangue-total',
'https://www.fleury.com.br/exames/audiometria-infantil',
'https://www.fleury.com.br/exames/audiometria-adulto',
'https://www.fleury.com.br/exames/audiometria-de-tronco-encefalico-infantil',
'https://www.fleury.com.br/exames/audiometria-de-altas-frequencias',
'https://www.fleury.com.br/exames/audiometria-de-tronco-encefalico',
'https://www.fleury.com.br/exames/avaliacao-da-funcao-auditiva-central',
'https://www.fleury.com.br/exames/avaliacao-da-mutacao-valina-para-metionina-pval30met-do-gene-ttr',
'https://www.fleury.com.br/exames/avaliacao-das-doencas-do-miocardio-ci-cardio-neuro',
'https://www.fleury.com.br/exames/avaliacao-de-fratura-vertebral',
'https://www.fleury.com.br/exames/avaliacao-de-alteracoes-geneticas-previamente-identificadas-varios-materiais',
'https://www.fleury.com.br/exames/avaliacao-do-funcionamento-de-marcapasso-implantado',
'https://www.fleury.com.br/exames/avaliacao-funcional-da-via-alternativa-sistema-complemento-soro',
'https://www.fleury.com.br/exames/avaliacao-transvaginal-do-colo-uterino-na-gestacao',
'https://www.fleury.com.br/exames/bcr-abl-tipo-p210-quantificacao-por-tempo-real-por-pcr-varios-materiais',
'https://www.fleury.com.br/exames/bcr-abl-hibridacao-in-situ-medula-ossea',
'https://www.fleury.com.br/exames/bcr-abl-hibridacao-in-situ-sangue-periferico',
'https://www.fleury.com.br/exames/bcr-abl-tipo-p190-e-p210-quanti-por-rt-pcr-em-tempo-real-varios-materiais',
'https://www.fleury.com.br/exames/bcr-abl-deteccao-de-mutacoes-do-gene-varios-materiais',
'https://www.fleury.com.br/exames/bk-pesquisa-varios-materiais',
'https://www.fleury.com.br/exames/brca1-e-brca2-mutacao-dos-genes-pesquisa-sangue-total',
'https://www.fleury.com.br/exames/brca1-e-brca2-sequenciamento-ngs-varios-materiais',
'https://www.fleury.com.br/exames/brca1-mutacao-do-gene-sangue-total',
'https://www.fleury.com.br/exames/brca1-pesquisa-de-mutacao-especifica-familiar-para-gene-sangue-total',
'https://www.fleury.com.br/exames/brca1mlpa-analise-separada-de-duplicacoes-e-insercoes-mlpa-sangue-total',
'https://www.fleury.com.br/exames/brca2-mutacao-do-gene-sangue-total',
'https://www.fleury.com.br/exames/brca2fam-pesquisa-de-mutacao-especifica-familiar-para-gene-sangue-total',
'https://www.fleury.com.br/exames/brca2mlpa-insercoes-e-delecoes-no-gene-brca2-sangue-total',
'https://www.fleury.com.br/exames/bacilo-de-hansen-varios-materiais',
'https://www.fleury.com.br/exames/bacterioscopico-varios-materiais',
'https://www.fleury.com.br/exames/bacterioscopico-conjuntival-varios-materiais',
'https://www.fleury.com.br/exames/bacterioscopico-escarro',
'https://www.fleury.com.br/exames/bacterioscopico-esperma',
'https://www.fleury.com.br/exames/bacterioscopico-feminina-secrecao-uretral',
'https://www.fleury.com.br/exames/bacterioscopico-fossa-nasal-varios-materiais',
'https://www.fleury.com.br/exames/bacterioscopico-geral-varios-materiais',
'https://www.fleury.com.br/exames/bacterioscopico-liquor',
'https://www.fleury.com.br/exames/bacterioscopico-nasofaringe',
'https://www.fleury.com.br/exames/bacterioscopico-orofaringe',
'https://www.fleury.com.br/exames/bacterioscopico-ouvido-varios-materiais',
'https://www.fleury.com.br/exames/bacterioscopico-para-helicobacter-pylori-varios-materiais',
'https://www.fleury.com.br/exames/bacterioscopico-secrecao-prostatica',
'https://www.fleury.com.br/exames/bacterioscopico-secrecao-uretral-masculina',
'https://www.fleury.com.br/exames/bacterioscopico-urina-jato-medio',
'https://www.fleury.com.br/exames/bacterioscopico-urina-primeiro-jato',
'https://www.fleury.com.br/exames/bacterioscopico-urina-sem-especificacao-de-jato',
'https://www.fleury.com.br/exames/bacterioscopico-vaginal-varios-materiais',
'https://www.fleury.com.br/exames/bandas-oligoclonais-pesquisa-liquor',
'https://www.fleury.com.br/exames/barbituricos-pesquisa-urina',
'https://www.fleury.com.br/exames/barreira-hemato-encefalica-estudo-da-liquor',
'https://www.fleury.com.br/exames/bartonella-henselae-e-bartonella-quintana-anticorpos-igg-e-igm-soro',
'https://www.fleury.com.br/exames/belimumabe-benlysta-1080mg-medicamento-e-aplicacao-endovenoso',
'https://www.fleury.com.br/exames/belimumabe-benlysta-1200mg-medicamento-e-aplicacao-endovenoso',
'https://www.fleury.com.br/exames/belimumabe-benlysta-360mg-medicamento-e-aplicacao-endovenoso',
'https://www.fleury.com.br/exames/belimumabe-benlysta-480mg-medicamento-e-aplicacao-endovenoso',
'https://www.fleury.com.br/exames/belimumabe-benlysta-600mg-medicamento-e-aplicacao-endovenoso',
'https://www.fleury.com.br/exames/belimumabe-benlysta-720mg-medicamento-e-aplicacao-endovenoso',
'https://www.fleury.com.br/exames/belimumabe-benlysta-840mg-medicamento-e-aplicacao-endovenoso',
'https://www.fleury.com.br/exames/belimumabe-benlysta-960mg-medicamento-e-aplicacao-endovenoso',
'https://www.fleury.com.br/exames/belimumabe-benlysta-fracao-1mg-iv',
'https://www.fleury.com.br/exames/belimumabe-benlysta-apenas-aplicacao-ev',
'https://www.fleury.com.br/exames/bence-jones-proteinas-de-urina',
'https://www.fleury.com.br/exames/benzodiazepinicos-pesquisa-urina',
'https://www.fleury.com.br/exames/berilio-sangue',
'https://www.fleury.com.br/exames/beta-2-glicoproteina-1-autoanticorpo-iga-soro',
'https://www.fleury.com.br/exames/beta-2-glicoproteina-1-autoanticorpos-soro',
'https://www.fleury.com.br/exames/beta-2-microglobulina-soro',
'https://www.fleury.com.br/exames/beta-2-microglobulina-urina',
'https://www.fleury.com.br/exames/beta-glicosidase-papel-de-filtro',
'https://www.fleury.com.br/exames/beta-hidroxibutirato-soro',
'https://www.fleury.com.br/exames/beta-talassemia-analise-molecular-sangue-total',
'https://www.fleury.com.br/exames/bicarbonato-varios-materiais',
'https://www.fleury.com.br/exames/bicarbonato-urina-amostra-isolada',
'https://www.fleury.com.br/exames/bilirrubinas-soro',
'https://www.fleury.com.br/exames/bilirrubinas-total-varios-materiais',
'https://www.fleury.com.br/exames/biodisponivel-para-testobio-soro',
'https://www.fleury.com.br/exames/biofeedback-anorretal',
'https://www.fleury.com.br/exames/biomarcadores-para-doenca-de-alzheimer-liquor',
'https://www.fleury.com.br/exames/biomarcadores-para-pre-eclampsia-soro',
'https://www.fleury.com.br/exames/biometria-ultra-sonica-olho-direito',
'https://www.fleury.com.br/exames/biometria-ultra-sonica-olho-esquerdo',
'https://www.fleury.com.br/exames/biometria-ultra-sonica-olhos-ambos',
'https://www.fleury.com.br/exames/biometria-optica-por-iol-master-olho-direito',
'https://www.fleury.com.br/exames/biometria-optica-por-iol-master-olho-esquerdo',
'https://www.fleury.com.br/exames/biometria-optica-por-iol-master-olhos-ambos',
'https://www.fleury.com.br/exames/biopsia-de-mama-dirigida-por-ultra-sonografia',
'https://www.fleury.com.br/exames/biopsia-de-mama-dirigida-por-ressonancia-magnetica',
'https://www.fleury.com.br/exames/biopsia-transretal-prostata',
'https://www.fleury.com.br/exames/biopsia-medula-ossea-biopsia-de-medula-ossea',
'https://www.fleury.com.br/exames/biopsia-anal',
'https://www.fleury.com.br/exames/biopsia-de-colo-uterino',
'https://www.fleury.com.br/exames/biopsia-de-endometrio-por-histeroscopia',
'https://www.fleury.com.br/exames/biopsia-de-vagina',
'https://www.fleury.com.br/exames/biopsia-de-vulva',
'https://www.fleury.com.br/exames/biopsia-muscular',
'https://www.fleury.com.br/exames/biopsia-por-peniscopia',
'https://www.fleury.com.br/exames/biopsia-vesical',
'https://www.fleury.com.br/exames/biotina-vitamina-b7-soro',
'https://www.fleury.com.br/exames/biotinidase-soro',
'https://www.fleury.com.br/exames/biopsia-de-bacia-guiada-por-tomografia-computadorizada',
'https://www.fleury.com.br/exames/biopsia-de-braco-guiada-por-tomografia-computadorizada',
'https://www.fleury.com.br/exames/biopsia-de-coluna-cervical-ate-3-discos-ou-4-vertebras-guiada-por-tomografia',
'https://www.fleury.com.br/exames/biopsia-de-coluna-cervical-ate-4-discos-ou-5-vertebras-guiada-por-tomografia',
'https://www.fleury.com.br/exames/biopsia-de-coluna-cervical-ate-5-discos-ou-6-vertebras-guiada-por-tomografia',
'https://www.fleury.com.br/exames/biopsia-de-coluna-dorsal-ate-3-discos-ou-4-vertebras-guiada-por-tomografia',
'https://www.fleury.com.br/exames/biopsia-de-coluna-dorsal-ate-4-discos-ou-5-vertebras-guiada-por-tomografia',
'https://www.fleury.com.br/exames/biopsia-de-coluna-dorsal-ate-5-discos-ou-6-vertebras-guiada-por-tomografia',
'https://www.fleury.com.br/exames/biopsia-de-coluna-lombar-ate-3-discos-ou-4-vertebras-guiada-por-tomografia',
'https://www.fleury.com.br/exames/biopsia-de-coluna-lombar-ate-4-discos-ou-5-vertebras-guiada-por-tomografia',
'https://www.fleury.com.br/exames/biopsia-de-coluna-lombar-ate-5-discos-ou-6-vertebras-guiada-por-tomografia',
'https://www.fleury.com.br/exames/biopsia-de-cotovelo-guiada-por-tomografia-computadorizada',
'https://www.fleury.com.br/exames/biopsia-de-coxa-guiada-por-tomografia-computadorizada',
'https://www.fleury.com.br/exames/biopsia-de-joelho-guiada-por-tomografia-computadorizada',
'https://www.fleury.com.br/exames/biopsia-de-mao-guiada-por-tomografia-computadorizada',
'https://www.fleury.com.br/exames/biopsia-de-ombro-guiada-por-tomografia-computadorizada',
'https://www.fleury.com.br/exames/biopsia-de-pelve-guiada-por-tomografia-computadorizada',
'https://www.fleury.com.br/exames/biopsia-de-perna-guiada-por-tomografia-computadorizada',
'https://www.fleury.com.br/exames/biopsia-de-punho-guiada-por-tomografia-computadorizada',
'https://www.fleury.com.br/exames/biopsia-de-pe-guiada-por-tomografia-computadorizada',
'https://www.fleury.com.br/exames/biopsia-de-quadril-guiada-por-tomografia-computadorizada',
'https://www.fleury.com.br/exames/biopsia-de-tornozelo-guiada-por-tomografia-computadorizada',
'https://www.fleury.com.br/exames/biopsia-de-antebraco-guiada-por-tomografia-computadorizada',
'https://www.fleury.com.br/exames/biopsia/puncao-de-mediastino-guiada-por-tomografia-computadorizada',
'https://www.fleury.com.br/exames/biopsia/puncao-de-partes-moles-guiada-por-ultrassom',
'https://www.fleury.com.br/exames/bordetella-pertussis-e-parapertussis-deteccao-do-dna-por-pcr-varios-materiais',
'https://www.fleury.com.br/exames/bordetella-pertussis-anticorpos-igg-soro',
'https://www.fleury.com.br/exames/boro-dosagem-plasma-plasma',
'https://www.fleury.com.br/exames/borrelia-burgdorferi-doenca-de-lyme-varios-materiais',
'https://www.fleury.com.br/exames/borrelia-burgdorferi-anticorpos-totais-soro',
'https://www.fleury.com.br/exames/borrelia-burgdorferi-pesquisa-de-anticorpos-por-imunoblot-soro',
'https://www.fleury.com.br/exames/bromazepam-dosagem-soro',
'https://www.fleury.com.br/exames/brometo-soro',
'https://www.fleury.com.br/exames/brucella-abortus-anticorpos-totais-soro',
'https://www.fleury.com.br/exames/c-pneumoniae-e-m-pneumoniae-dna-por-pcr-em-tempo-real-varios-materiais',
'https://www.fleury.com.br/exames/c1-esterase-inibidor-soro',
'https://www.fleury.com.br/exames/c1q-componente-do-complemento-soro',
'https://www.fleury.com.br/exames/c2-componente-do-complemento-soro',
'https://www.fleury.com.br/exames/c3-componente-do-complemento-soro',
'https://www.fleury.com.br/exames/c4-componente-do-complemento-soro',
'https://www.fleury.com.br/exames/c5-componente-do-complemento-soro',
'https://www.fleury.com.br/exames/c6-componente-do-complemento-soro',
'https://www.fleury.com.br/exames/c7-componente-do-complemento-soro',
'https://www.fleury.com.br/exames/c8-compponente-do-complemento-soro',
'https://www.fleury.com.br/exames/c9-componente-do-complemento-soro',
'https://www.fleury.com.br/exames/ca-125-em-liquidos-biologicos-varios-materiais',
'https://www.fleury.com.br/exames/ca-125-soro',
'https://www.fleury.com.br/exames/ca-15-3-soro',
'https://www.fleury.com.br/exames/ca-19-9-em-liquidos-biologicos-varios-materiais',
'https://www.fleury.com.br/exames/ca-19-9-soro',
'https://www.fleury.com.br/exames/ca-27-29-soro',
'https://www.fleury.com.br/exames/ca-72-4-soro',
'https://www.fleury.com.br/exames/carboximaltose-ferrica-somente-aplicacao-ferinjectr',
'https://www.fleury.com.br/exames/cd34-varios-materiais',
'https://www.fleury.com.br/exames/cd4/cd8-varios-materiais',
'https://www.fleury.com.br/exames/cd56/16-imunofenotipagem-sangue-total',
'https://www.fleury.com.br/exames/certolizumabe-400mg-cimzia-subcutaneo',
'https://www.fleury.com.br/exames/cgh-array-microarray-sangue-total',
'https://www.fleury.com.br/exames/ckmb-massa-prova-rapida-sangue-total',
'https://www.fleury.com.br/exames/copeptina-soro',
'https://www.fleury.com.br/exames/cortisol-apos-alta-dose-de-dexametasona-soro',
'https://www.fleury.com.br/exames/cortisol-globulina-transportadora-soro',
'https://www.fleury.com.br/exames/cyp21-estudo-molecular-do-gene-sangue',
'https://www.fleury.com.br/exames/cadasil-analise-do-gene-notch3-teste-molecular-varios-materiais',
'https://www.fleury.com.br/exames/cadeias-kappa-e-lambda-livres-soro',
'https://www.fleury.com.br/exames/cafeina-soro',
'https://www.fleury.com.br/exames/calcitonina-apos-estimulo-com-calcio-soro',
'https://www.fleury.com.br/exames/calcitonina-dosagem-em-material-obtido-por-puncao-puncao',
'https://www.fleury.com.br/exames/calcitonina-soro',
'https://www.fleury.com.br/exames/calprotectina-dosagem-fezes',
'https://www.fleury.com.br/exames/campimetria-computadorizada-campo-visual-olho-direito',
'https://www.fleury.com.br/exames/campimetria-computadorizada-campo-visual-olho-esquerdo',
'https://www.fleury.com.br/exames/campimetria-computadorizada-campo-visual-olhos-ambos',
'https://www.fleury.com.br/exames/canabinoides-pesquisa-urina',
'https://www.fleury.com.br/exames/canakinumab-ilaris-300mg-medicamento-e-aplicacao-subcutaneo',
'https://www.fleury.com.br/exames/canakinumab-ilaris-150mg-medicamento-e-aplicacao-subcutaneo',
'https://www.fleury.com.br/exames/canakinumab-ilaris-apenas-aplicacao-subcutaneo',
'https://www.fleury.com.br/exames/canal-de-calcio-regulado-por-voltagem-anticorpos-soro',
'https://www.fleury.com.br/exames/cancer-de-prostata-programa-pep-bloco-de-parafina',
'https://www.fleury.com.br/exames/candida-albicans-anticorpos-totais-soro',
'https://www.fleury.com.br/exames/candidiase-mucocutanea-cronica-varios-materiais',
'https://www.fleury.com.br/exames/capilaroscopia-periungueal',
'https://www.fleury.com.br/exames/carbamazepina-soro',
'https://www.fleury.com.br/exames/carbohidratos-cromatografia-urina',
'https://www.fleury.com.br/exames/carboxi-hemoglobina-dosagem-sangue-total',
'https://www.fleury.com.br/exames/carboximaltose-ferrica-ferinject-500mg-medicamento-e-aplicacao-endovenoso',
'https://www.fleury.com.br/exames/carboximaltose-ferrica-ferinject-1000mg-medicamento-e-aplicacao-endovenoso',
'https://www.fleury.com.br/exames/carboximaltose-ferrica-ferinject-apenas-aplicacao',
'https://www.fleury.com.br/exames/cardiolipina-anticorpo-iga-soro',
'https://www.fleury.com.br/exames/cardiolipina-anticorpos-igg-e-igm-soro',
'https://www.fleury.com.br/exames/cardiotocografia-anteparto-gestacao-multipla',
'https://www.fleury.com.br/exames/cardiotocografia-anteparto',
'https://www.fleury.com.br/exames/cariotipo-com-banda-g-p/doencas-hematologicas-varios-materiais',
'https://www.fleury.com.br/exames/cariotipo-com-banda-g-liquido-amniotico',
'https://www.fleury.com.br/exames/cariotipo-com-banda-g-material-de-aborto',
'https://www.fleury.com.br/exames/cariotipo-com-banda-g-sangue-fetal',
'https://www.fleury.com.br/exames/cariotipo-com-banda-g-sangue-periferico',
'https://www.fleury.com.br/exames/cariotipo-com-banda-g-vilo-corial',
'https://www.fleury.com.br/exames/cariotipo-para-sindromes-de-quebras-cromossomicas-sangue-periferico',
'https://www.fleury.com.br/exames/carnitina-soro',
'https://www.fleury.com.br/exames/caroteno-dosagem-soro',
'https://www.fleury.com.br/exames/catecolaminas-amostra-isolada-urina',
'https://www.fleury.com.br/exames/catecolaminas-urina',
'https://www.fleury.com.br/exames/caxumba-anticorpos-igg-e-igm-soro',
'https://www.fleury.com.br/exames/ceratoscopia-computadorizada-da-cornea-olho-direito',
'https://www.fleury.com.br/exames/ceratoscopia-computadorizada-da-cornea-olho-esquerdo',
'https://www.fleury.com.br/exames/ceratoscopia-computadorizada-da-cornea-olhos-ambos',
'https://www.fleury.com.br/exames/certolizumabe-200mg-cimzia-subcutaneo',
'https://www.fleury.com.br/exames/certolizumabe-cimzia-subcutaneo-somente-aplicacao',
'https://www.fleury.com.br/exames/certolizumabe-cimzia-subcutaneo',
'https://www.fleury.com.br/exames/ceruloplasmina-soro',
'https://www.fleury.com.br/exames/chlamydia-trachomatis-anticorpo-igg-soro',
'https://www.fleury.com.br/exames/chlamydia-trachomatis-anticorpos-igm-soro',
'https://www.fleury.com.br/exames/chlamydia-trachomatis-deteccao-do-dna-por-pcr-varios-materiais',
'https://www.fleury.com.br/exames/chlamydia-trachomatis-exame-direto-para-varios-materiais',
'https://www.fleury.com.br/exames/chlamydophila-pneumoniae-anticorpos-igg-e-igm-soro',
'https://www.fleury.com.br/exames/chlamydophila-psittaci-anticorpos-soro',
'https://www.fleury.com.br/exames/chumbo-sangue-total',
'https://www.fleury.com.br/exames/chumbo-urina',
'https://www.fleury.com.br/exames/cianeto-sangue-total',
'https://www.fleury.com.br/exames/ciclosporina-sangue-total',
'https://www.fleury.com.br/exames/cintilografia-cardiaca-com-pirofosfato-para-diagnostico-de-amiloidose',
'https://www.fleury.com.br/exames/cintilografia-cerebral-com-trodat-tc-99m',
'https://www.fleury.com.br/exames/cintilografia-cerebral-com-cloreto-de-talio-201',
'https://www.fleury.com.br/exames/cintilografia-da-tiroide-com-tecnecio-99m',
'https://www.fleury.com.br/exames/cintilografia-de-corpo-inteiro-com-iodo-123',
'https://www.fleury.com.br/exames/cintilografia-de-corpo-inteiro-com-mibi-marcada-com-tecnecio-99m',
'https://www.fleury.com.br/exames/cintilografia-de-corpo-inteiro-com-metaiodobenzilguanidina-iodo-123',
'https://www.fleury.com.br/exames/cintilografia-de-corpo-inteiro-com-metaiodobenzilguanidina-iodo-131',
'https://www.fleury.com.br/exames/cintilografia-de-mamas-com-mibi-marcada-com-tecnecio-99m',
'https://www.fleury.com.br/exames/cintilografia-de-perfusao-do-miocardio-associada-ao-dipiridamol',
'https://www.fleury.com.br/exames/cintilografia-de-perfusao-do-miocardio-associada-ao-teste-ergometrico',
'https://www.fleury.com.br/exames/cintilografia-de-perfusao-do-miocardio-associada-ao-teste-cardiopulmonar',
'https://www.fleury.com.br/exames/cintilografia-de-perfusao-do-miocardio-associada-a-dobutamina',
'https://www.fleury.com.br/exames/cintilografia-de-perfusao-do-miocardio-associada-a-adenosina',
'https://www.fleury.com.br/exames/cintilografia-de-perfusao-do-miocardio-associada-a-bicicleta-estacionaria',
'https://www.fleury.com.br/exames/cintilografia-do-miocardio-com-cloreto-de-talio-201-estudo-da-viabilidade',
'https://www.fleury.com.br/exames/cintilografia-mamaria-para-localizacao-de-lesao-oculta',
'https://www.fleury.com.br/exames/cintilografia-renal-com-dmsa',
'https://www.fleury.com.br/exames/cintilografia-cerebral',
'https://www.fleury.com.br/exames/cintilografia-glandulas-salivares',
'https://www.fleury.com.br/exames/cintilografia-hepatica',
'https://www.fleury.com.br/exames/cintilografia-pulmonar-de-perfusao',
'https://www.fleury.com.br/exames/cintilografia-pulmonar-de-perfusao-e-inalacao',
'https://www.fleury.com.br/exames/cintilografia-receptores-da-somatostatina',
'https://www.fleury.com.br/exames/cintilografia-vias-biliares',
'https://www.fleury.com.br/exames/cintilografia-com-iodo-131-de-corpo-inteiro',
'https://www.fleury.com.br/exames/cintilografia-com-citrato-de-galio-67',
'https://www.fleury.com.br/exames/cintilografia-com-hemacias-marcadas-para-pesquisa-de-hemangioma',
'https://www.fleury.com.br/exames/cintilografia-de-corpo-inteiro-ossea',
'https://www.fleury.com.br/exames/cintilografia-dos-espacos-liquoricos',
'https://www.fleury.com.br/exames/cintilografia-dos-testiculos-para-deteccao-de-varicocele',
'https://www.fleury.com.br/exames/cintilografia-dos-testiculos',
'https://www.fleury.com.br/exames/cintilografia-e-captacao-da-tiroide',
'https://www.fleury.com.br/exames/cintilografia-para-pesquisa-de-refluxo-gastroesofagico-e-aspiracao-pulmonar',
'https://www.fleury.com.br/exames/cintilografia-para-pesquisa-de-esvaziamento-gastrico',
'https://www.fleury.com.br/exames/cintilografia-paratiroides',
'https://www.fleury.com.br/exames/cistatina-c-soro',
'https://www.fleury.com.br/exames/cisticercose-anticorpo-igg-soro',
'https://www.fleury.com.br/exames/cistina-dosagem-em-amostra-de-24-horas-urina',
'https://www.fleury.com.br/exames/cistina-dosagem-urina',
'https://www.fleury.com.br/exames/cistocintilografia-direta',
'https://www.fleury.com.br/exames/cistografia-indireta',
'https://www.fleury.com.br/exames/cistoscopia',
'https://www.fleury.com.br/exames/citalopram-soro',
'https://www.fleury.com.br/exames/citologia-hormonal-urina',
'https://www.fleury.com.br/exames/citologia-cervico-vaginal-em-base-liquida-material-vaginal-e-colo-uterino',
'https://www.fleury.com.br/exames/citologico-anatomia-patologica-varios-materiais',
'https://www.fleury.com.br/exames/citologico-conjuntival-varios-materiais',
'https://www.fleury.com.br/exames/citologico-contagem-diferencial-varios-materiais',
'https://www.fleury.com.br/exames/citologico-de-puncao-de-nodulo-varios-materiais',
'https://www.fleury.com.br/exames/citologico-esperma',
'https://www.fleury.com.br/exames/citologico-lesao',
'https://www.fleury.com.br/exames/citologico-liquido-sinovial',
'https://www.fleury.com.br/exames/citologico-nasal',
'https://www.fleury.com.br/exames/citologico-raspado-de-orofaringe',
'https://www.fleury.com.br/exames/citomegalovirus-anticorpo-igg-soro',
'https://www.fleury.com.br/exames/citomegalovirus-anticorpo-igm-soro',
'https://www.fleury.com.br/exames/citomegalovirus-anticorpos-igg-e-igm-soro',
'https://www.fleury.com.br/exames/citomegalovirus-antigenemia-sangue-total',
'https://www.fleury.com.br/exames/citomegalovirus-avidez-de-igg-soro',
'https://www.fleury.com.br/exames/citomegalovirus-deteccao-qualitativa-do-dna-por-pcr-varios-materiais',
'https://www.fleury.com.br/exames/citomegalovirus-isolamento-por-shell-vial-varios-materiais',
'https://www.fleury.com.br/exames/citomegalovirus-quantificacao-por-pcr-em-tempo-real-varios-materiais',
'https://www.fleury.com.br/exames/citoplasma-de-neutrofilos-anticorpos-totais-soro',
'https://www.fleury.com.br/exames/citosol-hepatico-tipo-1-anticorpos-totais-soro',
'https://www.fleury.com.br/exames/citrato-amostra-isolada-urina',
'https://www.fleury.com.br/exames/citrato-urina',
'https://www.fleury.com.br/exames/classif-molectumores-origem-primaria-desconhecidaincerta-varios-materiais',
'https://www.fleury.com.br/exames/classificador-de-nodulo-tiroidiano-por-micro-rna-laminas',
'https://www.fleury.com.br/exames/clobazam-dosagem-soro',
'https://www.fleury.com.br/exames/clomipramina-dosagem-soro',
'https://www.fleury.com.br/exames/clonalidade-para-celulas-b-deteccao-por-pcr-varios-materiais',
'https://www.fleury.com.br/exames/clonalidade-para-celulas-t-deteccao-por-pcr-sangue-total',
'https://www.fleury.com.br/exames/clonazepam-dosagem-soro',
'https://www.fleury.com.br/exames/clordiazepoxido-e-metabolitos-dosagem-soro',
'https://www.fleury.com.br/exames/cloreto-dosagem-urina',
'https://www.fleury.com.br/exames/cloro-amostra-isolada-urina',
'https://www.fleury.com.br/exames/cloro-soro',
'https://www.fleury.com.br/exames/clostridioides-clostridium-difficile-cultura-com-teste-toxigenicidade-fezes',
'https://www.fleury.com.br/exames/clostridioides-clostridium-difficile-pesquisa-de-toxinas-a-e-b-fezes',
'https://www.fleury.com.br/exames/clostridium-difficile-toxigenico-pesquisa-do-dna-fezes',
'https://www.fleury.com.br/exames/clozapina-dosagem-soro',
'https://www.fleury.com.br/exames/coagulacao-inibidores-triagem-plasma',
'https://www.fleury.com.br/exames/coagulograma-optica-sangue-total',
'https://www.fleury.com.br/exames/coagulograma-urgente-optica-sangue',
'https://www.fleury.com.br/exames/cobalto-sangue-total',
'https://www.fleury.com.br/exames/cobre-amostra-isolada-urina',
'https://www.fleury.com.br/exames/cobre-plasma',
'https://www.fleury.com.br/exames/cobre-urina',
'https://www.fleury.com.br/exames/cocaina-dosagem-sangue-total',
'https://www.fleury.com.br/exames/cocaina-pesquisa-urina',
'https://www.fleury.com.br/exames/coccidios-e-cryptosporidium-spp-pesquisa-de-fezes',
'https://www.fleury.com.br/exames/coenzima-q10-dosagem-plasma',
'https://www.fleury.com.br/exames/colangiografia-por-ressonancia-magnetica',
'https://www.fleury.com.br/exames/colangiografia-pos-operatoria',
'https://www.fleury.com.br/exames/colesterol-total-e-fracoes-soro',
'https://www.fleury.com.br/exames/colesterol-varios-materiais',
'https://www.fleury.com.br/exames/colesterol-soro',
'https://www.fleury.com.br/exames/colesterol-esteres-do-soro',
'https://www.fleury.com.br/exames/coli-patogenico-pesquisa-fezes',
'https://www.fleury.com.br/exames/colinesterase-soro',
'https://www.fleury.com.br/exames/colonoscopia',
'https://www.fleury.com.br/exames/colonoscopia-virtual-por-tomografia-computadorizada',
'https://www.fleury.com.br/exames/colonoscopia-com-preparo-em-casa',
'https://www.fleury.com.br/exames/colpocitologia-hormonal-1-coleta-material-vaginal-e-colo-uterino',
'https://www.fleury.com.br/exames/colpocitologia-hormonal-seriada-varios-materiais',
'https://www.fleury.com.br/exames/colpocitologia-oncotica-material-vaginal-e-colo-uterino',
'https://www.fleury.com.br/exames/colposcopia',
'https://www.fleury.com.br/exames/colageno-ensaio-de-ligacao-plasma',
'https://www.fleury.com.br/exames/complementacao-do-exame-exoma-por-sequenciamento-sanger',
'https://www.fleury.com.br/exames/complemento-total-soro',
'https://www.fleury.com.br/exames/complexo-mycobacterium-tuberculosis-dna-fixado-formol-fragmento-tecido',
'https://www.fleury.com.br/exames/composicao-corporal-para-adulto',
'https://www.fleury.com.br/exames/concentracao-inibitoria-minima-varios-materiais',
'https://www.fleury.com.br/exames/confirmacao-ecstasy-urina',
'https://www.fleury.com.br/exames/consulta-check-up-dermatologia',
'https://www.fleury.com.br/exames/consulta-check-up-clinico-geral',
'https://www.fleury.com.br/exames/consulta-medica-ambulatorial',
'https://www.fleury.com.br/exames/consulta-pre-teste-genetico',
'https://www.fleury.com.br/exames/consulta-para-procedimento-invasivo',
'https://www.fleury.com.br/exames/consulta-para-tratamento-de-cancer',
'https://www.fleury.com.br/exames/consultoria-em-medicina-fetal',
'https://www.fleury.com.br/exames/contraste-para-ressonancia-magnetica-com-excrecao-hepatobiliar',
'https://www.fleury.com.br/exames/coombs-direto-monoespecifico-varios-materiais',
'https://www.fleury.com.br/exames/coombs-direto-sangue-total',
'https://www.fleury.com.br/exames/coprologico-fezes',
'https://www.fleury.com.br/exames/coracao-anticorpos-totais-soro',
'https://www.fleury.com.br/exames/corpos-cetonicos-urina',
'https://www.fleury.com.br/exames/corpusculos-de-heinz-sangue-total',
'https://www.fleury.com.br/exames/corticosterona-soro',
'https://www.fleury.com.br/exames/cortisol-e-acth-apos-estimulo-com-crh-e-dexametasona-soro-e-plasma',
'https://www.fleury.com.br/exames/cortisol-e-acth-apos-estimulo-com-crh-soro-e-plasma',
'https://www.fleury.com.br/exames/cortisol-e-acth-apos-estimulo-com-ddavp-soro-e-plasma',
'https://www.fleury.com.br/exames/cortisol-ritmo-de-soro',
'https://www.fleury.com.br/exames/cortisol-apos-estimulo-com-1-%C2%B5g-de-acth-soro',
'https://www.fleury.com.br/exames/cortisol-apos-estimulo-com-acth-soro',
'https://www.fleury.com.br/exames/cortisol-apos-estimulo-com-glucagon-soro',
'https://www.fleury.com.br/exames/cortisol-apos-estimulo-de-insulina-soro-e-plasma',
'https://www.fleury.com.br/exames/cortisol-livre-urina',
'https://www.fleury.com.br/exames/cortisol-para-prova-de-postura',
'https://www.fleury.com.br/exames/cortisol-por-hplc-soro',
'https://www.fleury.com.br/exames/cortisol-saliva',
'https://www.fleury.com.br/exames/cortisol-soro',
'https://www.fleury.com.br/exames/cortisona-soro',
'https://www.fleury.com.br/exames/corynebacterium-minutissimum-cultura-especifica-varios-materiais',
'https://www.fleury.com.br/exames/corynebacterium-minutissimum-pesquisa-varios-materiais',
'https://www.fleury.com.br/exames/coreia-de-huntington-sangue-total',
'https://www.fleury.com.br/exames/cotinina-soro',
'https://www.fleury.com.br/exames/cotinina-urina',
'https://www.fleury.com.br/exames/coxiella-burnetii-anticorpos-soro',
'https://www.fleury.com.br/exames/coxsackie-a-anticorpos-totais-soro',
'https://www.fleury.com.br/exames/coxsackie-a-anticorpos-totais-liquor-liquor',
'https://www.fleury.com.br/exames/coxsackie-b-anticorpos-varios-materiais',
'https://www.fleury.com.br/exames/creatina-fosfoquinase-em-liquidos-biologicos-varios-materiais',
'https://www.fleury.com.br/exames/creatinina-teste-rapido-injecao-de-contraste-sangue-capilar',
'https://www.fleury.com.br/exames/creatinina-depuracao-soro-e-urina',
'https://www.fleury.com.br/exames/creatinina-dosagem-especial-soro',
'https://www.fleury.com.br/exames/creatinina-varios-materiais',
'https://www.fleury.com.br/exames/creatinina-amostra-isolada-urina',
'https://www.fleury.com.br/exames/creatinina-soro',
'https://www.fleury.com.br/exames/creatinina-urina',
'https://www.fleury.com.br/exames/creatino-quinase-eletroforese-de-isoenzimas-da-creatino-quinase-soro',
'https://www.fleury.com.br/exames/creatinoquinase-ck-total-soro',
'https://www.fleury.com.br/exames/creatinoquinase-isoenzima-mb-massa-plasma',
'https://www.fleury.com.br/exames/crioaglutininas-soro',
'https://www.fleury.com.br/exames/crioglobulinas-soro',
'https://www.fleury.com.br/exames/cromo-plasma',
'https://www.fleury.com.br/exames/cromo-urina',
'https://www.fleury.com.br/exames/cromogranina-a-dosagem-soro',
'https://www.fleury.com.br/exames/cromossomo-x-e-y-hibridacao-in-situ-medula-ossea',
'https://www.fleury.com.br/exames/cromossomo-x-e-y-hibridacao-in-situ-sangue-periferico',
'https://www.fleury.com.br/exames/cromossomo-y-microdelecoes-deteccao-por-pcr-sangue-total',
'https://www.fleury.com.br/exames/cryptococcus-neoformans-anticorpos-totais-soro',
'https://www.fleury.com.br/exames/cryptococcus-neoformans-antigeno-soluvel-liquor',
'https://www.fleury.com.br/exames/cryptococcus-neoformans-antigeno-soro',
'https://www.fleury.com.br/exames/cryptosporidium-antigeno-fezes',
'https://www.fleury.com.br/exames/cultura-geral-varios-materiais',
'https://www.fleury.com.br/exames/cultura-quantitativa-para-microrganismos-aerobios-varios-materiais',
'https://www.fleury.com.br/exames/cultura-com-sonicacao-varios-materiais',
'https://www.fleury.com.br/exames/cultura-de-urina-varios-materiais',
'https://www.fleury.com.br/exames/cultura-de-urina-apos-massagem-prostatica-urina-pos-massagem-prostatica',
'https://www.fleury.com.br/exames/cultura-de-urina-urina-colhida-por-sonda-vesical-de-alivio',
'https://www.fleury.com.br/exames/cultura-enterococcus-sp-resistente-a-vancomicina-varios-materiais',
'https://www.fleury.com.br/exames/cultura-staphylococcus-aureus-resistente-a-oxacilina-varios-materiais',
'https://www.fleury.com.br/exames/cultura-streptococcus-beta-hemolitico-do-grupo-b-varios-materiais',
'https://www.fleury.com.br/exames/cultura-conduto-auditivo-varios-materiais',
'https://www.fleury.com.br/exames/cultura-conjuntival-varios-materiais',
'https://www.fleury.com.br/exames/cultura-escarro',
'https://www.fleury.com.br/exames/cultura-esperma',
'https://www.fleury.com.br/exames/cultura-feminina-secrecao-uretral-feminino',
'https://www.fleury.com.br/exames/cultura-fezes',
'https://www.fleury.com.br/exames/cultura-fossa-nasal-varios-materiais',
'https://www.fleury.com.br/exames/cultura-liquor',
'https://www.fleury.com.br/exames/cultura-orofaringe',
'https://www.fleury.com.br/exames/cultura-para-acanthamoeba-varios-materiais',
'https://www.fleury.com.br/exames/cultura-para-anaerobios-varios-materiais',
'https://www.fleury.com.br/exames/cultura-para-bordetella-pertussis-varios-materiais',
'https://www.fleury.com.br/exames/cultura-para-campylobacter-jejuni-fezes',
'https://www.fleury.com.br/exames/cultura-para-fungos-varios-materiais',
'https://www.fleury.com.br/exames/cultura-para-fungos-liquor',
'https://www.fleury.com.br/exames/cultura-para-leptospira-varios-materiais',
'https://www.fleury.com.br/exames/cultura-para-micobacterias-varios-materiais',
'https://www.fleury.com.br/exames/cultura-para-micobacterias-automatizada-lavado-gastrico',
'https://www.fleury.com.br/exames/cultura-para-mycoplasma-varios-materiais',
'https://www.fleury.com.br/exames/cultura-para-trichomonas-varios-materiais',
'https://www.fleury.com.br/exames/cultura-para-vibrio-cholerae-varios-materiais',
'https://www.fleury.com.br/exames/cultura-para-yersinia-fezes',
'https://www.fleury.com.br/exames/cultura-para-bacilo-gram-negativo-multirresistente-varios-materiais',
'https://www.fleury.com.br/exames/cultura-secrecao-de-nasofaringe',
'https://www.fleury.com.br/exames/cultura-secrecao-prostatica',
'https://www.fleury.com.br/exames/cultura-secrecao-uretral',
'https://www.fleury.com.br/exames/cultura-urina-jato-medio',
'https://www.fleury.com.br/exames/cultura-urina-primeiro-jato',
'https://www.fleury.com.br/exames/cultura-urina-terceiro-jato',
'https://www.fleury.com.br/exames/cultura-vaginal-varios-materiais',
'https://www.fleury.com.br/exames/curetagem-de-canal-endocervical',
'https://www.fleury.com.br/exames/curva-glicemica-de-2-horas-plasma',
'https://www.fleury.com.br/exames/curva-glicemica-de-3-horas-plasma',
'https://www.fleury.com.br/exames/curva-glicemica-de-4-horas-plasma',
'https://www.fleury.com.br/exames/curva-glicemica-de-5-horas-plasma',
'https://www.fleury.com.br/exames/curva-glicemica-de-6-horas-plasma',
'https://www.fleury.com.br/exames/curva-insulinemica-endovenosa-soro-e-plasma',
'https://www.fleury.com.br/exames/curva-insulinemica-de-2-horas-soro-e-plasma',
'https://www.fleury.com.br/exames/curva-insulinemica-de-3-horas-soro-e-plasma',
'https://www.fleury.com.br/exames/curva-insulinemica-de-4-horas-soro-e-plasma',
'https://www.fleury.com.br/exames/curva-insulinemica-de-5-horas-soro-e-plasma',
'https://www.fleury.com.br/exames/curva-insulinemica-de-6-horas-soro-e-plasma',
'https://www.fleury.com.br/exames/curva-tensional-diaria-ambos-os-olhos',
'https://www.fleury.com.br/exames/cadmio-plasma',
'https://www.fleury.com.br/exames/cadmio-urina',
'https://www.fleury.com.br/exames/calcio-ionizado-soro',
'https://www.fleury.com.br/exames/calcio-dosagem-especial-soro',
'https://www.fleury.com.br/exames/calcio-varios-materiais',
'https://www.fleury.com.br/exames/calcio-amostra-isolada-urina',
'https://www.fleury.com.br/exames/calcio-soro',
'https://www.fleury.com.br/exames/calcio-urina',
'https://www.fleury.com.br/exames/calculo-urinario-analise-quimica-calculo-urinario',
'https://www.fleury.com.br/exames/calculo-da-estimativa-da-taxa-de-filtracao-glomerular-soro',
'https://www.fleury.com.br/exames/capsula-endoscopica',
'https://www.fleury.com.br/exames/coclea-autoanticorpos-soro',
'https://www.fleury.com.br/exames/d-xilose-prova-de-absorcao-plasma',
'https://www.fleury.com.br/exames/deteccao-de-virus-e-bacterias-do-trato-respiratori-por-pcr-varios-materiais',
'https://www.fleury.com.br/exames/deteccao-de-agentes-de-meningites-e-encefalites-pcr-varios-materiais',
'https://www.fleury.com.br/exames/deteccao-intraoperatoria-radioguiada-de-linfonodo',
'https://www.fleury.com.br/exames/deteccao-qualitativa-sars-cov-2-por-pcr-varios-materiais',
'https://www.fleury.com.br/exames/dna-anticorpos-totais-soro',
'https://www.fleury.com.br/exames/dnase-b-anticorpos-implantacao-soro',
'https://www.fleury.com.br/exames/dacriocistocintilografia',
'https://www.fleury.com.br/exames/dacriocistografia',
'https://www.fleury.com.br/exames/defecograma-para-adulto',
'https://www.fleury.com.br/exames/defecograma-para-crianca',
'https://www.fleury.com.br/exames/defeitos-da-imunidade-inata-varios-materiais',
'https://www.fleury.com.br/exames/defeitos-da-motilidade-dos-fagocitos-painel-genetico-varios-materiais',
'https://www.fleury.com.br/exames/defeitos-de-fagocitos-defeitos-da-explosao-respiratoria-varios-materiais',
'https://www.fleury.com.br/exames/defeitos-de-fagocitos-neutropenias-congenitas-varios-materiais',
'https://www.fleury.com.br/exames/deficiencia-de-alfa-1-antitripsina-varios-materiais',
'https://www.fleury.com.br/exames/degluticao-e-fonacao',
'https://www.fleury.com.br/exames/dehidroepiandrosterona-sulfato-soro',
'https://www.fleury.com.br/exames/dehidroepiandrosterona-apos-estimulo-com-acth-soro',
'https://www.fleury.com.br/exames/dehidroepiandrosterona-soro',
'https://www.fleury.com.br/exames/demodex-sp-pesquisa-de-raspado-lesao-cutanea',
'https://www.fleury.com.br/exames/demencia-frontotemporal-varios-materiais',
'https://www.fleury.com.br/exames/demencia-e-parkinson-varios-materiais',
'https://www.fleury.com.br/exames/dengue-anticorpo-igg-soro',
'https://www.fleury.com.br/exames/dengue-anticorpo-igm-soro',
'https://www.fleury.com.br/exames/dengue-antigeno-ns1-soro',
'https://www.fleury.com.br/exames/dengue-antigeno-sangue-total',
'https://www.fleury.com.br/exames/denosumabe-prolia-subcutaneo',
'https://www.fleury.com.br/exames/denosumabe-prolia-subcutaneo-somente-aplicacao',
'https://www.fleury.com.br/exames/densidade-urina',
'https://www.fleury.com.br/exames/densitometria-ossea-infantil',
'https://www.fleury.com.br/exames/densitometria-ossea-de-coluna-lombar-e-femur',
'https://www.fleury.com.br/exames/densitometria-ossea-de-antebraco',
'https://www.fleury.com.br/exames/densitometria-ossea-para-avaliacao-do-escore-trabecular-osseo',
'https://www.fleury.com.br/exames/dermatoscopia-mapeamento-corporal-total-para-controle-de-nevos',
'https://www.fleury.com.br/exames/descarboxilase-do-acido-glutamico-anticorpos-soro',
'https://www.fleury.com.br/exames/desidrogenase-lactica-varios-materiais',
'https://www.fleury.com.br/exames/desidrogenase-lactica-liquor',
'https://www.fleury.com.br/exames/desidrogenase-lactica-soro',
'https://www.fleury.com.br/exames/deteccao-da-mutacao-w515l-ou-w515k-no-gene-mpl-varios-materiais',
'https://www.fleury.com.br/exames/deteccao-de-enterovirus-por-pcr-em-tempo-real-varios-materiais',
'https://www.fleury.com.br/exames/deteccao-de-haemophilus-ducreyi-por-pcr-tempo-real-varios-materiais',
'https://www.fleury.com.br/exames/deteccao-de-streptococcus-por-metodo-molecular-varios-materiais',
'https://www.fleury.com.br/exames/deteccao-de-treponema-pallidum-pcr-em-tempo-real-varios-materiais',
'https://www.fleury.com.br/exames/deteccao-de-trichomonas-vaginalis-pcr-em-tempo-real-varios-materiais',
'https://www.fleury.com.br/exames/deteccao-de-mayaro-virus-por-pcr-em-tempo-real-varios-materiais',
'https://www.fleury.com.br/exames/deteccao-de-mutacoes-no-gene-egfr-biopsia',
'https://www.fleury.com.br/exames/deteccao-de-ureaplasma-por-metodo-molecular-varios-materiais',
'https://www.fleury.com.br/exames/deteccao-de-virus-oropouche-por-pcr-em-tempo-real-varios-materiais',
'https://www.fleury.com.br/exames/deteccao-dos-genes-bla-kpcbla-ndmblaoxa-48-e-blaoxa-48-like-cepa-bacteriana',
'https://www.fleury.com.br/exames/deteccao-quantitativa-de-poliomavirus-bk-varios-materiais',
'https://www.fleury.com.br/exames/deteccao-de-poliomavirus-tipo-jc-liquor',
'https://www.fleury.com.br/exames/dexametasona-soro',
'https://www.fleury.com.br/exames/diabetes-tipo-mody-3-teste-genetico-sangue-total',
'https://www.fleury.com.br/exames/diagnostico-de-histoplasmose-atraves-de-pcr-em-tempo-real-varios-materiais',
'https://www.fleury.com.br/exames/diazepam-dosagem-soro',
'https://www.fleury.com.br/exames/difteria-anticorpos-soro',
'https://www.fleury.com.br/exames/digoxina-soro',
'https://www.fleury.com.br/exames/dihidro-rodamina-oxidacao-teste-sangue-total',
'https://www.fleury.com.br/exames/dihidrotestosterona-soro',
'https://www.fleury.com.br/exames/dimeros-d-por-turbidimetria-optica-plasma',
'https://www.fleury.com.br/exames/disfibrinogenemia-pesquisa-plasma',
'https://www.fleury.com.br/exames/disopiramida-plasma',
'https://www.fleury.com.br/exames/distonias-varios-materiais',
'https://www.fleury.com.br/exames/distrofia-miotonica-tipo-2-dm2-sangue-total',
'https://www.fleury.com.br/exames/distrofia-miotonica-de-steinert-teste-molecular-sangue-total',
'https://www.fleury.com.br/exames/dioxido-de-carbono-total-soro',
'https://www.fleury.com.br/exames/doenca-de-charcot-marie-tooth-extendida-varios-materiais',
'https://www.fleury.com.br/exames/doenca-de-charcot-marie-tooth-varios-materiais',
'https://www.fleury.com.br/exames/doenca-de-fabry-varios-materiais',
'https://www.fleury.com.br/exames/doenca-de-pompe-varios-materiais',
'https://www.fleury.com.br/exames/doenca-de-wilson-varios-materiais',
'https://www.fleury.com.br/exames/doenca-inflamatoria-intestinal-precoce-varios-materiais',
'https://www.fleury.com.br/exames/doppler-de-globo-ocular-e-de-orbita-olho-direito',
'https://www.fleury.com.br/exames/doppler-de-globo-ocular-e-de-orbita-olho-esquerdo',
'https://www.fleury.com.br/exames/doppler-de-globo-ocular-e-de-orbita-olhos-ambos',
'https://www.fleury.com.br/exames/dopplerfluxometria-fetal-gestacao-multipla',
'https://www.fleury.com.br/exames/dopplerfluxometria-fetal',
'https://www.fleury.com.br/exames/dosagem-de-adalimumab-soro',
'https://www.fleury.com.br/exames/dosagem-de-infliximab-soro',
'https://www.fleury.com.br/exames/dosagem-de-risperidona-e-metabolitos-soro',
'https://www.fleury.com.br/exames/dosagem-fator-8-por-metodo-cromogenico-plasma',
'https://www.fleury.com.br/exames/dosagem-fator-9-por-metodo-cromogenico',
'https://www.fleury.com.br/exames/dose-terapeutica-de-iodo-131-para-cancer-de-tireoide-ambulatorial',
'https://www.fleury.com.br/exames/dose-terapeutica-de-samario-153',
'https://www.fleury.com.br/exames/doxepina-dosagem-soro',
'https://www.fleury.com.br/exames/drogas-de-abuso-2-drogas-urina',
'https://www.fleury.com.br/exames/drogas-de-abuso-3-drogas-urina',
'https://www.fleury.com.br/exames/drogas-de-abuso-4-drogas-urina',
'https://www.fleury.com.br/exames/drogas-de-abuso-5-drogas-urina',
'https://www.fleury.com.br/exames/drogas-de-abuso-6-drogas-urina',
'https://www.fleury.com.br/exames/drogas-de-abuso-7-drogas-urina',
'https://www.fleury.com.br/exames/drogas-de-abuso-cabelo',
'https://www.fleury.com.br/exames/drogas-de-venda-livre-ou-sob-prescricao-triagem-para-soro',
'https://www.fleury.com.br/exames/drogas-de-venda-livre-ou-sob-prescricao-triagem-para-urina',
'https://www.fleury.com.br/exames/dupilumabe-dupixent-200mg-medicamento-e-aplicacao-subcutaneo',
'https://www.fleury.com.br/exames/dupilumabe-dupixent-300mg-medicamento-e-aplicacao-subcutaneo',
'https://www.fleury.com.br/exames/dupilumabe-dupixent-400mg-medicamento-e-aplicacao-subcutaneo',
'https://www.fleury.com.br/exames/dupilumabe-dupixent-600mg-medicamento-e-aplicacao-subcutaneo',
'https://www.fleury.com.br/exames/dupilumabe-dupixent-apenas-aplicacao-subcutaneo',
'https://www.fleury.com.br/exames/egfr-por-biopsia-liquida-sangue-total',
'https://www.fleury.com.br/exames/eos-de-quadril',
'https://www.fleury.com.br/exames/eoscorpo-para-realizacao-de-projetos-de-pesquisa',
'https://www.fleury.com.br/exames/erenumabe-pasurta-140mg-medicamento-e-aplicacao-subcutaneo',
'https://www.fleury.com.br/exames/erenumabe-pasurta-70mg-medicamento-e-aplicacao-subcutaneo',
'https://www.fleury.com.br/exames/ews-ewing/pnet-hibridacao-in-situ-por-fluorescencia',
'https://www.fleury.com.br/exames/echinococcus-anticorpos-totais-soro',
'https://www.fleury.com.br/exames/echovirus-anticorpos-totais-painel-varios-materiais',
'https://www.fleury.com.br/exames/ecocardiografia-fetal-gestacao-multipla',
'https://www.fleury.com.br/exames/ecocardiografia-fetal',
'https://www.fleury.com.br/exames/ecocardiograma',
'https://www.fleury.com.br/exames/ecocardiograma-bidimensional-infantil',
'https://www.fleury.com.br/exames/ecocardiograma-bidimensional-infantil-com-doppler',
'https://www.fleury.com.br/exames/ecocardiograma-bidimensional-com-mapeamento-de-fluxo-a-cores-infantil',
'https://www.fleury.com.br/exames/ecocardiograma-bidimensional-com-mapeamento-de-fluxo-a-cores',
'https://www.fleury.com.br/exames/ecocardiograma-bidimensional-com-doppler',
'https://www.fleury.com.br/exames/ecocardiograma-bidimensional-com-doppler-e-microbolhas',
'https://www.fleury.com.br/exames/ecocardiograma-tridimensional',
'https://www.fleury.com.br/exames/ecocardiograma-transesofagico',
'https://www.fleury.com.br/exames/ecocardiograma-com-analise-de-strain-miocardico',
'https://www.fleury.com.br/exames/ecocardiograma-com-analise-do-sincronismo-cardiaco',
'https://www.fleury.com.br/exames/ecocardiograma-com-contraste',
'https://www.fleury.com.br/exames/ecocardiograma-com-estresse-farmacologico',
'https://www.fleury.com.br/exames/ecocardiograma-de-esforco',
'https://www.fleury.com.br/exames/ecocardiograma-tridimensional-transesofagico',
'https://www.fleury.com.br/exames/ecoendoscopia-diagnostica-alta',
'https://www.fleury.com.br/exames/ecoendoscopia-diagnostica-baixa',
'https://www.fleury.com.br/exames/ecstasy-pesquisa-urina',
'https://www.fleury.com.br/exames/eculizumabe-soliris-1200mg',
'https://www.fleury.com.br/exames/eculizumabe-soliris-300mg-medicamento-e-aplicacao-endovenoso',
'https://www.fleury.com.br/exames/eculizumabe-soliris-600mg-medicamento-e-aplicacao-endovenoso',
'https://www.fleury.com.br/exames/eculizumabe-soliris-900mg-medicamento-e-aplicacao-endovenoso',
'https://www.fleury.com.br/exames/eculizumabe-soliris-apenas-aplicacao-endovenoso',
'https://www.fleury.com.br/exames/elastase-pancreatica-quantificacao-fezes',
'https://www.fleury.com.br/exames/elastografia-hepatica-para-controle-de-fibrose-por-ultrassonografia',
'https://www.fleury.com.br/exames/elastografia-por-ressonancia-magnetica',
'https://www.fleury.com.br/exames/eletrencefalograma-prolongado-com-video-de-12-horas',
'https://www.fleury.com.br/exames/eletrencefalograma-prolongado-de-2-horas',
'https://www.fleury.com.br/exames/eletrocardiograma-de-alta-resolucao',
'https://www.fleury.com.br/exames/eletrocardiograma-de-esforco',
'https://www.fleury.com.br/exames/eletrocardiograma-de-esforco-em-bicicleta-estacionaria',
'https://www.fleury.com.br/exames/eletrocardiograma-de-repouso',
'https://www.fleury.com.br/exames/eletrococleografia',
'https://www.fleury.com.br/exames/eletrococleografia-infantil',
'https://www.fleury.com.br/exames/eletroencefalograma',
'https://www.fleury.com.br/exames/eletroencefalograma-digital-com-mapeamento-cerebral',
'https://www.fleury.com.br/exames/eletroencefalograma-com-mapeamento-em-vigilia-e-sono',
'https://www.fleury.com.br/exames/eletroencefalograma-com-eletrodos-especiais',
'https://www.fleury.com.br/exames/eletroencefalograma-em-vigilia-e-sono',
'https://www.fleury.com.br/exames/eletroneuromiografia',
'https://www.fleury.com.br/exames/eletroneuromiografia-fibra-unica',
'https://www.fleury.com.br/exames/endomisio-anticorpo-igg-soro',
'https://www.fleury.com.br/exames/endomisio-anticorpos-iga-soro',
'https://www.fleury.com.br/exames/endoscopia-digestiva-alta',
'https://www.fleury.com.br/exames/enema-opaco-adulto-duplo-contraste-com-preparo',
'https://www.fleury.com.br/exames/enema-opaco-com-preparo-para-crianca',
'https://www.fleury.com.br/exames/enema-opaco-para-colostomizados',
'https://www.fleury.com.br/exames/enema-opaco-sem-preparo',
'https://www.fleury.com.br/exames/enolase-neuronio-especifica-liquor',
'https://www.fleury.com.br/exames/enolase-neuronio-especifica-soro',
'https://www.fleury.com.br/exames/enoxaparina-sodica-clexane-40mg-medicamento-e-aplicacao-subcutaneo',
'https://www.fleury.com.br/exames/enoxaparina-sodica-clexane-60mg-medicamento-e-aplicacao-subcutaneo',
'https://www.fleury.com.br/exames/enoxaparina-sodica-clexane-apenas-aplicacao-subcutaneo',
'https://www.fleury.com.br/exames/enoxaparina-sodica-clexane-80mg-medicamento-e-aplicacao-subcutaneo',
'https://www.fleury.com.br/exames/ensaio-de-ligacao-do-fator-viii-ao-fvw-investigacao-de-dvw-tipo-2n-plasma',
'https://www.fleury.com.br/exames/entamoeba-histolytica-e-e-dispar-pesq-de-trofozoitos-varios-materiais',
'https://www.fleury.com.br/exames/entamoeba-histolytica-anticorpos-soro',
'https://www.fleury.com.br/exames/entamoeba-histolytica-antigeno-abscesso-hepatico',
'https://www.fleury.com.br/exames/entamoeba-histolytica-antigeno-fezes',
'https://www.fleury.com.br/exames/enterografia-por-ressonancia-magnetica',
'https://www.fleury.com.br/exames/enterografia-por-tomografia-computadorizada',
'https://www.fleury.com.br/exames/enterovirus-anticorpos-painel-varios-materiais',
'https://www.fleury.com.br/exames/enterovirus-pesquisa-do-rna-liquor',
'https://www.fleury.com.br/exames/eosinofilos-pesquisa-fezes',
'https://www.fleury.com.br/exames/epilepsia-acionavel-varios-materiais',
'https://www.fleury.com.br/exames/epilepsia-completo-varios-materiais',
'https://www.fleury.com.br/exames/epilepsia-infantil-varios-materiais',
'https://www.fleury.com.br/exames/epitelio-anticorpos-totais-soro',
'https://www.fleury.com.br/exames/epstein-barr-virus-anticorpo-anti-antigeno-precoce-soro',
'https://www.fleury.com.br/exames/epstein-barr-virus-anticorpos-igg-e-igm-soro',
'https://www.fleury.com.br/exames/epstein-barr-virus-avidez-de-igg-soro',
'https://www.fleury.com.br/exames/epstein-barr-virus-hibridizacao-in-situ-varios-materiais',
'https://www.fleury.com.br/exames/epstein-barr-virus-por-pcr-em-tempo-real-qualitativo-varios-materiais',
'https://www.fleury.com.br/exames/epstein-barr-virus-quantificacao-por-pcr-em-tempo-real-plasma',
'https://www.fleury.com.br/exames/erenumabe-pasurta-apenas-aplicacao-subcutaneo',
'https://www.fleury.com.br/exames/eritrograma-sangue-total',
'https://www.fleury.com.br/exames/eritropoietina-soro',
'https://www.fleury.com.br/exames/eritrovirus-b19-anticorpos-igg-e-igm-soro',
'https://www.fleury.com.br/exames/eritrovirus-b19-deteccao-por-pcr-varios-materiais',
'https://www.fleury.com.br/exames/erros-inatos-do-metabolismo-triagem-para-sangue-total-e-urina',
'https://www.fleury.com.br/exames/escanometria',
'https://www.fleury.com.br/exames/escanometria-por-tomografia-membros-inferiores',
'https://www.fleury.com.br/exames/escherichia-coli-o157-fezes',
'https://www.fleury.com.br/exames/esclerose-lateral-amiotrofica-varios-materiais',
'https://www.fleury.com.br/exames/espectroscopia-por-ressonancia-magnetica',
'https://www.fleury.com.br/exames/espermatozoides-anticorpos-igg-e-iga-soro',
'https://www.fleury.com.br/exames/espermatozoides-contagem-esperma',
'https://www.fleury.com.br/exames/espermograma-esperma',
'https://www.fleury.com.br/exames/espirometria',
'https://www.fleury.com.br/exames/espirometria-com-broncodilatador',
'https://www.fleury.com.br/exames/espirometria-com-exercicio',
'https://www.fleury.com.br/exames/espirometria-para-check-up',
'https://www.fleury.com.br/exames/estadiamento-cancer-de-tiroide',
'https://www.fleury.com.br/exames/estanho-plasma',
'https://www.fleury.com.br/exames/estradiol-livre-soro',
'https://www.fleury.com.br/exames/estradiol-ultrasensivel-cg-ms/ms-soro',
'https://www.fleury.com.br/exames/estradiol-especial-soro',
'https://www.fleury.com.br/exames/estradiol-soro',
'https://www.fleury.com.br/exames/estriol-soro',
'https://www.fleury.com.br/exames/estrona-soro',
'https://www.fleury.com.br/exames/estudo-funcao-ventricular-em-repouso-tec-equilibrio',
'https://www.fleury.com.br/exames/estereo-foto-de-papila-olho-direito',
'https://www.fleury.com.br/exames/estereo-foto-de-papila-olho-esquerdo',
'https://www.fleury.com.br/exames/estereo-foto-de-papila-olhos-ambos',
'https://www.fleury.com.br/exames/etanercepte-enbrel-subcutaneo-somente-aplicacao',
'https://www.fleury.com.br/exames/etanercepte-enbrel-subcutaneo',
'https://www.fleury.com.br/exames/etanol-urina',
'https://www.fleury.com.br/exames/ethosuximida-soro',
'https://www.fleury.com.br/exames/everolimus-dosagem-sangue-total',
'https://www.fleury.com.br/exames/exame-conforme-especificado-agen',
'https://www.fleury.com.br/exames/exame-conforme-especificado-card-cardiologia',
'https://www.fleury.com.br/exames/exame-conforme-especificado-fran-franquias',
'https://www.fleury.com.br/exames/exame-conforme-especificado-infu-infusoes-fleury',
'https://www.fleury.com.br/exames/exame-conforme-especificado-mamo-mamografia',
'https://www.fleury.com.br/exames/exame-conforme-especificado-par',
'https://www.fleury.com.br/exames/exame-da-coluna-vertebral-em-equipamento-eos',
'https://www.fleury.com.br/exames/exame-de-corpo-inteiro-em-equipamento-eos',
'https://www.fleury.com.br/exames/exame-dos-membros-inferiores-em-equipamento-eos',
'https://www.fleury.com.br/exames/exerese-de-polipo-cervical',
'https://www.fleury.com.br/exames/exerese-de-polipo-endocervical-por-histeroscopia',
'https://www.fleury.com.br/exames/exerese-de-polipo-endometrial-por-histeroscopia',
'https://www.fleury.com.br/exames/expressao-da-proteina-was-wasp-sangue-total',
'https://www.fleury.com.br/exames/febre-amarela-pesquisa-de-anticorpos-igg-e-igm-varios-materiais',
'https://www.fleury.com.br/exames/fish-para-o-gene-alk-bloco-de-parafina',
'https://www.fleury.com.br/exames/fish-para-o-gene-ros1',
'https://www.fleury.com.br/exames/fish-para-o-gene-tp53-varios-materiais',
'https://www.fleury.com.br/exames/fkhr-pfkhr-rabdomiossarcoma-alveolar-hibridacao-in-situ-varios-materiais',
'https://www.fleury.com.br/exames/fsh-e-lh-apos-estimulo-com-lhrh-soro',
'https://www.fleury.com.br/exames/fator-antinucleo-anticorpos-totais-soro',
'https://www.fleury.com.br/exames/fator-ii-de-crescimento-simile-a-insulina-soro',
'https://www.fleury.com.br/exames/fator-ii-atividade-plasma',
'https://www.fleury.com.br/exames/fator-ix-atividade-plasma',
'https://www.fleury.com.br/exames/fator-ix-inibidor-quantificacao-plasma',
'https://www.fleury.com.br/exames/fator-intrinseco-anticorpos-bloqueadores-soro',
'https://www.fleury.com.br/exames/fator-intrinseco-anticorpos-soro',
'https://www.fleury.com.br/exames/fator-reumatoide-anticorpos-soro',
'https://www.fleury.com.br/exames/fator-v-atividade-plasma',
'https://www.fleury.com.br/exames/fator-vii-atividade-plasma',
'https://www.fleury.com.br/exames/fator-viii-atividade-plasma',
'https://www.fleury.com.br/exames/fator-viii-inibidor-quantificacao-plasma',
'https://www.fleury.com.br/exames/fator-x-atividade-plasma',
'https://www.fleury.com.br/exames/fator-xi-atividade-plasma',
'https://www.fleury.com.br/exames/fator-xii-atividade-plasma',
'https://www.fleury.com.br/exames/fator-xiii-atividade-plasma',
'https://www.fleury.com.br/exames/fator-xiii-qualitativo-para-deficiencia-grave-plasma',
'https://www.fleury.com.br/exames/fator-de-crescimento-de-fibroblastos-23-plasma',
'https://www.fleury.com.br/exames/fator-de-crescimento-do-endotelio-vascular-vegf-plasma',
'https://www.fleury.com.br/exames/fator-de-necrose-tumoral-soro',
'https://www.fleury.com.br/exames/fator-de-von-willebrand-multimeros-plasma',
'https://www.fleury.com.br/exames/fator-de-von-willebrand-quantificacao-plasma',
'https://www.fleury.com.br/exames/febre-maculosa-rickettsia-rickettsi-anticorpos-igg-e-igm-soro',
'https://www.fleury.com.br/exames/fenciclidina-phencyclidine-ou-pcp-pesquisa-urina',
'https://www.fleury.com.br/exames/fenilalanina-plasma',
'https://www.fleury.com.br/exames/fenitoina-soro',
'https://www.fleury.com.br/exames/fenobarbital-soro',
'https://www.fleury.com.br/exames/fenol-dosagem-urina',
'https://www.fleury.com.br/exames/fenotipagem-eritrocitaria-rh-e-kell-sangue',
'https://www.fleury.com.br/exames/fenotipagem-eritrocitaria-estendida-sangue',
'https://www.fleury.com.br/exames/fenotipagem-rhd-parcial-sangue',
'https://www.fleury.com.br/exames/ferritina-soro',
'https://www.fleury.com.br/exames/ferro-medula-ossea',
'https://www.fleury.com.br/exames/ferro-soro',
'https://www.fleury.com.br/exames/ferro-urina',
'https://www.fleury.com.br/exames/fibrilarina-anticorpos-anti-soro',
'https://www.fleury.com.br/exames/fibrilacao-atrial-painel-de-24-genes-varios-materiais',
'https://www.fleury.com.br/exames/fibrinogenio-optica-plasma',
'https://www.fleury.com.br/exames/fibrinogenio-urgente-optica-plasma',
'https://www.fleury.com.br/exames/fibroblastos-cultura-de-pele',
'https://www.fleury.com.br/exames/fibroblastos-cariotipo-de-pele',
'https://www.fleury.com.br/exames/fibrose-cistica-analise-molecular-sangue-total',
'https://www.fleury.com.br/exames/fibrose-cistica-sequencimaneto-completo-teste-molecular-varios-materiais',
'https://www.fleury.com.br/exames/fibrose-cistica-teste-de-triagem-para-suor',
'https://www.fleury.com.br/exames/filagrina-e-profilagrina-anticorpos-soro',
'https://www.fleury.com.br/exames/fish-para-pdgfrb/tel-varios-materiais',
'https://www.fleury.com.br/exames/fistulografia',
'https://www.fleury.com.br/exames/flunitrazepam-dosagem-soro',
'https://www.fleury.com.br/exames/fluoreto-de-sodio-18f-pet/ct',
'https://www.fleury.com.br/exames/fluoreto-urina',
'https://www.fleury.com.br/exames/fluoxetina-soro',
'https://www.fleury.com.br/exames/flurazepam-dosagem-soro',
'https://www.fleury.com.br/exames/fluxometria',
'https://www.fleury.com.br/exames/fosfatase-alcalina-fracao-ossea-soro',
'https://www.fleury.com.br/exames/fosfatase-alcalina-isoenzimas-soro',
'https://www.fleury.com.br/exames/fosfatase-alcalina-em-liquidos-biologicos-varios-materiais',
'https://www.fleury.com.br/exames/fosfatase-alcalina-soro',
'https://www.fleury.com.br/exames/fosfatase-acida-soro',
'https://www.fleury.com.br/exames/fosfatidiletanolamina-painel-de-anticorpos-iga-igg-e-igm-soro',
'https://www.fleury.com.br/exames/fosfatidilinositol-anticorpos-igg-iga-e-igm-soro',
'https://www.fleury.com.br/exames/fosfatidilserina-autoanticorpos-iga-igg-e-igm-soro',
'https://www.fleury.com.br/exames/fosfolipideos-soro',
'https://www.fleury.com.br/exames/foundation-one-para-neoplasias-hematologicas-varios-materiais',
'https://www.fleury.com.br/exames/foundation-one-para-tumor-solido-varios-materiais',
'https://www.fleury.com.br/exames/foundation-liquid-sangue-total',
'https://www.fleury.com.br/exames/fragmentacao-de-dna-espermatico-esperma',
'https://www.fleury.com.br/exames/francisella-tularensis-anticorpos-soro',
'https://www.fleury.com.br/exames/frutosamina-soro',
'https://www.fleury.com.br/exames/fungos-identificacao-morfologica-e-bioquimica-varios-materiais',
'https://www.fleury.com.br/exames/funcao-plaquetaria-avaliacao-sangue-total',
'https://www.fleury.com.br/exames/funcao-plaquetaria-em-uso-de-bloqueador-do-receptor-p2y12-sangue-total',
'https://www.fleury.com.br/exames/fusao-de-imagem-por-ressonancia-magnetica',
'https://www.fleury.com.br/exames/fosforo-reabsorcao-tubular-soro-e-urina',
'https://www.fleury.com.br/exames/fosforo-amostra-isolada-urina',
'https://www.fleury.com.br/exames/fosforo-soro',
'https://www.fleury.com.br/exames/fosforo-urina',
'https://www.fleury.com.br/exames/gene-n-ras-analise-de-mutacao-fragmento-de-tecido',
'https://www.fleury.com.br/exames/gene-pik3ca-analise-de-mutacao-fragmento-de-tecido',
'https://www.fleury.com.br/exames/galactocerebrosidase-sangue-total',
'https://www.fleury.com.br/exames/galactose-1-fosfato-uridiltransferase-sangue-total',
'https://www.fleury.com.br/exames/gama-glutamiltransferase-em-liquidos-biologicos-varios-materiais',
'https://www.fleury.com.br/exames/gama-glutamiltransferase-soro',
'https://www.fleury.com.br/exames/gangliosideos-anticorpos-soro',
'https://www.fleury.com.br/exames/gasometria-e-eletrolitos-arterial-sangue-arterial',
'https://www.fleury.com.br/exames/gasometria-e-eletrolitos-venosa-sangue-venoso',
'https://www.fleury.com.br/exames/gastrina-soro',
'https://www.fleury.com.br/exames/gene-braf-deteccao-da-mutacao-v600e-fragmento-de-tecido',
'https://www.fleury.com.br/exames/gene-k-ras-analise-de-mutacao-fragmento-de-tecido',
'https://www.fleury.com.br/exames/gene-mll-hibridacao-in-situ-por-fluorescencia-varios-materiais',
'https://www.fleury.com.br/exames/gene-npm1-pesquisa-de-mutacao-varios-materiais',
'https://www.fleury.com.br/exames/gene-rrna16s-identificacao-por-sequenciamento-parcial-cepa-de-microrganismo',
'https://www.fleury.com.br/exames/gene-susceptibilidade-doenca-cerebrovascular-painel-10-genes-varios-materiais',
'https://www.fleury.com.br/exames/genes-ccnd1-e-igh-hibridacao-in-situ-por-fluorescencia-varios-materiais',
'https://www.fleury.com.br/exames/genes-fip-chic-pdgfra-hibridacao-in-situ-por-fluorescencia-varios-materiais',
'https://www.fleury.com.br/exames/gentamicina-soro',
'https://www.fleury.com.br/exames/germes-vivos-cepa',
'https://www.fleury.com.br/exames/giardia-lamblia-anticorpos-totais-soro',
'https://www.fleury.com.br/exames/giardia-lamblia-antigeno-varios-materiais',
'https://www.fleury.com.br/exames/giardia-lamblia-antigeno-fezes',
'https://www.fleury.com.br/exames/gliadina-deaminada-anticorpo-iga-soro',
'https://www.fleury.com.br/exames/gliadina-anticorpos-igg-e-iga-soro',
'https://www.fleury.com.br/exames/glicoproteina-associada-a-mielina-anticorpos-soro',
'https://www.fleury.com.br/exames/glicose-6-fosfato-desidrogenase-sangue-total',
'https://www.fleury.com.br/exames/glicose-1-hora-apos-50-gramas-de-glicose-plasma',
'https://www.fleury.com.br/exames/glicose-2-horas-apos-75-gramas-de-glicose-plasma',
'https://www.fleury.com.br/exames/glicose-varios-materiais',
'https://www.fleury.com.br/exames/glicose-apos-estimulo-com-glucagon-plasma',
'https://www.fleury.com.br/exames/glicose-apos-estimulo-de-refeicao-refliq',
'https://www.fleury.com.br/exames/glicose-apos-estimulo-de-refeicao-refmis',
'https://www.fleury.com.br/exames/glicose-plasma',
'https://www.fleury.com.br/exames/glicose-pos-prandial-plasma',
'https://www.fleury.com.br/exames/glicose-urina',
'https://www.fleury.com.br/exames/glucagon-plasma',
'https://www.fleury.com.br/exames/glucoronato-de-3-alfa-androstanediol-soro',
'https://www.fleury.com.br/exames/golimumabe-simponi-iv-fracao-1mg-iv',
'https://www.fleury.com.br/exames/golimumabe-simponi-100mg-med-e-aplicacao-endovenoso',
'https://www.fleury.com.br/exames/golimumabe-simponi-150mg-med-e-aplicacao-endovenoso',
'https://www.fleury.com.br/exames/golimumabe-simponi-200mg-med-e-aplicacao-endovenoso',
'https://www.fleury.com.br/exames/golimumabe-simponi-250mg-med-e-aplicacao-endovenoso',
'https://www.fleury.com.br/exames/golimumabe-simponi-300mg-med-e-aplicacao-endovenoso',
'https://www.fleury.com.br/exames/golimumabe-simponi-350mg-med-e-aplicacao-endovenoso',
'https://www.fleury.com.br/exames/golimumabe-simponi-400mg-med-e-aplicacao-endovenoso',
'https://www.fleury.com.br/exames/golimumabe-simponi-apenas-aplicacao-endovenoso',
'https://www.fleury.com.br/exames/golimumabe-simponi-subcutaneo',
'https://www.fleury.com.br/exames/golimumabe-simponi-subcutaneo-somente-aplicacao',
'https://www.fleury.com.br/exames/gonadotrofina-corionica-especial-soro',
'https://www.fleury.com.br/exames/gonadotrofina-corionica-liquor',
'https://www.fleury.com.br/exames/gonadotrofina-corionica-soro',
'https://www.fleury.com.br/exames/gonadotrofina-corionica-subunidade-beta-livre-soro',
'https://www.fleury.com.br/exames/gonadotrofina-corionica-urina',
'https://www.fleury.com.br/exames/gonioscopia-olhos-ambos',
'https://www.fleury.com.br/exames/gordura-dosagem-fezes',
'https://www.fleury.com.br/exames/gordura-pesquisa-fezes',
'https://www.fleury.com.br/exames/gordura-por-micrometodo-fezes',
'https://www.fleury.com.br/exames/gp210-autoanticorpos-soro',
'https://www.fleury.com.br/exames/granulocitos-anticorpos-soro',
'https://www.fleury.com.br/exames/guselcumabe-tremfya-apenas-aplicacao-subcutaneo',
'https://www.fleury.com.br/exames/guselcumabe-tremfya-med-e-aplicacao-subcutaneo',
'https://www.fleury.com.br/exames/hdl-colesterol-varios-materiais',
'https://www.fleury.com.br/exames/hdl-colesterol-soro',
'https://www.fleury.com.br/exames/he4-sangue',
'https://www.fleury.com.br/exames/hgh-apos-depressao-com-glicose-soro',
'https://www.fleury.com.br/exames/hgh-apos-estimulo-com-clonidina-soro',
'https://www.fleury.com.br/exames/hgh-apos-estimulo-com-glucagon-soro',
'https://www.fleury.com.br/exames/hgh-apos-estimulo-com-l-dopa-e-propranolol-soro',
'https://www.fleury.com.br/exames/hgh-apos-estimulo-com-exercicio-e-clonidina-soro',
'https://www.fleury.com.br/exames/hgh-apos-estimulo-com-exercicio-e-l-dopa-soro',
'https://www.fleury.com.br/exames/hgh-apos-estimulo-de-insulina-soro-e-plasma',
'https://www.fleury.com.br/exames/hgh-apos-exercicio-soro',
'https://www.fleury.com.br/exames/hiv-1-anticorpos-soro',
'https://www.fleury.com.br/exames/hiv-1-integrase-genotipagem-plasma',
'https://www.fleury.com.br/exames/hiv-1-resistencia-a-medicamentos-antiretrovirais-plasma',
'https://www.fleury.com.br/exames/hiv-1-por-pcr-quantitativo-plasma',
'https://www.fleury.com.br/exames/hla-a-e-b-teste-de-histocompatibilidade-de-media-resolucao-sangue-total',
'https://www.fleury.com.br/exames/hla-c-teste-de-histocompatibilidade-de-media-resolucao-sangue-total',
'https://www.fleury.com.br/exames/hla-dq-teste-de-histocompatibilidade-de-alta-resolucao-sangue-total',
'https://www.fleury.com.br/exames/hla-dr-e-dq-teste-de-histocompatibilidade-de-alta-resolucao-sangue-total',
'https://www.fleury.com.br/exames/hla-dr-e-dq-teste-de-histocompatibilidade-de-media-resolucao-sangue-total',
'https://www.fleury.com.br/exames/hla-dr-teste-de-histocompatibilidade-de-alta-resolucao-sangue-total',
'https://www.fleury.com.br/exames/hla-dr-teste-de-histocompatibilidade-de-media-resolucao-sangue-total',
'https://www.fleury.com.br/exames/hla-para-coriorretinopatia-tipo-birdshot-varios-materiais',
'https://www.fleury.com.br/exames/hla-para-hipersensibilidade-ao-abacavir-varios-materiais',
'https://www.fleury.com.br/exames/hla-para-doenca-celiaca-genotipagem-sangue-total',
'https://www.fleury.com.br/exames/hla-a-teste-de-histocompatibilidade-de-alta-resolucao-sangue-total',
'https://www.fleury.com.br/exames/hla-a-teste-de-histocompatibilidade-de-media-resolucao-sangue-total',
'https://www.fleury.com.br/exames/hla-b-teste-de-histocompatibilidade-de-alta-resolucao-sangue-total',
'https://www.fleury.com.br/exames/hla-b-teste-de-histocompatibilidade-de-baixa-resolucao-sangue-total',
'https://www.fleury.com.br/exames/hla-b51-pesquisa-do-alelo-sangue-total',
'https://www.fleury.com.br/exames/hla-c-teste-de-histocompatibilidade-de-alta-resolucao-sangue-total',
'https://www.fleury.com.br/exames/hla-dp-teste-de-histocompatibilidade-de-alta-resolucao-sangue-total',
'https://www.fleury.com.br/exames/hlab27-genotipagem-por-pcr-em-tempo-real-sangue-total',
'https://www.fleury.com.br/exames/holter-de-glicose-6-dias-sangue',
'https://www.fleury.com.br/exames/htlv-i-e-htlv-ii-deteccao-qualitativa-do-dna-pcr-em-tempo-real-sangue',
'https://www.fleury.com.br/exames/htlv1-e-2-anticorpo-igg-por-western-blot-soro',
'https://www.fleury.com.br/exames/htlv1-e-2-anticorpos-totais-soro',
'https://www.fleury.com.br/exames/htlv1-e-2-anticorpos-liquor',
'https://www.fleury.com.br/exames/hu-anti-nuclear-neuronal-anticorpo-soro',
'https://www.fleury.com.br/exames/haloperidol-soro',
'https://www.fleury.com.br/exames/haptoglobina-soro',
'https://www.fleury.com.br/exames/helicobacter-pylori-anticorpo-igg-soro',
'https://www.fleury.com.br/exames/helicobacter-pylori-pesquisa-de-antigenos-fezes',
'https://www.fleury.com.br/exames/hemiplegia-hereditaria-avc-varios-materiais',
'https://www.fleury.com.br/exames/hemocromatose-hereditaria-mutacoes-do-gene-hfe-sangue-total',
'https://www.fleury.com.br/exames/hemocultura-varios-materiais',
'https://www.fleury.com.br/exames/hemocultura-material-enviado-kit-a-varios-materiais',
'https://www.fleury.com.br/exames/hemocultura-para-anaerobios-varios-materiais',
'https://www.fleury.com.br/exames/hemocultura-para-fungos-varios-materiais',
'https://www.fleury.com.br/exames/hemocultura-para-micobacterias-automatizada-varios-materiais',
'https://www.fleury.com.br/exames/hemofagocitose-linfohistiocitica-varios-materiais',
'https://www.fleury.com.br/exames/hemofilia-a-inversao-do-intron-22-do-gene-f8-fator-viii-sangue-total',
'https://www.fleury.com.br/exames/hemofilia-a-sequenciamento-do-gene-do-fator-viii-sangue-total',
'https://www.fleury.com.br/exames/hemofilia-a-teste-do-ddavp-para',
'https://www.fleury.com.br/exames/hemofilia-b-sequenciamento-do-gene-do-fator-ix-sangue-total',
'https://www.fleury.com.br/exames/hemoglobina-a2-sangue-total',
'https://www.fleury.com.br/exames/hemoglobina-fetal-sangue-total',
'https://www.fleury.com.br/exames/hemoglobina-h-pesquisa-citomorfologica-de-corpusculos-sangue-total',
'https://www.fleury.com.br/exames/hemoglobina-glicada-sangue-total',
'https://www.fleury.com.br/exames/hemoglobina-eletroforese-sangue-total',
'https://www.fleury.com.br/exames/hemoglobina-urina',
'https://www.fleury.com.br/exames/hemoglobina-venosa-urgente-sangue-total',
'https://www.fleury.com.br/exames/hemoglobinuria-paroxistica-noturna-sangue-total',
'https://www.fleury.com.br/exames/hemograma-apos-exercicio-sangue-total',
'https://www.fleury.com.br/exames/hemograma-com-contagem-de-bastonetes-sangue-total',
'https://www.fleury.com.br/exames/hemograma-sangue-total',
'https://www.fleury.com.br/exames/hemossedimentacao-velocidade-de-sangue-total',
'https://www.fleury.com.br/exames/hemostasia-termica-por-colonoscopia-argonio',
'https://www.fleury.com.br/exames/hemostasia-termica-por-endoscopia-argonio',
'https://www.fleury.com.br/exames/hemacias-e-leucocitos-contagem-esperma',
'https://www.fleury.com.br/exames/hemacias-e-leucocitos-direto-fezes',
'https://www.fleury.com.br/exames/hepatite-a-anticorpo-igm-soro',
'https://www.fleury.com.br/exames/hepatite-a-anticorpos-totais-soro',
'https://www.fleury.com.br/exames/hepatite-b-anticorpo-igm-contra-antigeno-central-soro',
'https://www.fleury.com.br/exames/hepatite-b-anticorpo-anti-e-soro',
'https://www.fleury.com.br/exames/hepatite-b-anticorpo-contra-antigeno-de-superficie-soro',
'https://www.fleury.com.br/exames/hepatite-b-anticorpos-totais-contra-antigeno-central-soro',
'https://www.fleury.com.br/exames/hepatite-b-antigeno-de-superficie-soro',
'https://www.fleury.com.br/exames/hepatite-b-antigeno-e-soro',
'https://www.fleury.com.br/exames/hepatite-b-sequenciamento-resistencia-a-antivirais-plasma',
'https://www.fleury.com.br/exames/hepatite-b-sequenciamento-c/mutacao-na-regiao-pre-core-plasma',
'https://www.fleury.com.br/exames/hepatite-b-por-pcr-em-tempo-real-pesquisa-e-quantificacao-plasma',
'https://www.fleury.com.br/exames/hepatite-c-anticorpos-soro',
'https://www.fleury.com.br/exames/hepatite-c-genotipagem-plasma',
'https://www.fleury.com.br/exames/hepatite-c-imunoblot-soro',
'https://www.fleury.com.br/exames/hepatite-c-por-pcr-em-tempo-real-pesquisa-e-quantificacao-plasma',
'https://www.fleury.com.br/exames/hepatite-e-anticorpos-igg-e-igm-soro',
'https://www.fleury.com.br/exames/hepatite-e-deteccao-do-rna-por-pcr-soro',
'https://www.fleury.com.br/exames/her-2-hibridacao-in-situ-por-fluorescencia',
'https://www.fleury.com.br/exames/herpes-virus-1-e-2-anticorpos-igg-e-igm-liquor',
'https://www.fleury.com.br/exames/herpes-simplex-i/ii-anticorpos-igg-e-igm-soro',
'https://www.fleury.com.br/exames/herpes-simplex-tipos-1-e-2-dna-por-pcr-em-tempo-real-varios-materiais',
'https://www.fleury.com.br/exames/herpes-virus-tipo-6-anticorpos-igg-e-igm-soro',
'https://www.fleury.com.br/exames/herpes-virus-tipo-6-deteccao-quantitativa-do-dna-por-pcr-plasma',
'https://www.fleury.com.br/exames/herpes-virus-tipo-6-qualitativo-por-pcr-liquor',
'https://www.fleury.com.br/exames/herpes-virus-tipo-7-pesquisa-quantitativa-do-dna-por-pcr-soro',
'https://www.fleury.com.br/exames/herpes-virus-tipo-8-pesquisa-do-dna-por-pcr-soro',
'https://www.fleury.com.br/exames/hibridacao-in-situ-por-fluorescencia-para-linfomas-por-fish-linfonodo',
'https://www.fleury.com.br/exames/hibridacao-in-situ-por-fluorescencia-para-llc-varios-materiais',
'https://www.fleury.com.br/exames/hibridacao-in-situ-por-fluorescencia-para-mieloma-multiplo-varios-materiais',
'https://www.fleury.com.br/exames/hipercolesterolemia-familiar-painel-de-10-genes-varios-materiais',
'https://www.fleury.com.br/exames/histeroscopia-diagnostica',
'https://www.fleury.com.br/exames/histerossalpingografia',
'https://www.fleury.com.br/exames/histerossonografia',
'https://www.fleury.com.br/exames/histona-anticorpos-soro',
'https://www.fleury.com.br/exames/histoplasma-anticorpos-totais-liquor',
'https://www.fleury.com.br/exames/histoplasma-anticorpos-totais-soro',
'https://www.fleury.com.br/exames/histoplasma-antigeno-urina',
'https://www.fleury.com.br/exames/holter-e-mapa-de-24-horas',
'https://www.fleury.com.br/exames/holter-de-24-horas',
'https://www.fleury.com.br/exames/holter-de-7-dias',
'https://www.fleury.com.br/exames/holter-de-glicose-1-dia-sangue',
'https://www.fleury.com.br/exames/holter-de-glicose-2-dias-sangue',
'https://www.fleury.com.br/exames/holter-de-glicose-3-dias-sangue',
'https://www.fleury.com.br/exames/homocisteina-plasma',
'https://www.fleury.com.br/exames/hormonio-foliculo-estimulante-pediatrico-soro',
'https://www.fleury.com.br/exames/hormonio-foliculo-estimulante-soro',
'https://www.fleury.com.br/exames/hormonio-luteinizante-especial-soro',
'https://www.fleury.com.br/exames/hormonio-luteinizante-pediatrico-soro',
'https://www.fleury.com.br/exames/hormonio-luteinizante-soro',
'https://www.fleury.com.br/exames/hormonio-tireoestimulante-apos-estimulo-com-trh-soro',
'https://www.fleury.com.br/exames/hormonio-tireoestimulante-soro',
'https://www.fleury.com.br/exames/hormonio-de-crescimento-anticorpos-soro',
'https://www.fleury.com.br/exames/hormonio-de-crescimento-estudo-molecular-do-gene-sangue-total',
'https://www.fleury.com.br/exames/hormonio-de-crescimento-apos-estimulo-com-arginina-soro',
'https://www.fleury.com.br/exames/hormonio-de-crescimento-apos-estimulo-com-piridostigmina-soro',
'https://www.fleury.com.br/exames/hormonio-de-crescimento-apos-estimulo-com-ghrh-arginina-soro',
'https://www.fleury.com.br/exames/hormonio-de-crescimento-apos-estimulo-com-trh-soro',
'https://www.fleury.com.br/exames/hormonio-de-crescimento-estudo-do-gene-receptor-sangue-total',
'https://www.fleury.com.br/exames/hormonio-de-crescimento-estudo-do-receptor-do-homonio-liberador-sangue-total',
'https://www.fleury.com.br/exames/hormonio-de-crescimento-soro',
'https://www.fleury.com.br/exames/hormonio-de-crescimento-placentario-soro',
'https://www.fleury.com.br/exames/hormonios-glicoproteicos-hipofisarios-subunidade-alfa-soro',
'https://www.fleury.com.br/exames/hormonios-sexuais-globulina-ligadora-soro',
'https://www.fleury.com.br/exames/hormonios-sexuais-globulina-ligadora-para-testobio-especial-soro',
'https://www.fleury.com.br/exames/ica-512-anticorpos-soro',
'https://www.fleury.com.br/exames/imunohistoquimca-para-o-gene-pan-trk',
'https://www.fleury.com.br/exames/identificacao-fenotipica-de-bacterias-varios-materiais',
'https://www.fleury.com.br/exames/identificacao-molecular-do-coronavirus-mers-varios-materiais',
'https://www.fleury.com.br/exames/ige-especifico-d202-acaro-dermatophagoides-pteroyssinus-n-der-p1-soro',
'https://www.fleury.com.br/exames/ige-especifico-corante-vermelho-carmin-soro',
'https://www.fleury.com.br/exames/ige-especifico-d205-tropomiosina-do-acaro-der-p-10-soro',
'https://www.fleury.com.br/exames/ige-especifico-f232-ovoalbumina-soro',
'https://www.fleury.com.br/exames/ige-especifico-f351-tropomiosina-do-camarao-soro',
'https://www.fleury.com.br/exames/ige-especifico-f416-omega-5-gliadina-tri-a-19-soro',
'https://www.fleury.com.br/exames/ige-especifico-gx1-para-gramineas-soro',
'https://www.fleury.com.br/exames/ige-especifico-i3-vespa-comum-soro',
'https://www.fleury.com.br/exames/ige-especifico-c1-penicilloyl-g-soro',
'https://www.fleury.com.br/exames/ige-especifico-c2-penicilina-v-soro',
'https://www.fleury.com.br/exames/ige-especifico-c73-insulina-humana-soro',
'https://www.fleury.com.br/exames/ige-especifico-d1-d-pteronyssinus-soro',
'https://www.fleury.com.br/exames/ige-especifico-d2-d-farinae-soro',
'https://www.fleury.com.br/exames/ige-especifico-d201-blomia-tropicalis-soro',
'https://www.fleury.com.br/exames/ige-especifico-d3-dermatophagoides-microceras-soro',
'https://www.fleury.com.br/exames/ige-especifico-d70-acarus-siro-soro',
'https://www.fleury.com.br/exames/ige-especifico-d73-glycyphagus-domesticus-soro',
'https://www.fleury.com.br/exames/ige-especifico-e1-descamacao-pele-de-gato-soro',
'https://www.fleury.com.br/exames/ige-especifico-e3-descamacao-pele-de-cavalo-soro',
'https://www.fleury.com.br/exames/ige-especifico-e4-epitelio-de-vaca-soro',
'https://www.fleury.com.br/exames/ige-especifico-e5-epitelio-ou-pelo-de-cao-soro',
'https://www.fleury.com.br/exames/ige-especifico-e81-epitelio-de-carneiro-soro',
'https://www.fleury.com.br/exames/ige-especifico-e85-penas-de-galinha-soro',
'https://www.fleury.com.br/exames/ige-especifico-ex1-animais-soro',
'https://www.fleury.com.br/exames/ige-especifico-ex2-animais-soro',
'https://www.fleury.com.br/exames/ige-especifico-ex71-penas-soro',
'https://www.fleury.com.br/exames/ige-especifico-ex72-penas-soro',
'https://www.fleury.com.br/exames/ige-especifico-f1-clara-de-ovo-soro',
'https://www.fleury.com.br/exames/ige-especifico-f10-gergelim-soro',
'https://www.fleury.com.br/exames/ige-especifico-f12-ervilha-soro',
'https://www.fleury.com.br/exames/ige-especifico-f13-amendoim-soro',
'https://www.fleury.com.br/exames/ige-especifico-f14-soja-soro',
'https://www.fleury.com.br/exames/ige-especifico-f15-feijao-branco-soro',
'https://www.fleury.com.br/exames/ige-especifico-f17-avela-soro',
'https://www.fleury.com.br/exames/ige-especifico-f18-castanha-do-para-soro',
'https://www.fleury.com.br/exames/ige-especifico-f2-leite-de-vaca-soro',
'https://www.fleury.com.br/exames/ige-especifico-f20-amendoa-soro',
'https://www.fleury.com.br/exames/ige-especifico-f202-castanha-de-caju-soro',
'https://www.fleury.com.br/exames/ige-especifico-f207-marisco-soro',
'https://www.fleury.com.br/exames/ige-especifico-f208-limao-soro',
'https://www.fleury.com.br/exames/ige-especifico-f210-abacaxi-soro',
'https://www.fleury.com.br/exames/ige-especifico-f221-cafe-soro',
'https://www.fleury.com.br/exames/ige-especifico-f23-caranguejo-soro',
'https://www.fleury.com.br/exames/ige-especifico-f24-camarao-soro',
'https://www.fleury.com.br/exames/ige-especifico-f25-tomate-soro',
'https://www.fleury.com.br/exames/ige-especifico-f256-noz-soro',
'https://www.fleury.com.br/exames/ige-especifico-f259-uva-soro',
'https://www.fleury.com.br/exames/ige-especifico-f26-carne-de-porco-soro',
'https://www.fleury.com.br/exames/ige-especifico-f27-carne-de-vaca-soro',
'https://www.fleury.com.br/exames/ige-especifico-f279-pimenta-malagueta-soro',
'https://www.fleury.com.br/exames/ige-especifico-f280-pimenta-do-reino-soro',
'https://www.fleury.com.br/exames/ige-especifico-f293-papaia-soro',
'https://www.fleury.com.br/exames/ige-especifico-f3-bacalhau-soro',
'https://www.fleury.com.br/exames/ige-especifico-f300-leite-de-cabra-soro',
'https://www.fleury.com.br/exames/ige-especifico-f308-sardinha-soro',
'https://www.fleury.com.br/exames/ige-especifico-f31-cenoura-soro',
'https://www.fleury.com.br/exames/ige-especifico-f33-laranja-soro',
'https://www.fleury.com.br/exames/ige-especifico-f35-batata-soro',
'https://www.fleury.com.br/exames/ige-especifico-f36-coco-soro',
'https://www.fleury.com.br/exames/ige-especifico-f37-mexilhao-azul-soro',
'https://www.fleury.com.br/exames/ige-especifico-f4-trigo-soro',
'https://www.fleury.com.br/exames/ige-especifico-f40-atum-soro',
'https://www.fleury.com.br/exames/ige-especifico-f41-salmao-soro',
'https://www.fleury.com.br/exames/ige-especifico-f44-morango-soro',
'https://www.fleury.com.br/exames/ige-especifico-f45-fermento-de-pao-soro',
'https://www.fleury.com.br/exames/ige-especifico-f47-alho-soro',
'https://www.fleury.com.br/exames/ige-especifico-f48-cebola-soro',
'https://www.fleury.com.br/exames/ige-especifico-f49-maca-soro',
'https://www.fleury.com.br/exames/ige-especifico-f5-centeio-soro',
'https://www.fleury.com.br/exames/ige-especifico-f58-lula-do-pacifico-soro',
'https://www.fleury.com.br/exames/ige-especifico-f59-polvo-soro',
'https://www.fleury.com.br/exames/ige-especifico-f6-cevada-soro',
'https://www.fleury.com.br/exames/ige-especifico-f7-aveia-soro',
'https://www.fleury.com.br/exames/ige-especifico-f75-gema-de-ovo-soro',
'https://www.fleury.com.br/exames/ige-especifico-f76-alfalactoalbumina-soro',
'https://www.fleury.com.br/exames/ige-especifico-f77-betalactoglobulina-soro',
'https://www.fleury.com.br/exames/ige-especifico-f78-caseina-soro',
'https://www.fleury.com.br/exames/ige-especifico-f79-gluten-soro',
'https://www.fleury.com.br/exames/ige-especifico-f8-milho-soro',
'https://www.fleury.com.br/exames/ige-especifico-f80-lagosta-soro',
'https://www.fleury.com.br/exames/ige-especifico-f81-queijo-cheddar-soro',
'https://www.fleury.com.br/exames/ige-especifico-f82-queijo-camembert-soro',
'https://www.fleury.com.br/exames/ige-especifico-f83-carne-de-galinha-soro',
'https://www.fleury.com.br/exames/ige-especifico-f84-kiwi-soro',
'https://www.fleury.com.br/exames/ige-especifico-f87-melao-soro',
'https://www.fleury.com.br/exames/ige-especifico-f89-mostarda-soro',
'https://www.fleury.com.br/exames/ige-especifico-f9-arroz-soro',
'https://www.fleury.com.br/exames/ige-especifico-f91-manga-soro',
'https://www.fleury.com.br/exames/ige-especifico-f92-banana-soro',
'https://www.fleury.com.br/exames/ige-especifico-f93-cacau-chocolate-soro',
'https://www.fleury.com.br/exames/ige-especifico-f95-pessego-soro',
'https://www.fleury.com.br/exames/ige-especifico-f96-abacate-soro',
'https://www.fleury.com.br/exames/ige-especifico-fx1-alimentos-soro',
'https://www.fleury.com.br/exames/ige-especifico-fx15-alimentos-soro',
'https://www.fleury.com.br/exames/ige-especifico-fx2-alimentos-soro',
'https://www.fleury.com.br/exames/ige-especifico-fx3-alimentos-soro',
'https://www.fleury.com.br/exames/ige-especifico-fx5-alimentos-soro',
'https://www.fleury.com.br/exames/ige-especifico-gx2-gramineas-soro',
'https://www.fleury.com.br/exames/ige-especifico-h2-po-domestico-hollister-soro',
'https://www.fleury.com.br/exames/ige-especifico-hx2-poeira-soro',
'https://www.fleury.com.br/exames/ige-especifico-i1-abelha-soro',
'https://www.fleury.com.br/exames/ige-especifico-i2-vespa-branca-soro',
'https://www.fleury.com.br/exames/ige-especifico-i4-marimbondo-soro',
'https://www.fleury.com.br/exames/ige-especifico-i5-vespa-amarela-soro',
'https://www.fleury.com.br/exames/ige-especifico-i6-barata-soro',
'https://www.fleury.com.br/exames/ige-especifico-i70-formiga-soro',
'https://www.fleury.com.br/exames/ige-especifico-i71-pernilongo-soro',
'https://www.fleury.com.br/exames/ige-especifico-k82-latex-soro',
'https://www.fleury.com.br/exames/ige-especifico-m1-penicillium-notatum-soro',
'https://www.fleury.com.br/exames/ige-especifico-m2-cladosporium-herbaru-soro',
'https://www.fleury.com.br/exames/ige-especifico-m3-aspergillus-fumigatus-soro',
'https://www.fleury.com.br/exames/ige-especifico-m5-candida-albicans-soro',
'https://www.fleury.com.br/exames/ige-especifico-m6-alternaria-alternata-soro',
'https://www.fleury.com.br/exames/ige-especifico-mx1-fungos-soro',
'https://www.fleury.com.br/exames/ige-especifico-mx2-fungos-soro',
'https://www.fleury.com.br/exames/ige-especifico-o1-algodao-soro',
'https://www.fleury.com.br/exames/ige-especifico-o201-folha-de-tabaco-soro',
'https://www.fleury.com.br/exames/ige-especifico-para-centeio-polen-g12-soro',
'https://www.fleury.com.br/exames/ige-especifico-para-inalantes-e-alimentos-infantis-soro',
'https://www.fleury.com.br/exames/ige-especifico-para-ovomucoide-soro',
'https://www.fleury.com.br/exames/ige-especifico-raha-h8-amendoim-soro',
'https://www.fleury.com.br/exames/ige-especifico-rgly-m4-soja-soro',
'https://www.fleury.com.br/exames/ige-especifico-rgly-m5-soja-soro',
'https://www.fleury.com.br/exames/ige-especifico-rgly-m6-soja-soro',
'https://www.fleury.com.br/exames/ige-especifico-rhev-b1-latex-soro',
'https://www.fleury.com.br/exames/ige-especifico-rhev-b3-latex-soro',
'https://www.fleury.com.br/exames/ige-especifico-rhev-b5-latex-soro',
'https://www.fleury.com.br/exames/ige-especifico-rhev-b601-latex-soro',
'https://www.fleury.com.br/exames/ige-especifico-rhev-b8-latex-soro',
'https://www.fleury.com.br/exames/ige-especifico-rf287-feijao-soro',
'https://www.fleury.com.br/exames/ige-especifico-tx7-polens-de-arvores-soro',
'https://www.fleury.com.br/exames/ige-especifico-wx5-ervas-daninhas-soro',
'https://www.fleury.com.br/exames/ige-especifico-wx7-ervas-daninhas-soro',
'https://www.fleury.com.br/exames/ige-especifico-raha-h1-amendoim-soro',
'https://www.fleury.com.br/exames/ige-especifico-raha-h2-amendoim-soro',
'https://www.fleury.com.br/exames/ige-especifico-raha-h3-amendoim-soro',
'https://www.fleury.com.br/exames/imipramina-dosagem-soro',
'https://www.fleury.com.br/exames/impedanciometria',
'https://www.fleury.com.br/exames/impedancio-phmetria-esofagica-prolongada',
'https://www.fleury.com.br/exames/imuno-histoquimico-p/hepatite-b-antigenos-c-e-s-varios-materiais',
'https://www.fleury.com.br/exames/imuno-histoquimico-p/deteccao-do-produto-genico-mlh1-e-msh2-varios-materiais',
'https://www.fleury.com.br/exames/imuno-histoquimico-para-adenovirus-varios-materiais',
'https://www.fleury.com.br/exames/imuno-histoquimico-para-citomegalovirus-varios-materiais',
'https://www.fleury.com.br/exames/imuno-histoquimico-para-diagnosticos-varios-materiais',
'https://www.fleury.com.br/exames/imuno-histoquimico-para-herpes-simplex-tipos-1-e-2-varios-materiais',
'https://www.fleury.com.br/exames/imunocomplexos-circulantes-soro',
'https://www.fleury.com.br/exames/imunodeficiencias-combinadas-scid-varios-materiais',
'https://www.fleury.com.br/exames/imunodeficiencias-combinadas-hiper-igm-varios-materiais',
'https://www.fleury.com.br/exames/imunodeficiencias-combinadas-pid-combinadas-com-sindromes-varios-materiais',
'https://www.fleury.com.br/exames/imunoeletroforese-soro',
'https://www.fleury.com.br/exames/imunofenotipagem-painel-proliferativo-varios-materiais',
'https://www.fleury.com.br/exames/imunofluorescencia-direta-material-renal',
'https://www.fleury.com.br/exames/imunofluorescencia-direta-pele',
'https://www.fleury.com.br/exames/imunoglobulina-endobulin-kiovig-fracao-1g-iv',
'https://www.fleury.com.br/exames/imunoglobulina-a-secretora-varios-materiais',
'https://www.fleury.com.br/exames/imunoglobulina-a-liquor',
'https://www.fleury.com.br/exames/imunoglobulina-a-soro',
'https://www.fleury.com.br/exames/imunoglobulina-d-soro',
'https://www.fleury.com.br/exames/imunoglobulina-e-soro',
'https://www.fleury.com.br/exames/imunoglobulina-endobulin-kiovig-35g-med-e-aplicacao-endovenoso',
'https://www.fleury.com.br/exames/imunoglobulina-endobulin-kiovig-40g-med-e-aplicacao-endovenoso',
'https://www.fleury.com.br/exames/imunoglobulina-endobulin-kiovig-45g-med-e-aplicacao-endovenoso',
'https://www.fleury.com.br/exames/imunoglobulina-endobulin-kiovig-20g-med-e-aplicacao-endovenoso',
'https://www.fleury.com.br/exames/imunoglobulina-endobulin-kiovig-25g-med-e-aplicacao-endovenoso',
'https://www.fleury.com.br/exames/imunoglobulina-endobulin-kiovig-30g-med-e-aplicacao-endovenoso',
'https://www.fleury.com.br/exames/imunoglobulina-g-subclasse-1-soro',
'https://www.fleury.com.br/exames/imunoglobulina-g-subclasse-2-soro',
'https://www.fleury.com.br/exames/imunoglobulina-g-subclasse-3-soro',
'https://www.fleury.com.br/exames/imunoglobulina-g-subclasse-4-soro',
'https://www.fleury.com.br/exames/imunoglobulina-g-liquor',
'https://www.fleury.com.br/exames/imunoglobulina-g-soro',
'https://www.fleury.com.br/exames/imunoglobulina-hizentra-1-ampola-de-1g-subcutaneo',
'https://www.fleury.com.br/exames/imunoglobulina-humana-sandoglobulina-privigen-apenas-aplicacao-ev',
'https://www.fleury.com.br/exames/imunoglobulina-hyqvia-15g-medicamento-e-aplicacao-subcutaneo',
'https://www.fleury.com.br/exames/imunoglobulina-hyqvia-25g-medicamento-e-aplicacao-subcutaneo',
'https://www.fleury.com.br/exames/imunoglobulina-hyqvia-20g-medicamento-e-aplicacao-subcutaneo',
'https://www.fleury.com.br/exames/imunoglobulina-hyqvia-30g-medicamento-e-aplicacao-subcutaneo',
'https://www.fleury.com.br/exames/imunoglobulina-hyqvia-10g-medicamento-e-aplicacao-subcutaneo',
'https://www.fleury.com.br/exames/imunoglobulina-hyqvia-25g-medicamento-e-aplicacao-subcutaneo',
'https://www.fleury.com.br/exames/imunoglobulina-hyqvia-40g-medicamento-e-aplicacao-subcutaneo',
'https://www.fleury.com.br/exames/imunoglobulina-hyqvia-5g-medicamento-e-aplicacao-subcutaneo',
'https://www.fleury.com.br/exames/imunoglobulina-hyqvia-fracao-1g-subcutaneo',
'https://www.fleury.com.br/exames/imunoglobulina-hyqvia-apenas-aplicacao-subcutaneo',
'https://www.fleury.com.br/exames/imunoglobulina-m-liquor',
'https://www.fleury.com.br/exames/imunoglobulina-m-sangue-de-cordao-umbilical',
'https://www.fleury.com.br/exames/imunoglobulina-m-soro',
'https://www.fleury.com.br/exames/imunoglobulina-sandoglobulina-privige-35g-med-e-aplicacao-endovenoso',
'https://www.fleury.com.br/exames/imunoglobulina-sandoglobulina-privigen-15g-med-e-aplicacao-endovenoso',
'https://www.fleury.com.br/exames/imunoglobulina-sandoglobulina-privigen-20g-med-e-aplicacao-endovenoso',
'https://www.fleury.com.br/exames/imunoglobulina-sandoglobulina-privigen-25g-med-e-aplicacao-endovenoso',
'https://www.fleury.com.br/exames/imunoglobulina-sandoglobulina-privigen-30g-med-e-aplicacao-endovenoso',
'https://www.fleury.com.br/exames/imunoglobulina-sandoglobulina-privigen-40g-med-e-aplicacao-endovenoso',
'https://www.fleury.com.br/exames/imunoglobulina-sandoglobulina-privigen-45g-med-e-aplicacao-endovenoso',
'https://www.fleury.com.br/exames/imunoglobulina-sandoglobulina-privigen-fracao-1g-iv',
'https://www.fleury.com.br/exames/imunoglobulina-anti-antigeno-d',
'https://www.fleury.com.br/exames/imunoglobulina-endobulin-kiovig-15g-med-e-aplicacao-endovenoso',
'https://www.fleury.com.br/exames/imunoglobulina-hizentra-10g-medicamento-e-aplicacao-subcutaneo',
'https://www.fleury.com.br/exames/imunoglobulina-hizentra-15g-medicamento-e-aplicacao-subcutaneo',
'https://www.fleury.com.br/exames/imunoglobulina-hizentra-20g-medicamento-e-aplicacao-subcutaneo',
'https://www.fleury.com.br/exames/imunoglobulina-hizentra-25g-medicamento-e-aplicacao-subcutaneo',
'https://www.fleury.com.br/exames/imunoglobulina-hizentra-2g-medicamento-e-aplicacao-subcutaneo',
'https://www.fleury.com.br/exames/imunoglobulina-hizentra-30g-medicamento-e-aplicacao-subcutaneo',
'https://www.fleury.com.br/exames/imunoglobulina-hizentra-5g-medicamento-e-aplicacao-subcutaneo',
'https://www.fleury.com.br/exames/imunoglobulina-hizentra-apenas-aplicacao-subcutaneo',
'https://www.fleury.com.br/exames/imunoistoquimica-para-alk-varios-materiais',
'https://www.fleury.com.br/exames/infiltracao-das-articulacoes-sacroiliacas-guiada-por-radioscopia',
'https://www.fleury.com.br/exames/infiltracao-das-articulacoes-sacroiliacas-guiada-por-tomografia',
'https://www.fleury.com.br/exames/infiltracao-de-coluna-cervical-ate-3-discos-ou-4-vertebras-guiada-por-tc',
'https://www.fleury.com.br/exames/infiltracao-de-coluna-cervical-ate-4-discos-ou-5-vertebras-guiada-por-tc',
'https://www.fleury.com.br/exames/infiltracao-de-coluna-cervical-ate-5-discos-ou-6-vertebras-guiada-por-tc',
'https://www.fleury.com.br/exames/infiltracao-de-coluna-dorsal-ate-3-discos-ou-4-vertebras-guiada-por-tc',
'https://www.fleury.com.br/exames/infiltracao-de-coluna-dorsal-ate-4-discos-ou-5-vertebras-guiada-por-tc',
'https://www.fleury.com.br/exames/infiltracao-de-coluna-dorsal-ate-5-discos-ou-6-vertebras-guiada-por-tc',
'https://www.fleury.com.br/exames/infiltracao-de-coluna-lombar-ate-3-discos-ou-4-vertebras-guiada-por-tc',
'https://www.fleury.com.br/exames/infiltracao-de-coluna-lombar-ate-4-discos-ou-5-vertebras-guiada-por-tc',
'https://www.fleury.com.br/exames/infiltracao-de-coluna-lombar-ate-5-discos-ou-6-vertebras-guiada-por-tc',
'https://www.fleury.com.br/exames/infiltracao-de-coluna-guiada-por-radioscopia',
'https://www.fleury.com.br/exames/infiltracao-extra-articular-de-cotovelo-guiada-por-ultrassom',
'https://www.fleury.com.br/exames/infiltracao-extra-articular-de-joelho-guiada-por-ultrassom',
'https://www.fleury.com.br/exames/infiltracao-extra-articular-de-ombro-guiada-por-radioscopia',
'https://www.fleury.com.br/exames/infiltracao-extra-articular-de-ombro-guiada-por-tomografia',
'https://www.fleury.com.br/exames/infiltracao-extra-articular-de-ombro-guiada-por-ultrassom',
'https://www.fleury.com.br/exames/infiltracao-extra-articular-de-punho-guiada-por-ultrassom',
'https://www.fleury.com.br/exames/infiltracao-extra-articular-de-tornozelo-guiada-por-ultrassom',
'https://www.fleury.com.br/exames/infiltracao-extra-articular-quadril-guiada-por-radioscopia',
'https://www.fleury.com.br/exames/infiltracao-extra-articular-quadril-guiada-por-tomografia',
'https://www.fleury.com.br/exames/infiltracao-extra-articular-quadril-guiada-por-ultrassom',
'https://www.fleury.com.br/exames/infliximabe-remicade-fracao-1mg-iv',
'https://www.fleury.com.br/exames/infliximabe-remicade-100mg-medicamento-e-aplicacao-endovenoso',
'https://www.fleury.com.br/exames/infliximabe-remicade-150mg-medicamento-e-aplicacao-endovenoso',
'https://www.fleury.com.br/exames/infliximabe-remicade-200mg-medicamento-e-aplicacao-endovenoso',
'https://www.fleury.com.br/exames/infliximabe-remicade-250mg-medicamento-e-aplicacao-endovenoso',
'https://www.fleury.com.br/exames/infliximabe-remicade-300mg-medicamento-e-aplicacao-endovenoso',
'https://www.fleury.com.br/exames/infliximabe-remicade-350mg-medicamento-e-aplicacao-endovenoso',
'https://www.fleury.com.br/exames/infliximabe-remicade-400mg-medicamento-e-aplicacao-endovenoso',
'https://www.fleury.com.br/exames/infliximabe-remicade-450mg-medicamento-e-aplicacao-endovenoso',
'https://www.fleury.com.br/exames/infliximabe-remicade-500mg-medicamento-e-aplicacao-endovenoso',
'https://www.fleury.com.br/exames/infliximabe-remicade-550mg-medicamento-e-aplicacao-endovenoso',
'https://www.fleury.com.br/exames/infliximabe-remicade-600mg-medicamento-e-aplicacao-endovenoso',
'https://www.fleury.com.br/exames/infliximabe-remicade-apenas-aplicacao-endovenoso',
'https://www.fleury.com.br/exames/influenza-a-pesquisa-de-anticorpos-igg-e-igm-soro',
'https://www.fleury.com.br/exames/inibina-a-soro',
'https://www.fleury.com.br/exames/inibina-b-soro',
'https://www.fleury.com.br/exames/insercao-de-dispositivo-intrauterino-diu-andalan-silverflex',
'https://www.fleury.com.br/exames/insercao-de-dispositivo-intrauterino-diu-de-cobre',
'https://www.fleury.com.br/exames/insercao-de-dispositivo-intrauterino-mini-de-cobre-guiada-por-histeroscopia',
'https://www.fleury.com.br/exames/instabilidade-de-microssatelites-para-tumores-varios-materiais',
'https://www.fleury.com.br/exames/instabilidade-de-microssatelites-msi-deteccao-por-pcr-bloco-de-parafina',
'https://www.fleury.com.br/exames/insulina-2-horas-apos-75-gramas-de-glicose-soro',
'https://www.fleury.com.br/exames/insulina-anticorpos-soro',
'https://www.fleury.com.br/exames/insulina-apos-estimulo-de-refeicao-refliq',
'https://www.fleury.com.br/exames/insulina-apos-estimulo-de-refeicao-refmis',
'https://www.fleury.com.br/exames/insulina-livre-soro',
'https://www.fleury.com.br/exames/insulina-pos-prandial-soro',
'https://www.fleury.com.br/exames/insulina-soro',
'https://www.fleury.com.br/exames/interferona-beta-1a-avonex-30mcg-medicamento-e-aplicacao-intramuscular',
'https://www.fleury.com.br/exames/interferona-beta-1a-avonex-apenas-aplicacao-intramuscular',
'https://www.fleury.com.br/exames/interferonopatias-varios-materiais',
'https://www.fleury.com.br/exames/interleucina-10-sangue',
'https://www.fleury.com.br/exames/interleucina-2-receptor-soluvel-soro',
'https://www.fleury.com.br/exames/interleucina-6-soro',
'https://www.fleury.com.br/exames/interligadores-c-terminais-soro',
'https://www.fleury.com.br/exames/intolerancia-a-lactose-teste-molecular-sangue-periferico',
'https://www.fleury.com.br/exames/iodo-plasma',
'https://www.fleury.com.br/exames/iodo-urina',
'https://www.fleury.com.br/exames/iodocisternografia-cerebral',
'https://www.fleury.com.br/exames/isoaglutininas-soro',
'https://www.fleury.com.br/exames/isoenzimas-da-dhl-soro',
'https://www.fleury.com.br/exames/ixequizumabe-taltz-160mg-medicacao-e-aplicacao-subcutaneo',
'https://www.fleury.com.br/exames/ixequizumabe-taltz-80mg-medicacao-e-aplicacao-subcutaneo',
'https://www.fleury.com.br/exames/ixequizumabe-taltz-apenas-aplicacao-subcutaneo',
'https://www.fleury.com.br/exames/jo-1-anticorpos-soro',
'https://www.fleury.com.br/exames/kennedy-doenca-de-sangue-total',
'https://www.fleury.com.br/exames/ldl-colesterol-dosagem-soro',
'https://www.fleury.com.br/exames/ledgf75-anticorpos-soro',
'https://www.fleury.com.br/exames/lkm-1-anticorpos-totais-soro',
'https://www.fleury.com.br/exames/lla-hibridacao-in-situ-por-fluorescencia-varios-materiais',
'https://www.fleury.com.br/exames/lacosamida-soro',
'https://www.fleury.com.br/exames/lactamase-teste-da-cepa',
'https://www.fleury.com.br/exames/lactose-prova-de-absorcao-plasma',
'https://www.fleury.com.br/exames/lamotrigina-soro',
'https://www.fleury.com.br/exames/laco-prova-do-hospital-materiais-outro',
'https://www.fleury.com.br/exames/legionella-pneumophila-anticorpos-totais-soro',
'https://www.fleury.com.br/exames/legionella-pneumophila-pesquisa-do-dna-varios-materiais',
'https://www.fleury.com.br/exames/leishmania-anticorpo-igg-soro',
'https://www.fleury.com.br/exames/leishmania-pesquisa-varios-materiais',
'https://www.fleury.com.br/exames/leptina-soro',
'https://www.fleury.com.br/exames/leptospira-anticorpo-igm-soro',
'https://www.fleury.com.br/exames/leptospira-pesquisa-varios-materiais',
'https://www.fleury.com.br/exames/leucograma-com-contagem-de-bastonetes-sangue-total',
'https://www.fleury.com.br/exames/leucograma-sangue-total',
'https://www.fleury.com.br/exames/leucocitos-peroxidase-varios-materiais',
'https://www.fleury.com.br/exames/levetiracetam-soro',
'https://www.fleury.com.br/exames/levotiroxina-teste-de-absorcao',
'https://www.fleury.com.br/exames/lidocaina-soro',
'https://www.fleury.com.br/exames/linfocintilografia-mamaria-p/linfonodo-sentinela-e-localizacao-lesao-oculta',
'https://www.fleury.com.br/exames/linfocintilografia-para-avaliacao-de-drenagem-linfatica',
'https://www.fleury.com.br/exames/linfocintilografia-para-pesquisa-de-linfonodo-sentinela',
'https://www.fleury.com.br/exames/linfocitos-b-cd20-contagem-sangue-periferico',
'https://www.fleury.com.br/exames/linfocitos-b-contagem-sangue-total',
'https://www.fleury.com.br/exames/linfocitos-cd4-e-cd8-subpopulacao-sangue-total',
'https://www.fleury.com.br/exames/linfocitos-t-e-b-imunofenotipagem-sangue-total',
'https://www.fleury.com.br/exames/linfocitos-t-totais-sangue-total',
'https://www.fleury.com.br/exames/lipase-em-liquidos-biologicos-varios-materiais',
'https://www.fleury.com.br/exames/lipase-soro',
'https://www.fleury.com.br/exames/lipoproteina-a-soro',
'https://www.fleury.com.br/exames/listeria-monocytogenes-anticorpos-totais-soro',
'https://www.fleury.com.br/exames/listeria-anticorpos-totais-liquor',
'https://www.fleury.com.br/exames/localizacao-de-corpo-estranho-no-globo-ocular',
'https://www.fleury.com.br/exames/localizacao-adenoma-paratiroidiano',
'https://www.fleury.com.br/exames/liquido-cefalorraquiano-exame-do-liquor',
'https://www.fleury.com.br/exames/litio-soro',
'https://www.fleury.com.br/exames/m-tuberculosis-e-resistencia-a-rifampicina-teste-rapido-varios-materiais',
'https://www.fleury.com.br/exames/mav-angioma-cavernoso-snc-varios-materiais',
'https://www.fleury.com.br/exames/mi-2-autoanticorpos-soro',
'https://www.fleury.com.br/exames/mlpa-para-o-gene-msh6',
'https://www.fleury.com.br/exames/mlpa-para-os-genes-ap-sangue-total',
'https://www.fleury.com.br/exames/mlpa-para-os-genes-mlh1-e-msh2',
'https://www.fleury.com.br/exames/mlpa-para-os-genes-mlh1-msh2-e-msh6',
'https://www.fleury.com.br/exames/mlpa-para-os-genes-mutyh-sangue-total',
'https://www.fleury.com.br/exames/mutacao-dominio-tkd-gene-flt3-pcr-e-eletroforese-capilar-varios-materiais',
'https://www.fleury.com.br/exames/mabthera-fracao-1mg-iv',
'https://www.fleury.com.br/exames/macroamilase-soro',
'https://www.fleury.com.br/exames/macrocreatinoquinase-pesquisa-soro',
'https://www.fleury.com.br/exames/macroprolactina-pesquisa-soro',
'https://www.fleury.com.br/exames/magnesio-amostra-isolada-urina',
'https://www.fleury.com.br/exames/magnesio-eritrocitario-sangue',
'https://www.fleury.com.br/exames/magnesio-soro',
'https://www.fleury.com.br/exames/magnesio-urina',
'https://www.fleury.com.br/exames/mamografia',
'https://www.fleury.com.br/exames/mamografia-3d-tomossintese',
'https://www.fleury.com.br/exames/mamografia-e-ultra-sonografia-com-doppler-mamas',
'https://www.fleury.com.br/exames/mamografia-e-ultra-sonografia-mamas',
'https://www.fleury.com.br/exames/mamografia-e-ultra-sonografia-masculina-de-mamas',
'https://www.fleury.com.br/exames/mamografia-em-pacientes-com-protese-mamaria',
'https://www.fleury.com.br/exames/mamografia-para-homens',
'https://www.fleury.com.br/exames/mamotomia-com-biopsia',
'https://www.fleury.com.br/exames/mamotomia-guiada-por-ultra-som',
'https://www.fleury.com.br/exames/manganes-plasma',
'https://www.fleury.com.br/exames/manganes-urina',
'https://www.fleury.com.br/exames/manometria-computadorizada-do-esofago',
'https://www.fleury.com.br/exames/manometria-computadorizada-ano-retal',
'https://www.fleury.com.br/exames/mapeamento-de-retina-olho-direito',
'https://www.fleury.com.br/exames/mapeamento-de-retina-olho-esquerdo',
'https://www.fleury.com.br/exames/mapeamento-de-retina-olhos-ambos',
'https://www.fleury.com.br/exames/maprotilina-dosagem-soro',
'https://www.fleury.com.br/exames/marcacao-mamaria-com-clipe-metalico',
'https://www.fleury.com.br/exames/material-fecal-pesquisa-e/ou-identificacao-varios-materiais',
'https://www.fleury.com.br/exames/matriz-proteica-nuclear-nmp-22-urina',
'https://www.fleury.com.br/exames/mc-ardle-prova-de-sangue',
'https://www.fleury.com.br/exames/megacolon-congenito-doenca-de-hirschsprung-sangue-total',
'https://www.fleury.com.br/exames/megateste-soro',
'https://www.fleury.com.br/exames/membrana-basal-glomerular-anticorpos-soro',
'https://www.fleury.com.br/exames/mepolizumabe-nucala-100mg-medicamento-e-aplicacao-subcutaneo',
'https://www.fleury.com.br/exames/mepolizumabe-nucala-300mg-medicamento-e-aplicacao-subcutaneo',
'https://www.fleury.com.br/exames/mepolizumabe-nucala-apenas-aplicacao-subcutaneo',
'https://www.fleury.com.br/exames/mercurio-dosagem-de-urina',
'https://www.fleury.com.br/exames/mercurio-sangue-total',
'https://www.fleury.com.br/exames/meta-hemoglobina-sangue-total',
'https://www.fleury.com.br/exames/metabolismo-de-repouso',
'https://www.fleury.com.br/exames/metanefrinas-amostra-isolada-urina',
'https://www.fleury.com.br/exames/metanefrinas-dosagem-de-plasma',
'https://www.fleury.com.br/exames/metanefrinas-urina',
'https://www.fleury.com.br/exames/metanol-dosagem-sangue-total',
'https://www.fleury.com.br/exames/metanol-dosagem-urina',
'https://www.fleury.com.br/exames/metilenotetraidrofolato-redutase-mutacao-c677t-e-a1298c-por-pcr-sangue-total',
'https://www.fleury.com.br/exames/metilprednisolona-solu-medrol-apenas-aplicacao-ev',
'https://www.fleury.com.br/exames/metilprednisolona-solu-medrol-medicamento-e-aplicacao-endovenoso',
'https://www.fleury.com.br/exames/metotrexato-soro',
'https://www.fleury.com.br/exames/micobacterias-identificacao-de-cepa-cepa-de-microrganismo',
'https://www.fleury.com.br/exames/micologico-direto-varios-materiais',
'https://www.fleury.com.br/exames/micologico-direto-liquor',
'https://www.fleury.com.br/exames/microarray-de-biopsia-de-vilo-corial-com-plataforma-agilent-cghsnparray-180k',
'https://www.fleury.com.br/exames/microarray-pre-natal-com-plataforma-agilent-cghs-liquido-aminiotico',
'https://www.fleury.com.br/exames/microarray-pre-natal-produto-de-abortamento-material-de-aborto',
'https://www.fleury.com.br/exames/microarray-pos-natal-com-plataforma-agilent-cghsnparray-180k-sangue-total',
'https://www.fleury.com.br/exames/microscopia-especular-de-cornea-olho-direito',
'https://www.fleury.com.br/exames/microscopia-especular-de-cornea-olho-esquerdo',
'https://www.fleury.com.br/exames/microscopia-especular-de-cornea-olhos-ambos',
'https://www.fleury.com.br/exames/microsporidia-pesquisa-varios-materiais',
'https://www.fleury.com.br/exames/microsporidia-pesquisa-fezes',
'https://www.fleury.com.br/exames/mielina-anticorpos-soro',
'https://www.fleury.com.br/exames/mielina-proteina-basica-da-liquor',
'https://www.fleury.com.br/exames/mielograma-medula-ossea',
'https://www.fleury.com.br/exames/mieloperoxidase-anticorpos-soro',
'https://www.fleury.com.br/exames/mini-painel-tumoral-fragmento-tecido',
'https://www.fleury.com.br/exames/mioglobina-plasma',
'https://www.fleury.com.br/exames/mioglobina-urina',
'https://www.fleury.com.br/exames/mitocondria-anticorpos-anti-soro',
'https://www.fleury.com.br/exames/mitocondria-pesquisa-de-anticorpos-por-western-blot-soro',
'https://www.fleury.com.br/exames/mitotano-dosagem-plasma',
'https://www.fleury.com.br/exames/monitorizacao-ambulatorial-da-pressao-arterial',
'https://www.fleury.com.br/exames/motilidade-ocular-teste-de',
'https://www.fleury.com.br/exames/mucopolissacaridose-tipo-i-varios-materiais',
'https://www.fleury.com.br/exames/mucopolissacarides-screening-urina',
'https://www.fleury.com.br/exames/mucosa-gastrica-anticorpos-totais-soro',
'https://www.fleury.com.br/exames/mucosectomia',
'https://www.fleury.com.br/exames/mutacao-v617f-no-gene-jak2-deteccao-quantitativa-por-qpcr-varios-materiais',
'https://www.fleury.com.br/exames/mutacao-tipo-itd-no-gene-flt3-por-pcr-e-eletroforese-capilar-varios-materiais',
'https://www.fleury.com.br/exames/mycobacterium-tuberculosis-deteccao-resposta-linfocitaria-especifica-soro',
'https://www.fleury.com.br/exames/mycobacterium-tuberculosis-deteccao-por-pcr-varios-materiais',
'https://www.fleury.com.br/exames/mycoplasma-hominis-e-mycoplasma-genitalium-pesquisa-do-dna-varios-materiais',
'https://www.fleury.com.br/exames/mycoplasma-pneumoniae-anticorpos-igg-e-igm-soro',
'https://www.fleury.com.br/exames/musculo-estriado-anticorpos-totais-soro',
'https://www.fleury.com.br/exames/musculo-liso-anticorpos-totais-soro',
'https://www.fleury.com.br/exames/neurossonografia-fetal',
'https://www.fleury.com.br/exames/nipt-estendido-varios-materiais',
'https://www.fleury.com.br/exames/notch-1-mutacao-do-gene-pesquisa-varios-materiais',
'https://www.fleury.com.br/exames/nt-pro-bnp-fragmento-n-terminal-do-peptideo-natriuretico-tipo-b-plasma',
'https://www.fleury.com.br/exames/nasofibrofaringolaringoscopia',
'https://www.fleury.com.br/exames/natalizumabe-tysabri-300mg-medicamento-e-aplicacao-endovenoso',
'https://www.fleury.com.br/exames/natalizumabe-tysabri-apenas-aplicacao-ev',
'https://www.fleury.com.br/exames/nefrograma-basal-e-apos-captopril',
'https://www.fleury.com.br/exames/nefrograma-radioisotopico',
'https://www.fleury.com.br/exames/nefrolitiase-perfil-metabolico-varios-materiais',
'https://www.fleury.com.br/exames/neisseria-gonorrhoeae-anticorpos-soro',
'https://www.fleury.com.br/exames/neisseria-gonorrhoeae-deteccao-do-dna-por-pcr-varios-materiais',
'https://www.fleury.com.br/exames/neoplasia-endocrina-multipla-tipo-1-sangue-total',
'https://www.fleury.com.br/exames/neoplasia-endocrina-multipla-tipo-2a-confirmatorio-sangue',
'https://www.fleury.com.br/exames/neoplasia-endocrina-multipla-tipo-2a-varios-materiais',
'https://www.fleury.com.br/exames/neurocisticercose-anticorpos-liquor',
'https://www.fleury.com.br/exames/neurofibromatose-tipo-i-analise-do-nf1-teste-molecular-sangue-total',
'https://www.fleury.com.br/exames/neurofibromatose-varios-materiais',
'https://www.fleury.com.br/exames/neuropatia-distal-motora-varios-materiais',
'https://www.fleury.com.br/exames/neuropatia-hereditaria-sensitiva-varios-materiais',
'https://www.fleury.com.br/exames/neuropatia-hereditaria-suscetibilidade-a-paralisia-por-pressao-sangue-total',
'https://www.fleury.com.br/exames/nistagmografia-computadorizada',
'https://www.fleury.com.br/exames/nitrazepam-dosagem-soro',
'https://www.fleury.com.br/exames/norovirus-pesquisa-do-rna-por-pcr-fezes',
'https://www.fleury.com.br/exames/nao-compactacao-ventricular-esquerda-painel-de-24-genes-varios-materiais',
'https://www.fleury.com.br/exames/niquel-plasma',
'https://www.fleury.com.br/exames/niquel-urina',
'https://www.fleury.com.br/exames/ocrelizumabe-ocrevus-300mg-medicamento-e-aplicacao-endovenoso',
'https://www.fleury.com.br/exames/ocrelizumabe-ocrevus-600mg-medicamento-e-aplicacao-endovenoso',
'https://www.fleury.com.br/exames/ocrelizumabe-ocrevus-apenas-aplicacao-ev',
'https://www.fleury.com.br/exames/omalizumabe-xolair-150mg-medicamento-e-aplicacao-subcutaneo',
'https://www.fleury.com.br/exames/omalizumabe-xolair-300mg-medicamento-e-aplicacao-subcutaneo',
'https://www.fleury.com.br/exames/omalizumabe-xolair-450mg-medicamento-e-aplicacao-subcutaneo',
'https://www.fleury.com.br/exames/omalizumabe-xolair-600mg-medicamento-e-aplicacao-subcutaneo',
'https://www.fleury.com.br/exames/omalizumabe-xolair-apenas-aplicacao-subcutaneo',
'https://www.fleury.com.br/exames/oncotype-adenocarcinoma-invasivo-varios-materiais',
'https://www.fleury.com.br/exames/oncotype-mamario-in-situ-varios-materiais',
'https://www.fleury.com.br/exames/oncotype-mamario-invasivo-varios-materiais',
'https://www.fleury.com.br/exames/oncotype-de-prostata-varios-materiais',
'https://www.fleury.com.br/exames/opiaceos-pesquisa-urina',
'https://www.fleury.com.br/exames/osmolalidade-soro',
'https://www.fleury.com.br/exames/osmolalidade-urina',
'https://www.fleury.com.br/exames/osteocalcina-soro',
'https://www.fleury.com.br/exames/otoemissoes-acusticas',
'https://www.fleury.com.br/exames/otoemissoes-acusticas-infantil',
'https://www.fleury.com.br/exames/ovos-e-larvas-de-helmintos-e-cistos-de-protozoario-pesquisa-de-fezes',
'https://www.fleury.com.br/exames/ovario-anticorpos-soro',
'https://www.fleury.com.br/exames/oxazepam-dosagem-soro',
'https://www.fleury.com.br/exames/oxcarbazepina-soro',
'https://www.fleury.com.br/exames/painel-farmacogenomico-sangue-total',
'https://www.fleury.com.br/exames/painel-somatico-para-cancer-de-mama-fragmento-de-tecido',
'https://www.fleury.com.br/exames/painel-somatico-para-os-genes-brca1-e-brca2-fragmento-de-tecido',
'https://www.fleury.com.br/exames/pca3-urina',
'https://www.fleury.com.br/exames/pci-e-tg-apos-estimulo-tsh-recombinante-fornec-cliente',
'https://www.fleury.com.br/exames/pdl1-imunoistoquimica-varios-materiais',
'https://www.fleury.com.br/exames/pet-ct-com-galio-68',
'https://www.fleury.com.br/exames/ph-metria-computadorizada-do-esofago',
'https://www.fleury.com.br/exames/pm/scl-anticorpos-soro',
'https://www.fleury.com.br/exames/pml-rara-hibridacao-in-situ-medula-ossea',
'https://www.fleury.com.br/exames/pml-rara-hibridacao-in-situ-sangue-periferico',
'https://www.fleury.com.br/exames/pml-rara-translocacao-por-pcr-varios-materiais',
'https://www.fleury.com.br/exames/ppd-teste-tuberculinico',
'https://www.fleury.com.br/exames/preparo-laminas-vermelho-congo-para-microdisseccao-a-laser-bloco-de-parafina',
'https://www.fleury.com.br/exames/pth-1-84-pth-paratormonio-bioativo-soro',
'https://www.fleury.com.br/exames/painel-autismo-varios-materiais',
'https://www.fleury.com.br/exames/painel-cancer-colorretal-hereditario-17-genes-varios-materiais',
'https://www.fleury.com.br/exames/painel-cancer-gastrico-hereditario-varios-materiais',
'https://www.fleury.com.br/exames/painel-cancer-de-prostata-hereditario-varios-materiais',
'https://www.fleury.com.br/exames/painel-feocromocitoma-varios-materiais',
'https://www.fleury.com.br/exames/painel-genetico-para-doenca-de-alzheimer-varios-materiais',
'https://www.fleury.com.br/exames/painel-genetico-para-paraplegia-espastica-varios-materiais',
'https://www.fleury.com.br/exames/painel-glioblastoma-metilacao-do-promotor-do-gene-mgmt-bloco-de-parafina',
'https://www.fleury.com.br/exames/painel-li-fraumeni-2-genes-sangue-total',
'https://www.fleury.com.br/exames/painel-melanoma-hereditario-varios-materiais',
'https://www.fleury.com.br/exames/painel-neuropatias-hereditarias-com-amiloidose-familiar-varios-materiais',
'https://www.fleury.com.br/exames/painel-polipose-adenomatose-familiar-varios-materiais',
'https://www.fleury.com.br/exames/painel-sindrome-lynch-varios-materiais',
'https://www.fleury.com.br/exames/painel-de-3-mutacoes-para-cancer-de-mama-em-judeus-ashkenazi-sangue-total',
'https://www.fleury.com.br/exames/painel-de-5-mutacoes-do-gene-g6pd-teste-molecular-sangue-total',
'https://www.fleury.com.br/exames/painel-de-cardiomiopatias-hereditarias-varios-materiais',
'https://www.fleury.com.br/exames/painel-de-deficiencias-de-anticorpos-agamaglobulinemia-varios-materiais',
'https://www.fleury.com.br/exames/painel-de-deficiencias-de-anticorpos-hipogamaglobulinemia-varios-materiais',
'https://www.fleury.com.br/exames/painel-de-fusoes-oncologia-varios-materiais',
'https://www.fleury.com.br/exames/painel-de-genes-para-pancreatite-cronica-varios-materiais',
'https://www.fleury.com.br/exames/painel-de-mutacoes-de-neoplasias-mieloides-varios-materiais',
'https://www.fleury.com.br/exames/painel-genetico-para-hipertrigliceridemias',
'https://www.fleury.com.br/exames/painel-genetico-para-diabetes-do-tipo-mody-expandido-varios-materiais',
'https://www.fleury.com.br/exames/painel-genetico-para-diabetes-do-tipo-mody-varios-materiais',
'https://www.fleury.com.br/exames/painel-genetico-para-diabetes-neonatal-varios-materiais',
'https://www.fleury.com.br/exames/painel-para-duchenne-ngs-sangue-total',
'https://www.fleury.com.br/exames/painel-para-duchenne/becker-ampliado-mlpa-sangue-total',
'https://www.fleury.com.br/exames/painel-para-duchenne/becker-ampliado-ngsmlpa-sangue-total',
'https://www.fleury.com.br/exames/painel-para-pesquisa-de-autoanticorpos-presentes-em-miosites-soro',
'https://www.fleury.com.br/exames/painel-para-imunodesregulacao-e-autoimunidade-varios-materiais',
'https://www.fleury.com.br/exames/painel-arritmias-hereditarias-varios-materiais',
'https://www.fleury.com.br/exames/painel-cancer-hereditario-expandido-varios-materiais',
'https://www.fleury.com.br/exames/painel-cancer-de-mama-e-ovario-varios-materiais',
'https://www.fleury.com.br/exames/paineis-geneticos-para-hiperparatireoidismo-varios-materiais',
'https://www.fleury.com.br/exames/paineis-geneticos-para-erros-inatos-do-metabolismo-varios-materiais',
'https://www.fleury.com.br/exames/palivizumabe-synagis-250mg-med-e-aplicacao-intramuscular',
'https://www.fleury.com.br/exames/palivizumabe-synagis-100mg-med-e-aplicacao-intramuscular',
'https://www.fleury.com.br/exames/palivizumabe-synagis-150mg-med-e-aplicacao-intramuscular',
'https://www.fleury.com.br/exames/palivizumabe-synagis-200mg-med-e-aplicacao-intramuscular',
'https://www.fleury.com.br/exames/palivizumabe-synagis-50mg-med-e-aplicacao-intramuscular',
'https://www.fleury.com.br/exames/palivizumabe-synagis-apenas-aplicacao-im',
'https://www.fleury.com.br/exames/papilomavirus-humano-hibridizacao-in-situ-varios-materiais',
'https://www.fleury.com.br/exames/papilomavirus-humano-oncoproteinas-virais-e6/e7-varios-materiais',
'https://www.fleury.com.br/exames/paquimetria-ultra-sonica-olho-direito',
'https://www.fleury.com.br/exames/paquimetria-ultra-sonica-olho-esquerdo',
'https://www.fleury.com.br/exames/paquimetria-ultra-sonica-olhos-ambos',
'https://www.fleury.com.br/exames/paracoccidioidomicose-anticorpos-totais-soro',
'https://www.fleury.com.br/exames/paraneoplasicos-painel-de-anticorpos-liquor',
'https://www.fleury.com.br/exames/paraneoplasicos-painel-de-autoanticorpos-soro',
'https://www.fleury.com.br/exames/parasitologico-fezes',
'https://www.fleury.com.br/exames/paratormonio-molecula-intacta-soro',
'https://www.fleury.com.br/exames/paratormonio-proteina-relacionada-plasma',
'https://www.fleury.com.br/exames/paratormonio-dosagem-em-material-obtido-por-puncao-puncao',
'https://www.fleury.com.br/exames/patch-para-dermatite-de-contato-alimentos',
'https://www.fleury.com.br/exames/patch-para-dermatite-de-contato-bateria-padrao',
'https://www.fleury.com.br/exames/patch-para-dermatite-de-contato-capilar',
'https://www.fleury.com.br/exames/patch-para-dermatite-de-contato-corticoides',
'https://www.fleury.com.br/exames/patch-para-dermatite-de-contato-cosmeticos',
'https://www.fleury.com.br/exames/patch-para-dermatite-de-contato-regional',
'https://www.fleury.com.br/exames/patch-para-dermatite-de-contato-unha',
'https://www.fleury.com.br/exames/paternidade-teste-por-dna-suposto-pai-falecido-sangue-total',
'https://www.fleury.com.br/exames/paternidade-teste-por-dna-sangue-total',
'https://www.fleury.com.br/exames/pedido-de-laminas-blocos-de-parafina-material-genomica',
'https://www.fleury.com.br/exames/peniscopia',
'https://www.fleury.com.br/exames/pentacam-olho-direito',
'https://www.fleury.com.br/exames/pentacam-olho-esquerdo',
'https://www.fleury.com.br/exames/pentacam-olhos-ambos',
'https://www.fleury.com.br/exames/peptideo-apos-estimulo-de-refeicao-refmis',
'https://www.fleury.com.br/exames/peptideo-apos-estimulo-de-refeicao-refliq',
'https://www.fleury.com.br/exames/peptideo-c-apos-estimulo-com-glucagon-soro',
'https://www.fleury.com.br/exames/peptideo-c-apos-teste-de-glicose-75-g-soro',
'https://www.fleury.com.br/exames/peptideo-c-pos-prandial-soro',
'https://www.fleury.com.br/exames/peptideo-c-soro',
'https://www.fleury.com.br/exames/peptideo-natriuretico-tipo-b-plasma',
'https://www.fleury.com.br/exames/peptideo-vasoativo-intestinal-plasma',
'https://www.fleury.com.br/exames/peptideo-citrulinado-ciclico-anticorpos-soro',
'https://www.fleury.com.br/exames/perfil-longo-de-esteroides-soro',
'https://www.fleury.com.br/exames/perfil-tumoral-ampliado-varios-materiais',
'https://www.fleury.com.br/exames/perfil-tumoral-direcionado-varios-materiais',
'https://www.fleury.com.br/exames/perfil-biofisico-fetal',
'https://www.fleury.com.br/exames/perfil-biofisico-fetal-gestacao-multipla',
'https://www.fleury.com.br/exames/perfusao-por-ressonancia-magnetica',
'https://www.fleury.com.br/exames/perimetria-de-dupla-frequencia-olho-direito',
'https://www.fleury.com.br/exames/perimetria-de-dupla-frequencia-olho-esquerdo',
'https://www.fleury.com.br/exames/perimetria-de-dupla-frequencia-ambos-os-olhos',
'https://www.fleury.com.br/exames/pesquisa-anti-rna-polimerase-iii-autoanticorpos-soro',
'https://www.fleury.com.br/exames/pesquisa-da-mutacao-c797s-no-gene-egfr-por-biopsia-liquida-sangue-total',
'https://www.fleury.com.br/exames/pesquisa-da-mutacao-l265p-no-gene-myd88-varios-materiais',
'https://www.fleury.com.br/exames/pesquisa-da-mutacao-no-dna-mitocondrial-3243agreater-g-sangue-total',
'https://www.fleury.com.br/exames/pesquisa-de-rotavirus-e-adenovirus-fezes',
'https://www.fleury.com.br/exames/pesquisa-de-autoanticorpos-antissa/-soro',
'https://www.fleury.com.br/exames/pesquisa-de-delecao-no-exon-19-do-gene-egfr-em-plasma-biopsia-liquida',
'https://www.fleury.com.br/exames/pesquisa-de-mutacao-no-gene-cebpa-leucemia-mieloide-aguda-varios-materiais',
'https://www.fleury.com.br/exames/pesquisa-de-mutacoes-no-exon-9-do-gene-da-calreticulina-varios-materiais',
'https://www.fleury.com.br/exames/pesquisa-de-mutacoes-nos-exons-14-15-e-17-do-gene-csf3r-varios-materiais',
'https://www.fleury.com.br/exames/pesquisa-de-mutacoes-no-gene-sf3b1-varios-materiais',
'https://www.fleury.com.br/exames/pesquisa-de-olho-seco',
'https://www.fleury.com.br/exames/pesquisa-do-papilomavirus-humano-hpv-por-tecnica-molecular-varios-materiais',
'https://www.fleury.com.br/exames/pesquisa-genes-mcr1-mcr-2-mcr-3-mcr-4-mcr-5-varios-materiais',
'https://www.fleury.com.br/exames/pesquisa-mutacao-familial-gene-idua-mucopolissacaridose-tipo-i-sangue-total',
'https://www.fleury.com.br/exames/pesquisa-mutacao-familial/gene-ids-mucopolissacaridose-tipo-ii-sangue-total',
'https://www.fleury.com.br/exames/pesquisa-de-diverticulo-de-meckel',
'https://www.fleury.com.br/exames/pesquisa-de-hemorragia-digestiva',
'https://www.fleury.com.br/exames/pesquisa-pos-dose-terapeutica-de-corpo-inteiro',
'https://www.fleury.com.br/exames/piruvato-quinase-sangue-total',
'https://www.fleury.com.br/exames/planigrafia',
'https://www.fleury.com.br/exames/planigrafia-articulacoes-temporomandibulares',
'https://www.fleury.com.br/exames/planigrafia-coluna-vertebral',
'https://www.fleury.com.br/exames/planigrafia-da-laringe',
'https://www.fleury.com.br/exames/planigrafia-dos-rins',
'https://www.fleury.com.br/exames/planigrafia-face',
'https://www.fleury.com.br/exames/planigrafia-sela-turca',
'https://www.fleury.com.br/exames/planigrafia-torax',
'https://www.fleury.com.br/exames/plaquetas-anticorpos-soro',
'https://www.fleury.com.br/exames/plaquetas-contagem-sangue-total',
'https://www.fleury.com.br/exames/plasminogenio-atividade-plasma',
'https://www.fleury.com.br/exames/pneumococos-anticorpos-soro',
'https://www.fleury.com.br/exames/pneumocystis-jirovecii-pcr-quantitativo-varios-materiais',
'https://www.fleury.com.br/exames/pneumocystis-carinii-pesquisa-varios-materiais',
'https://www.fleury.com.br/exames/polimorfismo-4g/5g-inibidor-do-ativador-de-plasminogenio-pai-1-sangue-total',
'https://www.fleury.com.br/exames/polimorfismos-no-gene-dpyd-e-dpd-sangue-total',
'https://www.fleury.com.br/exames/poliomielite-anticorpos-soro',
'https://www.fleury.com.br/exames/polipectomia-de-colon',
'https://www.fleury.com.br/exames/polipepitideo-pancreatico-plasma',
'https://www.fleury.com.br/exames/polissonografia',
'https://www.fleury.com.br/exames/polissonografia-com-cpap',
'https://www.fleury.com.br/exames/polissonografia-para-hipotese-diagnostica',
'https://www.fleury.com.br/exames/porfiria-aguda-intermitente-gene-hmbs-teste-molecular-sangue-total',
'https://www.fleury.com.br/exames/porfirinas-dosagem-urina',
'https://www.fleury.com.br/exames/porfobilinogenio-dosagem-urina',
'https://www.fleury.com.br/exames/porfobilinogenio-pesquisa-urina',
'https://www.fleury.com.br/exames/posaconazol-dosagem-por-lc-ms-e-ms-soro',
'https://www.fleury.com.br/exames/potencial-evocado-miogenico-vestibular-cervical-cvemp-e-ocularovemp',
'https://www.fleury.com.br/exames/potencial-evocado-somato-sensitivo-de-membros-inferiores',
'https://www.fleury.com.br/exames/potencial-evocado-somato-sensitivo-de-membros-superiores',
'https://www.fleury.com.br/exames/potencial-evocado-genito-cortical',
'https://www.fleury.com.br/exames/potassio-soro',
'https://www.fleury.com.br/exames/potassio-urina',
'https://www.fleury.com.br/exames/prader-willi/angelman-sindrome-diagnostico-sangue-total',
'https://www.fleury.com.br/exames/pregnenolona-soro',
'https://www.fleury.com.br/exames/prick-inalantes',
'https://www.fleury.com.br/exames/prick-para-alimentos',
'https://www.fleury.com.br/exames/primidona-soro',
'https://www.fleury.com.br/exames/procedimentos-ambulatoriais-diversos',
'https://www.fleury.com.br/exames/processamento-seminal-prognostico-esperma',
'https://www.fleury.com.br/exames/progesterona-especial-soro',
'https://www.fleury.com.br/exames/progesterona-soro',
'https://www.fleury.com.br/exames/projeto-proteses-mama-motiva',
'https://www.fleury.com.br/exames/prolactina-apos-estimulo-com-trh-soro',
'https://www.fleury.com.br/exames/prolactina-soro',
'https://www.fleury.com.br/exames/propeptideo-amino-terminal-do-procolageno-tipo-1-soro',
'https://www.fleury.com.br/exames/proteina-ligadora-da-vitamina-b12-transcobalamina-soro',
'https://www.fleury.com.br/exames/proteinase-3-anticorpos-soro',
'https://www.fleury.com.br/exames/proteina-a-plasmatica-associada-a-gravidez-soro',
'https://www.fleury.com.br/exames/proteina-c-ativada-pesquisa-de-resistencia-plasma',
'https://www.fleury.com.br/exames/proteina-c-reativa-soro',
'https://www.fleury.com.br/exames/proteina-c-antigeno-plasma',
'https://www.fleury.com.br/exames/proteina-c-funcional-plasma',
'https://www.fleury.com.br/exames/proteina-p-ribossomal-anticorpos-soro',
'https://www.fleury.com.br/exames/proteina-s-funcional-plasma',
'https://www.fleury.com.br/exames/proteina-s-livre-plasma',
'https://www.fleury.com.br/exames/proteina-s-total-e-livre-plasma',
'https://www.fleury.com.br/exames/proteina-ligadora-1-de-igf-soro',
'https://www.fleury.com.br/exames/proteina-ligadora-2-de-igf-soro',
'https://www.fleury.com.br/exames/proteina-ligadora-3-de-igf-soro',
'https://www.fleury.com.br/exames/proteina-amostra-isolada-urina',
'https://www.fleury.com.br/exames/proteina-total-e-fracoes-varios-materiais',
'https://www.fleury.com.br/exames/proteina-urina',
'https://www.fleury.com.br/exames/proteinas-eletroforese-liquor',
'https://www.fleury.com.br/exames/proteinas-eletroforese-soro',
'https://www.fleury.com.br/exames/proteinas-eletroforese-urina',
'https://www.fleury.com.br/exames/proteinas-totais-e-fracoes-soro',
'https://www.fleury.com.br/exames/proteinas-totais-soro',
'https://www.fleury.com.br/exames/protrombina-e-fator-v-leiden-mutacao-do-gene-pesquisa-sangue-total',
'https://www.fleury.com.br/exames/prova-de-restricao-hidrica-varios-materiais',
'https://www.fleury.com.br/exames/prova-de-funcao-pulmonar',
'https://www.fleury.com.br/exames/prova-de-funcao-pulmonar-com-broncodilatador',
'https://www.fleury.com.br/exames/prova-de-refeicao-liquida-de-2-horas-soro',
'https://www.fleury.com.br/exames/prova-de-refeicao-liquida-de-3-horas-soro',
'https://www.fleury.com.br/exames/prova-de-refeicao-liquida-de-4-horas-soro',
'https://www.fleury.com.br/exames/prova-de-refeicao-liquida-de-5-horas-soro',
'https://www.fleury.com.br/exames/prova-de-refeicao-mista-3-horas-soro',
'https://www.fleury.com.br/exames/prova-de-refeicao-mista-4-horas-soro',
'https://www.fleury.com.br/exames/prova-de-refeicao-mista-de-2-horas-soro',
'https://www.fleury.com.br/exames/prova-de-refeicao-mista-de-5-horas-soro',
'https://www.fleury.com.br/exames/pre-natal-hibridacao-in-situ-varios-materiais',
'https://www.fleury.com.br/exames/pre-albumina-soro',
'https://www.fleury.com.br/exames/pre-calicreina-teste-qualitativo-para-deficiencia-de-plasma',
'https://www.fleury.com.br/exames/pro-insulina-soro',
'https://www.fleury.com.br/exames/pro-calcitonina-dosagem-plasma',
'https://www.fleury.com.br/exames/puncao-liquorica',
'https://www.fleury.com.br/exames/puncao-de-liquido-ascitico-guiada-por-ultrassom',
'https://www.fleury.com.br/exames/puncao-de-liquido-sinovial-de-cotovelo-guiada-por-ultrassom',
'https://www.fleury.com.br/exames/puncao-de-liquido-sinovial-de-joelho-guiada-por-ultrassom',
'https://www.fleury.com.br/exames/puncao-de-liquido-sinovial-de-ombro-guiada-por-ultrassom',
'https://www.fleury.com.br/exames/puncao-de-liquido-sinovial-de-quadril-guiada-por-ultrassom',
'https://www.fleury.com.br/exames/puncao-de-liquido-sinovial-guiada-por-ultrassom-tornozelo',
'https://www.fleury.com.br/exames/puncao-de-nodulos-superficiais-de-partes-moles-guiado-por-ultrassonografia',
'https://www.fleury.com.br/exames/puncao-de-colecoes-superficiais-por-us-ou-tc',
'https://www.fleury.com.br/exames/puncao-de-liquido-amniotico',
'https://www.fleury.com.br/exames/puncao-de-liquido-amniotico-para-gestacao-multipla',
'https://www.fleury.com.br/exames/puncao-de-nodulo-de-tiroide-guiado-por-ultra-sonografia',
'https://www.fleury.com.br/exames/puncao-de-nodulos-superficiais',
'https://www.fleury.com.br/exames/puncao-de-orgaos-fetais',
'https://www.fleury.com.br/exames/puncao-de-orgaos-fetais-gestacao-multipla',
'https://www.fleury.com.br/exames/puncao-guiada-por-ecoendoscopia',
'https://www.fleury.com.br/exames/puncao-para-cariotipo-do-vilo-corial',
'https://www.fleury.com.br/exames/puncao-para-cariotipo-do-vilo-corial-gestacao-multipla',
'https://www.fleury.com.br/exames/puncao-para-procedimento-invasivo',
'https://www.fleury.com.br/exames/puncao-orgaos-e-cavidades',
'https://www.fleury.com.br/exames/quimerismo-apos-transplante-de-medula-ossea-por-pcr-varios-materiais',
'https://www.fleury.com.br/exames/recebimento-de-imagem-de-ressonancia-magnetica',
'https://www.fleury.com.br/exames/rett-pesquisa-varios-materiais',
'https://www.fleury.com.br/exames/raios-x-torax-f-pa',
'https://www.fleury.com.br/exames/raios-x-abdomen-agudo-em-pe-deitado-cupulas',
'https://www.fleury.com.br/exames/raios-x-abdomen-simples',
'https://www.fleury.com.br/exames/raios-x-abdomen-ap-lateral-ou-localizada',
'https://www.fleury.com.br/exames/raios-x-antebraco-frenteperfil',
'https://www.fleury.com.br/exames/raios-x-antebracos',
'https://www.fleury.com.br/exames/raios-x-arcada-zigomatica-malar-fobliquas',
'https://www.fleury.com.br/exames/raios-x-articulacao-acromio-clavicular',
'https://www.fleury.com.br/exames/raios-x-articulacao-coxofemural-fobliquas',
'https://www.fleury.com.br/exames/raios-x-articulacao-temporomandibular-bilateral',
'https://www.fleury.com.br/exames/raios-x-articulacao-tibiotarsica-fp',
'https://www.fleury.com.br/exames/raios-x-articulacoes-coxofemurais',
'https://www.fleury.com.br/exames/raios-x-articulacoes-esternoclaviculares',
'https://www.fleury.com.br/exames/raios-x-articulacoes-sacroiliacas-fobliquas',
'https://www.fleury.com.br/exames/raios-x-articulacoes-tibiotarsicas-fp',
'https://www.fleury.com.br/exames/raios-x-bacia',
'https://www.fleury.com.br/exames/raios-x-braco-fp',
'https://www.fleury.com.br/exames/raios-x-bracos',
'https://www.fleury.com.br/exames/raios-x-calcaneo',
'https://www.fleury.com.br/exames/raios-x-calcaneos',
'https://www.fleury.com.br/exames/raios-x-cavum-lateral-e/ou-hirtz',
'https://www.fleury.com.br/exames/raios-x-clavicula',
'https://www.fleury.com.br/exames/raios-x-claviculas',
'https://www.fleury.com.br/exames/raios-x-coluna-cervical-fpobliquas',
'https://www.fleury.com.br/exames/raios-x-coluna-cervical-frenteperfil',
'https://www.fleury.com.br/exames/raios-x-coluna-cervical-funcional-ou-dinamica',
'https://www.fleury.com.br/exames/raios-x-coluna-dorsal-ou-toracica-fp',
'https://www.fleury.com.br/exames/raios-x-coluna-lombossacra-fp-e/ou-localizada',
'https://www.fleury.com.br/exames/raios-x-coluna-lombossacra-fpobliquas',
'https://www.fleury.com.br/exames/raios-x-coluna-lombossacra-funcional-ou-dinamica',
'https://www.fleury.com.br/exames/raios-x-condutos-auditivos-internos',
'https://www.fleury.com.br/exames/raios-x-costelas-dos-2-hemitorax-apobliqua',
'https://www.fleury.com.br/exames/raios-x-costelas-por-hemitorax-apobliquas',
'https://www.fleury.com.br/exames/raios-x-cotovelo-fp',
'https://www.fleury.com.br/exames/raios-x-cotovelos-fp',
'https://www.fleury.com.br/exames/raios-x-coxa-fp',
'https://www.fleury.com.br/exames/raios-x-coxas-fp',
'https://www.fleury.com.br/exames/raios-x-cranio-fp',
'https://www.fleury.com.br/exames/raios-x-cranio-fpobliqua-ou-towne-hirtz',
'https://www.fleury.com.br/exames/raios-x-cranio-fptowne',
'https://www.fleury.com.br/exames/raios-x-da-coluna-panoramica',
'https://www.fleury.com.br/exames/raios-x-escapulas',
'https://www.fleury.com.br/exames/raios-x-esqueleto',
'https://www.fleury.com.br/exames/raios-x-esterno-perfilobliqua',
'https://www.fleury.com.br/exames/raios-x-estomago-e-duodeno',
'https://www.fleury.com.br/exames/raios-x-esofago',
'https://www.fleury.com.br/exames/raios-x-esofago-hiato-estomago-e-duodeno',
'https://www.fleury.com.br/exames/raios-x-hipofaringe',
'https://www.fleury.com.br/exames/raios-x-joelho-ou-rotula-fpaxial',
'https://www.fleury.com.br/exames/raios-x-joelho-fp',
'https://www.fleury.com.br/exames/raios-x-joelho-fpobliquasaxiais',
'https://www.fleury.com.br/exames/raios-x-joelhos-fpaxial',
'https://www.fleury.com.br/exames/raios-x-joelhos-fpobliquasaxiais',
'https://www.fleury.com.br/exames/raios-x-joelhos-frenteperfil',
'https://www.fleury.com.br/exames/raios-x-laringe',
'https://www.fleury.com.br/exames/raios-x-maxilar-inferior-fobliquas',
'https://www.fleury.com.br/exames/raios-x-mediastino',
'https://www.fleury.com.br/exames/raios-x-membros-inferiores',
'https://www.fleury.com.br/exames/raios-x-mao',
'https://www.fleury.com.br/exames/raios-x-maos',
'https://www.fleury.com.br/exames/raios-x-ombro',
'https://www.fleury.com.br/exames/raios-x-ombros',
'https://www.fleury.com.br/exames/raios-x-ossos-da-face-fnmnlathirtz',
'https://www.fleury.com.br/exames/raios-x-para-visto-torax-f-pa',
'https://www.fleury.com.br/exames/raios-x-para-idade-ossea-maos-e-punhos-ap',
'https://www.fleury.com.br/exames/raios-x-perna-fobliquas',
'https://www.fleury.com.br/exames/raios-x-pernas',
'https://www.fleury.com.br/exames/raios-x-punho-fpobliquas',
'https://www.fleury.com.br/exames/raios-x-punhos',
'https://www.fleury.com.br/exames/raios-x-pe-ou-pododactilos-fobliquas',
'https://www.fleury.com.br/exames/raios-x-pes',
'https://www.fleury.com.br/exames/raios-x-penis',
'https://www.fleury.com.br/exames/raios-x-sacrococcix-fp',
'https://www.fleury.com.br/exames/raios-x-seios-da-face-fnmnlat',
'https://www.fleury.com.br/exames/raios-x-seios-da-face-fnmnlathirtz',
'https://www.fleury.com.br/exames/raios-x-sela-turca-fptowne',
'https://www.fleury.com.br/exames/raios-x-transito-colonico',
'https://www.fleury.com.br/exames/raios-x-torax-apico-lordotica',
'https://www.fleury.com.br/exames/raios-x-torax-f-inspirado-e-expirado-p',
'https://www.fleury.com.br/exames/raios-x-torax-fp',
'https://www.fleury.com.br/exames/raios-x-torax-fpobliquas',
'https://www.fleury.com.br/exames/raios-x-orbita-fobliquahirtz',
'https://www.fleury.com.br/exames/raios-x-orbitas-fobliquahirtz',
'https://www.fleury.com.br/exames/raios-x-cranio-e-sela-turca-f-p-localizada',
'https://www.fleury.com.br/exames/rapamicina-dosagem-sangue-total',
'https://www.fleury.com.br/exames/reatividade-contra-painel-de-linfocitos-classe-i-e-ii-soro',
'https://www.fleury.com.br/exames/receptor-4-da-melanocorticotropina-estudo-molecular-do-gene-sangue-total',
'https://www.fleury.com.br/exames/remapeamento-corporal-total-para-controle-de-nevos-ja-diagnosticados',
'https://www.fleury.com.br/exames/renina-atividade-plasmatica-plasma',
'https://www.fleury.com.br/exames/renina-para-prova-de-postura',
'https://www.fleury.com.br/exames/reposicionamento-de-dispositivo-intrauterino-por-histeroscopia-diagnostica',
'https://www.fleury.com.br/exames/resistencia-globular-sangue-total',
'https://www.fleury.com.br/exames/resistencia-aos-hormonios-tiroidianos-sangue-total',
'https://www.fleury.com.br/exames/resseccao-endoscopica-de-lesoes-neoplasicas',
'https://www.fleury.com.br/exames/ressonancia-magnetica-multiparametrica-prostata',
'https://www.fleury.com.br/exames/ressonancia-magnetica-escapula',
'https://www.fleury.com.br/exames/ressonancia-magnetica-hemocromatose',
'https://www.fleury.com.br/exames/ressonancia-magnetica-parede-toracica',
'https://www.fleury.com.br/exames/ressonancia-magnetica-perna-com-esforco',
'https://www.fleury.com.br/exames/ressonancia-magnetica-plexo-braquial',
'https://www.fleury.com.br/exames/ressonancia-magnetica-plexo-sacral',
'https://www.fleury.com.br/exames/ressonancia-magnetica-tractografia',
'https://www.fleury.com.br/exames/ressonancia-magnetica-volumetria-de-hipocampo',
'https://www.fleury.com.br/exames/ressonancia-magnetica-abdomen-superior',
'https://www.fleury.com.br/exames/ressonancia-magnetica-abdomen-total-feminino',
'https://www.fleury.com.br/exames/ressonancia-magnetica-abdomen-total-masculino',
'https://www.fleury.com.br/exames/ressonancia-magnetica-adrenal',
'https://www.fleury.com.br/exames/ressonancia-magnetica-antebraco',
'https://www.fleury.com.br/exames/ressonancia-magnetica-antebracos',
'https://www.fleury.com.br/exames/ressonancia-magnetica-aorta-abdominal',
'https://www.fleury.com.br/exames/ressonancia-magnetica-aorta-toracica',
'https://www.fleury.com.br/exames/ressonancia-magnetica-arterial-cerebral',
'https://www.fleury.com.br/exames/ressonancia-magnetica-arterial-pescoco-carotida',
'https://www.fleury.com.br/exames/ressonancia-magnetica-articulacao-coxofemoral',
'https://www.fleury.com.br/exames/ressonancia-magnetica-articulacao-sacroiliaca',
'https://www.fleury.com.br/exames/ressonancia-magnetica-assoalho-pelvico',
'https://www.fleury.com.br/exames/ressonancia-magnetica-bacia',
'https://www.fleury.com.br/exames/ressonancia-magnetica-base-do-cranio',
'https://www.fleury.com.br/exames/ressonancia-magnetica-bilateral-atm',
'https://www.fleury.com.br/exames/ressonancia-magnetica-bilateral-rins',
'https://www.fleury.com.br/exames/ressonancia-magnetica-braco',
'https://www.fleury.com.br/exames/ressonancia-magnetica-bracos',
'https://www.fleury.com.br/exames/ressonancia-magnetica-coluna-cervical',
'https://www.fleury.com.br/exames/ressonancia-magnetica-coluna-lombossacra',
'https://www.fleury.com.br/exames/ressonancia-magnetica-coluna-sacrococcigea',
'https://www.fleury.com.br/exames/ressonancia-magnetica-coluna-toracica',
'https://www.fleury.com.br/exames/ressonancia-magnetica-coluna-total',
'https://www.fleury.com.br/exames/ressonancia-magnetica-com-estresse-farmacologico-coracao',
'https://www.fleury.com.br/exames/ressonancia-magnetica-coracao',
'https://www.fleury.com.br/exames/ressonancia-magnetica-cotovelo',
'https://www.fleury.com.br/exames/ressonancia-magnetica-cotovelos',
'https://www.fleury.com.br/exames/ressonancia-magnetica-coxa',
'https://www.fleury.com.br/exames/ressonancia-magnetica-coxas-femures',
'https://www.fleury.com.br/exames/ressonancia-magnetica-cranio',
'https://www.fleury.com.br/exames/ressonancia-magnetica-cranio-transicao-craniovertebral',
'https://www.fleury.com.br/exames/ressonancia-magnetica-cranio-e-sela-turca',
'https://www.fleury.com.br/exames/ressonancia-magnetica-craniovertebral-ou-craniocervical',
'https://www.fleury.com.br/exames/ressonancia-magnetica-de-arterias-membro-inferior',
'https://www.fleury.com.br/exames/ressonancia-magnetica-de-arterias-membro-superior',
'https://www.fleury.com.br/exames/ressonancia-magnetica-de-arterias-membros-inferiores',
'https://www.fleury.com.br/exames/ressonancia-magnetica-de-arterias-membros-superiores',
'https://www.fleury.com.br/exames/ressonancia-magnetica-de-corpo-inteiro',
'https://www.fleury.com.br/exames/ressonancia-magnetica-de-mamas-para-protese',
'https://www.fleury.com.br/exames/ressonancia-magnetica-de-parede-de-vasos-cervicais',
'https://www.fleury.com.br/exames/ressonancia-magnetica-de-parede-de-vasos-intracranianos',
'https://www.fleury.com.br/exames/ressonancia-magnetica-escrotal',
'https://www.fleury.com.br/exames/ressonancia-magnetica-face',
'https://www.fleury.com.br/exames/ressonancia-magnetica-faringe/laringe/traqueia',
'https://www.fleury.com.br/exames/ressonancia-magnetica-fetal',
'https://www.fleury.com.br/exames/ressonancia-magnetica-fluxo-liquorico',
'https://www.fleury.com.br/exames/ressonancia-magnetica-joelho',
'https://www.fleury.com.br/exames/ressonancia-magnetica-joelhos',
'https://www.fleury.com.br/exames/ressonancia-magnetica-mamas',
'https://www.fleury.com.br/exames/ressonancia-magnetica-mamas-para-homens',
'https://www.fleury.com.br/exames/ressonancia-magnetica-mastoides',
'https://www.fleury.com.br/exames/ressonancia-magnetica-mao',
'https://www.fleury.com.br/exames/ressonancia-magnetica-maos',
'https://www.fleury.com.br/exames/ressonancia-magnetica-ombro',
'https://www.fleury.com.br/exames/ressonancia-magnetica-ombros',
'https://www.fleury.com.br/exames/ressonancia-magnetica-para-estudo-de-parkinson',
'https://www.fleury.com.br/exames/ressonancia-magnetica-pelve-feminina',
'https://www.fleury.com.br/exames/ressonancia-magnetica-pelve-masculina',
'https://www.fleury.com.br/exames/ressonancia-magnetica-perna',
'https://www.fleury.com.br/exames/ressonancia-magnetica-pernas',
'https://www.fleury.com.br/exames/ressonancia-magnetica-pescoco/regiao-cervical/glandulas',
'https://www.fleury.com.br/exames/ressonancia-magnetica-placenta',
'https://www.fleury.com.br/exames/ressonancia-magnetica-punho',
'https://www.fleury.com.br/exames/ressonancia-magnetica-punhos',
'https://www.fleury.com.br/exames/ressonancia-magnetica-pe',
'https://www.fleury.com.br/exames/ressonancia-magnetica-pes',
'https://www.fleury.com.br/exames/ressonancia-magnetica-penis',
'https://www.fleury.com.br/exames/ressonancia-magnetica-quadril-unilateral-direito-ou-esquerdo',
'https://www.fleury.com.br/exames/ressonancia-magnetica-quadris',
'https://www.fleury.com.br/exames/ressonancia-magnetica-sela/hipofise',
'https://www.fleury.com.br/exames/ressonancia-magnetica-tornozelo',
'https://www.fleury.com.br/exames/ressonancia-magnetica-tornozelos',
'https://www.fleury.com.br/exames/ressonancia-magnetica-torax',
'https://www.fleury.com.br/exames/ressonancia-magnetica-venosa-cerebral',
'https://www.fleury.com.br/exames/ressonancia-magnetica-venosa-pescoco',
'https://www.fleury.com.br/exames/ressonancia-magnetica-orbita',
'https://www.fleury.com.br/exames/restricao-calorica-prova-de-soro',
'https://www.fleury.com.br/exames/reticulocitos-sangue-total',
'https://www.fleury.com.br/exames/retinoblastoma-hereditario-teste-molecular-sangue-total',
'https://www.fleury.com.br/exames/retinografia-olho-direito',
'https://www.fleury.com.br/exames/retinografia-olho-esquerdo',
'https://www.fleury.com.br/exames/retinografia-olhos-ambos',
'https://www.fleury.com.br/exames/retinol-proteina-transportadora-urina',
'https://www.fleury.com.br/exames/retirada-de-diu-por-histeroscopia',
'https://www.fleury.com.br/exames/retossigmoidoscopia',
'https://www.fleury.com.br/exames/retrovirus-anticorpos-totais-soro',
'https://www.fleury.com.br/exames/revisao-de-laminas-varios-materiais',
'https://www.fleury.com.br/exames/ristocetina-cofator-atividade-plasma',
'https://www.fleury.com.br/exames/rituximabe-mabthera-apenas-aplicacao-ev',
'https://www.fleury.com.br/exames/rituximabe-mabthera-1000mg-medicamento-e-aplicacao-endovenoso',
'https://www.fleury.com.br/exames/rituximabe-mabthera-500mg-medicamento-e-aplicacao-endovenoso',
'https://www.fleury.com.br/exames/ro/ssa-e-la/ssb-anticorpos-soro',
'https://www.fleury.com.br/exames/rotura-das-membranas-fetais-teste-da',
'https://www.fleury.com.br/exames/rubeola-anticorpo-igg-soro',
'https://www.fleury.com.br/exames/rubeola-anticorpo-igm-soro',
'https://www.fleury.com.br/exames/rubeola-anticorpos-igg-e-igm-soro',
'https://www.fleury.com.br/exames/rubeola-avidez-de-igg-soro',
'https://www.fleury.com.br/exames/sarampo-deteccao-qualitativa-por-pcr-varios-materiais-varios-materiais',
'https://www.fleury.com.br/exames/sars-cov-2-anticorpos-igm-igg-e-totais-por-quimioluminescencia-soro',
'https://www.fleury.com.br/exames/scl-70-anticorpos-soro',
'https://www.fleury.com.br/exames/screening-para-doencas-de-deposito-lisossomal-papel-filtro',
'https://www.fleury.com.br/exames/shox-estudo-molecular-do-gene-sangue-total',
'https://www.fleury.com.br/exames/synvisc-classsic',
'https://www.fleury.com.br/exames/synvisc-one',
'https://www.fleury.com.br/exames/syt-sarcoma-sinovial-hibridacao-in-situ-por-fluorescencia-varios-materiais',
'https://www.fleury.com.br/exames/sacarato-de-hidroxido-ferrico-noripurum-apenas-aplicacao-ev',
'https://www.fleury.com.br/exames/sacarato-de-hidroxido-ferrico-noripurum-medicamento-e-aplicacao-endovenoso',
'https://www.fleury.com.br/exames/saccharomyces-cerevisae-anticorpos-soro',
'https://www.fleury.com.br/exames/salmonella-typhi-e-praratyphi-anticorpos-totais-soro',
'https://www.fleury.com.br/exames/sangramento-tempo-de-sangue-total',
'https://www.fleury.com.br/exames/sangue-oculto-pesquisa-fezes',
'https://www.fleury.com.br/exames/sarampo-anticorpos-igg-e-igm-soro',
'https://www.fleury.com.br/exames/sarcoptes-scabiei-pesquisa-raspado-lesao-cutanea',
'https://www.fleury.com.br/exames/saturacao-da-transferrina-soro',
'https://www.fleury.com.br/exames/schistosoma-mansoni-pesquisa-de-ovos-fezes',
'https://www.fleury.com.br/exames/schistosoma-anticorpo-igg-soro',
'https://www.fleury.com.br/exames/sedimento-urinario-varios-materiais',
'https://www.fleury.com.br/exames/sedimento-urina-jato-medio',
'https://www.fleury.com.br/exames/sedimento-urina-primeiro-jato',
'https://www.fleury.com.br/exames/sedimento-urina-terceiro-jato',
'https://www.fleury.com.br/exames/sekukinumabe-150mg-cosentyx-subcutaneo',
'https://www.fleury.com.br/exames/sekukinumabe-300mg-cosentyx-subcutaneo',
'https://www.fleury.com.br/exames/sekukinumabe-cosentyx',
'https://www.fleury.com.br/exames/sekukinumabe-cosentyx-subcutaneo-somente-aplicacao',
'https://www.fleury.com.br/exames/selenio-plasma',
'https://www.fleury.com.br/exames/seq-do-gene-shoc2-teste-molecular-para-sindrome-de-noonan-sangue-total',
'https://www.fleury.com.br/exames/seq-do-gene-sox9-teste-molecular-para-displasia-campomelica-varios-materiais',
'https://www.fleury.com.br/exames/sequenciamento-ngs-do-gene-atm',
'https://www.fleury.com.br/exames/sequenciamento-completo-do-gene-apc-sangue-total',
'https://www.fleury.com.br/exames/sequenciamento-de-exoma-varios-materiais',
'https://www.fleury.com.br/exames/sequenciamento-do-gene-alpl-sindrome-de-hipofosfatasia-sangue-total',
'https://www.fleury.com.br/exames/sequenciamento-do-gene-col1a1-e-col1a2-osteogenese-imperfeita-sangue-total',
'https://www.fleury.com.br/exames/sequenciamento-do-gene-col1a1-osteogenese-imperfeita-varios-materiais',
'https://www.fleury.com.br/exames/sequenciamento-do-gene-col1a2-osteogenese-imperfeita-tipo-v-sangue-total',
'https://www.fleury.com.br/exames/sequenciamento-do-gene-gba-para-doenca-de-gaucher-varios-materiais',
'https://www.fleury.com.br/exames/sequenciamento-do-gene-hras-varios-materiais',
'https://www.fleury.com.br/exames/sequenciamento-do-gene-ifitm5-osteogenese-imperfeita-tipo-v-sangue-total',
'https://www.fleury.com.br/exames/sequenciamento-do-gene-pms2-long-range-pcr-complementacao-varios-materiais',
'https://www.fleury.com.br/exames/sequenciamento-do-gene-pms2-usando-long-range-pcr-varios-materiais',
'https://www.fleury.com.br/exames/sequenciamento-gene-idua-mucopolissacaridose-tipo-i-sangue-total',
'https://www.fleury.com.br/exames/seringa-injetora-para-ressonancia-magnetica',
'https://www.fleury.com.br/exames/serotonina-soro',
'https://www.fleury.com.br/exames/sialografia-de',
'https://www.fleury.com.br/exames/sialografia-por-glandula',
'https://www.fleury.com.br/exames/somatomedina-c-igf-1-soro',
'https://www.fleury.com.br/exames/sorotipagem-sangue-total',
'https://www.fleury.com.br/exames/sp-100/pml-anticorpo-igg-soro',
'https://www.fleury.com.br/exames/streptococcus-pneumoniae-pesquisa-de-antigenos-urina',
'https://www.fleury.com.br/exames/streptococcus-pyogenes-prova-rapida-orofaringe',
'https://www.fleury.com.br/exames/streptococcus-anticorpo-igg-23-sorotipos-soro',
'https://www.fleury.com.br/exames/strongyloides-stercoralis-anticorpo-igg-soro',
'https://www.fleury.com.br/exames/subpopulacoes-linfocitarias-sangue-total',
'https://www.fleury.com.br/exames/substancias-redutoras-pesquisa-fezes',
'https://www.fleury.com.br/exames/sulfametoxazol-soro',
'https://www.fleury.com.br/exames/suscetibilidade-mendeliana-a-infeccoes-micobacterias-msmd-varios-materiais',
'https://www.fleury.com.br/exames/swelling-test-esperma',
'https://www.fleury.com.br/exames/sindrome-febre-periodica-painel-com-21-genes-varios-materiais',
'https://www.fleury.com.br/exames/sindrome-hiper-ige-varios-materiais',
'https://www.fleury.com.br/exames/sindrome-marfan-e-aneurisma-toracicoaortico-painel-22-genes-varios-materiais',
'https://www.fleury.com.br/exames/sindrome-noonan-painel-com-12-genes-varios-materiais',
'https://www.fleury.com.br/exames/sindrome-poliglandular-autoimune-tipo-i-sangue-total',
'https://www.fleury.com.br/exames/sindrome-qtlongo-painel-com-13-genes-varios-materiais',
'https://www.fleury.com.br/exames/sindrome-autoimune-linfoproliferativa-alps-varios-materiais',
'https://www.fleury.com.br/exames/sindrome-de-brugada-painel-de-15-genes-varios-materiais',
'https://www.fleury.com.br/exames/sindrome-de-morte-subita-painel-de-68-genes-varios-materiais',
'https://www.fleury.com.br/exames/sindrome-de-neurodegeneracao-por-acumulo-de-ferro-varios-materiais',
'https://www.fleury.com.br/exames/sindrome-do-qt-curto-painel-de-4-genes-varios-materiais',
'https://www.fleury.com.br/exames/sindrome-mielodisplasica-hibridacao-in-situ-varios-materiais',
'https://www.fleury.com.br/exames/sodio-soro',
'https://www.fleury.com.br/exames/sodio-urina',
'https://www.fleury.com.br/exames/t3-reverso-soro',
'https://www.fleury.com.br/exames/t3-livre-soro',
'https://www.fleury.com.br/exames/t3-soro',
'https://www.fleury.com.br/exames/t4-livre-apos-dialise-soro',
'https://www.fleury.com.br/exames/t4-livre-soro',
'https://www.fleury.com.br/exames/t4-soro',
'https://www.fleury.com.br/exames/taxa-para-sigla-amiloidprot',
'https://www.fleury.com.br/exames/teste-rapido-glutamato-desidrogenase-e-toxinas-a-e-b-de-c-difficile-fezes',
'https://www.fleury.com.br/exames/tg-e-tsh-apos-estimulo-com-tsh-recombinante-humano-soro',
'https://www.fleury.com.br/exames/tilt-test-sincope-para',
'https://www.fleury.com.br/exames/tp53-mutacao-do-gene-varios-materiais',
'https://www.fleury.com.br/exames/tp53-pela-tecnica-de-mlpa-varios-materiais',
'https://www.fleury.com.br/exames/tsh-e-t3-apos-estimulo-com-trh-soro',
'https://www.fleury.com.br/exames/tsh-receptor-de-anticorpos-soro',
'https://www.fleury.com.br/exames/tacrolimus-sangue-total',
'https://www.fleury.com.br/exames/tamisacao-fezes',
'https://www.fleury.com.br/exames/taquicardia-ventric-polimorf-catecolaminerg-painel-9-genes-varios-materiais',
'https://www.fleury.com.br/exames/taxa-de-aplicacao-com-medicamento-alta-complexidade',
'https://www.fleury.com.br/exames/taxa-de-aplicacao-com-medicamento-baixa-complexidade',
'https://www.fleury.com.br/exames/taxa-de-traducao-do-laudo-em-lingua-estrangeira-anatomia-patologica',
'https://www.fleury.com.br/exames/taxa-de-traducao-do-laudo-em-lingua-estrangeira-capilaroscopia',
'https://www.fleury.com.br/exames/taxa-de-traducao-do-laudo-em-lingua-estrangeira-dermatologia',
'https://www.fleury.com.br/exames/taxa-de-traducao-do-laudo-em-lingua-estrangeira-ecocardiografia',
'https://www.fleury.com.br/exames/taxa-de-traducao-do-laudo-em-lingua-estrangeira-eletroencefalografia',
'https://www.fleury.com.br/exames/taxa-de-traducao-do-laudo-em-lingua-estrangeira-eletroneuromiografia',
'https://www.fleury.com.br/exames/taxa-de-traducao-do-laudo-em-lingua-estrangeira-endoscopia',
'https://www.fleury.com.br/exames/taxa-de-traducao-do-laudo-em-lingua-estrangeira-genomica',
'https://www.fleury.com.br/exames/taxa-de-traducao-do-laudo-em-lingua-estrangeira-mamografia',
'https://www.fleury.com.br/exames/taxa-de-traducao-do-laudo-em-lingua-estrangeira-medicina-fetal',
'https://www.fleury.com.br/exames/taxa-de-traducao-do-laudo-em-lingua-estrangeira-medicina-nuclear',
'https://www.fleury.com.br/exames/taxa-de-traducao-do-laudo-em-lingua-estrangeira-motilidade-disgestiva',
'https://www.fleury.com.br/exames/taxa-de-traducao-do-laudo-em-lingua-estrangeira-metodos-graficos',
'https://www.fleury.com.br/exames/taxa-de-traducao-do-laudo-em-lingua-estrangeira-oftalmologia',
'https://www.fleury.com.br/exames/taxa-de-traducao-do-laudo-em-lingua-estrangeira-otorrinolaringologia',
'https://www.fleury.com.br/exames/taxa-de-traducao-do-laudo-em-lingua-estrangeira-pneumologia',
'https://www.fleury.com.br/exames/taxa-de-traducao-do-laudo-em-lingua-estrangeira-polissonografia',
'https://www.fleury.com.br/exames/taxa-de-traducao-do-laudo-em-lingua-estrangeira-radiologia',
'https://www.fleury.com.br/exames/taxa-de-traducao-do-laudo-em-lingua-estrangeira-ressonancia-magnetica',
'https://www.fleury.com.br/exames/taxa-de-traducao-do-laudo-em-lingua-estrangeira-tomografia',
'https://www.fleury.com.br/exames/taxa-de-traducao-do-laudo-em-lingua-estrangeira-urologia',
'https://www.fleury.com.br/exames/taxa-de-traducao-do-laudo-em-lingua-estrangeira-colposcopia',
'https://www.fleury.com.br/exames/taxa-de-traducao-do-laudo-em-lingua-estrangeira-ultrassom',
'https://www.fleury.com.br/exames/taxa-de-traducao-do-laudo-em-lingua-estrangeiradensitometria-ossea',
'https://www.fleury.com.br/exames/tempo-de-protrombina-optica-plasma',
'https://www.fleury.com.br/exames/tempo-de-protrombina-urgente-optica-plasma',
'https://www.fleury.com.br/exames/tempo-de-trombina-optica-plasma',
'https://www.fleury.com.br/exames/tempo-de-trombina-urgente-optica-plasma',
'https://www.fleury.com.br/exames/tempo-de-tromboplastina-pacial-ativada-optica-plasma',
'https://www.fleury.com.br/exames/tempo-de-tromboplastina-pacial-ativada-urgente-optica-plasma',
'https://www.fleury.com.br/exames/teofilina-soro',
'https://www.fleury.com.br/exames/teste-pre-natal-nao-invasivo-sangue-total',
'https://www.fleury.com.br/exames/teste-sorologico-confirmatorio-para-zika-virus-soro',
'https://www.fleury.com.br/exames/teste-cardiopulmonar',
'https://www.fleury.com.br/exames/teste-cardiopulmonar-associado-ao-ecocardiograma-de-estresse-fisico',
'https://www.fleury.com.br/exames/teste-cardiopulmonar-em-bicicleta',
'https://www.fleury.com.br/exames/teste-da-clonidina-para-feocromocitoma-plasma',
'https://www.fleury.com.br/exames/teste-da-patergia',
'https://www.fleury.com.br/exames/teste-de-bloqueio-de-lh-pos-tratamento-com-analogo-de-gnrh',
'https://www.fleury.com.br/exames/teste-de-broncoprovocacao-com-metacolina',
'https://www.fleury.com.br/exames/teste-de-gemelaridade-por-analise-de-dna-sangue',
'https://www.fleury.com.br/exames/teste-de-postura-sangue',
'https://www.fleury.com.br/exames/teste-de-sensibilidade-de-micobacterias-varios-materiais',
'https://www.fleury.com.br/exames/teste-de-sensibilidade-micobacterias-crescimento-rapido-cepa-de-microrganismo',
'https://www.fleury.com.br/exames/teste-de-funcao-tubaria-com-timpano-perfurado',
'https://www.fleury.com.br/exames/teste-de-supressao-do-cortisol-apos-1-mg-de-dexametasona-soro',
'https://www.fleury.com.br/exames/teste-de-supressao-do-cortisol-apos-2-mg-de-dexametasona-soro',
'https://www.fleury.com.br/exames/teste-de-supressao-do-cortisol-com-2-mg-de-dexametasona-soro',
'https://www.fleury.com.br/exames/teste-de-leuprolide-soro',
'https://www.fleury.com.br/exames/teste-para-deteccao-da-mutacao-egfr-l858r-por-biopsia-liquida-sangue-total',
'https://www.fleury.com.br/exames/teste-para-deteccao-da-mutacao-egfr-t790m-por-biopsia-liquida',
'https://www.fleury.com.br/exames/teste-rapido-influenza-a-b-e-h1n1-automatizado-genexpert-varios-materiais',
'https://www.fleury.com.br/exames/teste-funcao-tubaria-com-timpano-integro',
'https://www.fleury.com.br/exames/teste-de-supressao-da-tiroide',
'https://www.fleury.com.br/exames/testosterona-livre-por-lc-ms/ms-soro',
'https://www.fleury.com.br/exames/testosterona-biodisponivel-soro',
'https://www.fleury.com.br/exames/testosterona-apos-estimulo-com-gonadotrofina-corionica-15-soro',
'https://www.fleury.com.br/exames/testosterona-apos-estimulo-com-gonadotrofina-corionica-3-soro',
'https://www.fleury.com.br/exames/testosterona-livre-soro',
'https://www.fleury.com.br/exames/testosterona-por-lc-ms/ms-soro',
'https://www.fleury.com.br/exames/testosterona-soro',
'https://www.fleury.com.br/exames/tetrahidrocanabinol-dosagem-sangue-total',
'https://www.fleury.com.br/exames/tipificacao-de-hla-para-narcolepsia-varios-materiais',
'https://www.fleury.com.br/exames/tiroglobulina-dosagem-em-material-obtido-por-puncao-puncao',
'https://www.fleury.com.br/exames/tiroglobulina-por-lc-ms/ms-soro',
'https://www.fleury.com.br/exames/tiroglobulina-soro',
'https://www.fleury.com.br/exames/tiroide-anticorpos-soro',
'https://www.fleury.com.br/exames/tiroperoxidase-anticorpos-soro',
'https://www.fleury.com.br/exames/tirosina-quinase-anti-musculo-especifico-musk-quantitativo-soro',
'https://www.fleury.com.br/exames/tiroxina-globulina-ligadora-soro',
'https://www.fleury.com.br/exames/tocilizumabe-actemrar-880mg-medicamento-e-aplicacao-endovenoso',
'https://www.fleury.com.br/exames/tocilizumabe-actemra-160mg-medicamento-e-aplicacao-endovenoso',
'https://www.fleury.com.br/exames/tocilizumabe-actemra-200mg-medicamento-e-aplicacao-endovenoso',
'https://www.fleury.com.br/exames/tocilizumabe-actemra-240mg-medicamento-e-aplicacao-endovenoso',
'https://www.fleury.com.br/exames/tocilizumabe-actemra-280mg-medicamento-e-aplicacao-endovenoso',
'https://www.fleury.com.br/exames/tocilizumabe-actemra-320mg-medicamento-e-aplicacao-endovenoso',
'https://www.fleury.com.br/exames/tocilizumabe-actemra-400mg-medicamento-e-aplicacao-endovenoso',
'https://www.fleury.com.br/exames/tocilizumabe-actemra-480mg-medicamento-e-aplicacao-endovenoso',
'https://www.fleury.com.br/exames/tocilizumabe-actemra-560mg-medicamento-e-aplicacao-endovenoso',
'https://www.fleury.com.br/exames/tocilizumabe-actemra-640mg-medicamento-e-aplicacao-endovenoso',
'https://www.fleury.com.br/exames/tocilizumabe-actemra-720mg-medicamento-e-aplicacao-endovenoso',
'https://www.fleury.com.br/exames/tocilizumabe-actemra-800mg-medicamento-e-aplicacao-endovenoso',
'https://www.fleury.com.br/exames/tocilizumabe-actemra-80mg-medicamento-e-aplicacao-endovenoso',
'https://www.fleury.com.br/exames/tocilizumabe-actemra-960mg-medicamento-e-aplicacao-endovenoso',
'https://www.fleury.com.br/exames/tocilizumabe-actemra-iv-fracao-1mg-iv',
'https://www.fleury.com.br/exames/tocilizumabe-actemra-apenas-aplicacao-endovenoso',
'https://www.fleury.com.br/exames/tocilizumabe-actemra-subcutaneo',
'https://www.fleury.com.br/exames/tocilizumabe-actemra-subcutaneo-somente-aplicacao',
'https://www.fleury.com.br/exames/tomografia-computadorizada-corpo',
'https://www.fleury.com.br/exames/tomografia-computadorizada-abdome-total',
'https://www.fleury.com.br/exames/tomografia-computadorizada-abdomen-superior',
'https://www.fleury.com.br/exames/tomografia-computadorizada-abdomen-total-feminino',
'https://www.fleury.com.br/exames/tomografia-computadorizada-abdomen-total-masculino',
'https://www.fleury.com.br/exames/tomografia-computadorizada-adomen-superior-pelve-e-torax',
'https://www.fleury.com.br/exames/tomografia-computadorizada-adrenal',
'https://www.fleury.com.br/exames/tomografia-computadorizada-antebraco',
'https://www.fleury.com.br/exames/tomografia-computadorizada-antebracos-de',
'https://www.fleury.com.br/exames/tomografia-computadorizada-aorta-abdominal',
'https://www.fleury.com.br/exames/tomografia-computadorizada-aorta-toracica',
'https://www.fleury.com.br/exames/tomografia-computadorizada-aorta-total',
'https://www.fleury.com.br/exames/tomografia-computadorizada-arcada-dentaria-mandibula',
'https://www.fleury.com.br/exames/tomografia-computadorizada-arcada-dentaria-maxilar',
'https://www.fleury.com.br/exames/tomografia-computadorizada-articulacao-sacroiliaca',
'https://www.fleury.com.br/exames/tomografia-computadorizada-articulacoes-temporomandibulares',
'https://www.fleury.com.br/exames/tomografia-computadorizada-arterias-e-veias-dos-membros-inferiores',
'https://www.fleury.com.br/exames/tomografia-computadorizada-arterias-renais',
'https://www.fleury.com.br/exames/tomografia-computadorizada-bacia',
'https://www.fleury.com.br/exames/tomografia-computadorizada-base-de-cranio',
'https://www.fleury.com.br/exames/tomografia-computadorizada-braco',
'https://www.fleury.com.br/exames/tomografia-computadorizada-bracos-de',
'https://www.fleury.com.br/exames/tomografia-computadorizada-coluna-cervical-ate-3-discos-ou-4-vertebras',
'https://www.fleury.com.br/exames/tomografia-computadorizada-coluna-cervical-ate-4-discos-ou-5-vertebras',
'https://www.fleury.com.br/exames/tomografia-computadorizada-coluna-cervical-ate-5-discos-ou-6-vertebras',
'https://www.fleury.com.br/exames/tomografia-computadorizada-coluna-dorsal-ate-3-discos-ou-4-vertebras',
'https://www.fleury.com.br/exames/tomografia-computadorizada-coluna-dorsal-ate-4-discos-ou-5-vertebras',
'https://www.fleury.com.br/exames/tomografia-computadorizada-coluna-dorsal-ate-5-discos-ou-6-vertebras',
'https://www.fleury.com.br/exames/tomografia-computadorizada-coluna-lombar-ate-3-discos-ou-4-vertebras',
'https://www.fleury.com.br/exames/tomografia-computadorizada-coluna-lombar-ate-4-discos-ou-5-vertebras',
'https://www.fleury.com.br/exames/tomografia-computadorizada-coluna-lombar-ate-5-discos-ou-6-vertebras',
'https://www.fleury.com.br/exames/tomografia-computadorizada-coluna-sacrococcigea',
'https://www.fleury.com.br/exames/tomografia-computadorizada-cotovelo',
'https://www.fleury.com.br/exames/tomografia-computadorizada-cotovelos-de',
'https://www.fleury.com.br/exames/tomografia-computadorizada-coxa',
'https://www.fleury.com.br/exames/tomografia-computadorizada-coxas-de',
'https://www.fleury.com.br/exames/tomografia-computadorizada-cranio',
'https://www.fleury.com.br/exames/tomografia-computadorizada-cranio-transicao-craniovertebral',
'https://www.fleury.com.br/exames/tomografia-computadorizada-cranio-e-sela-turca',
'https://www.fleury.com.br/exames/tomografia-computadorizada-da-mandibula',
'https://www.fleury.com.br/exames/tomografia-computadorizada-do-maxilar',
'https://www.fleury.com.br/exames/tomografia-computadorizada-em-3-dimensoes',
'https://www.fleury.com.br/exames/tomografia-computadorizada-escore-coronariano-de-calcio-quantificacao',
'https://www.fleury.com.br/exames/tomografia-computadorizada-face',
'https://www.fleury.com.br/exames/tomografia-computadorizada-joelho',
'https://www.fleury.com.br/exames/tomografia-computadorizada-joelhos-de',
'https://www.fleury.com.br/exames/tomografia-computadorizada-mastoides',
'https://www.fleury.com.br/exames/tomografia-computadorizada-mao',
'https://www.fleury.com.br/exames/tomografia-computadorizada-maos-de',
'https://www.fleury.com.br/exames/tomografia-computadorizada-ombro',
'https://www.fleury.com.br/exames/tomografia-computadorizada-ombros-de',
'https://www.fleury.com.br/exames/tomografia-computadorizada-para-tep-tromboembolismo-pulmonar',
'https://www.fleury.com.br/exames/tomografia-computadorizada-para-avaliacao-de-perfusao-pulmonar-torax',
'https://www.fleury.com.br/exames/tomografia-computadorizada-pelve',
'https://www.fleury.com.br/exames/tomografia-computadorizada-perna',
'https://www.fleury.com.br/exames/tomografia-computadorizada-perna-direito',
'https://www.fleury.com.br/exames/tomografia-computadorizada-pernas-de',
'https://www.fleury.com.br/exames/tomografia-computadorizada-punho',
'https://www.fleury.com.br/exames/tomografia-computadorizada-punhos-de',
'https://www.fleury.com.br/exames/tomografia-computadorizada-pe',
'https://www.fleury.com.br/exames/tomografia-computadorizada-pes',
'https://www.fleury.com.br/exames/tomografia-computadorizada-quadril',
'https://www.fleury.com.br/exames/tomografia-computadorizada-quadris',
'https://www.fleury.com.br/exames/tomografia-computadorizada-rastreamento-de-cancer-pulmao',
'https://www.fleury.com.br/exames/tomografia-computadorizada-regiao-cervical',
'https://www.fleury.com.br/exames/tomografia-computadorizada-seios-da-face',
'https://www.fleury.com.br/exames/tomografia-computadorizada-seios-da-face-e-torax',
'https://www.fleury.com.br/exames/tomografia-computadorizada-sela-tursica',
'https://www.fleury.com.br/exames/tomografia-computadorizada-tornozelo',
'https://www.fleury.com.br/exames/tomografia-computadorizada-tornozelos-de',
'https://www.fleury.com.br/exames/tomografia-computadorizada-transicao-craniovertebral-ou-craniocervical',
'https://www.fleury.com.br/exames/tomografia-computadorizada-torax',
'https://www.fleury.com.br/exames/tomografia-computadorizada-orbitas',
'https://www.fleury.com.br/exames/tomografia-por-emissao-de-positrons-cerebral-com-associacao-a-tomografia',
'https://www.fleury.com.br/exames/tomografia-por-emissao-de-positrons-com-ga68-psma',
'https://www.fleury.com.br/exames/tomografia-por-emissao-de-positrons-com-fusao-de-imagens-por-tomografia',
'https://www.fleury.com.br/exames/tomografia-por-emissao-de-positrons-para-pesquisa-de-viabilidade-miocardica',
'https://www.fleury.com.br/exames/tomografia-de-coerencia-optica-olho-direito',
'https://www.fleury.com.br/exames/tomografia-de-coerencia-optica-olho-esquerdo',
'https://www.fleury.com.br/exames/tomografia-de-coerencia-optica-olhos-ambos',
'https://www.fleury.com.br/exames/tomografia-do-segmento-anterior-ambos-os-olhos',
'https://www.fleury.com.br/exames/tomografia-do-segmento-anterior-olho-direito',
'https://www.fleury.com.br/exames/tomografia-do-segmento-anterior-olho-esquerdo',
'https://www.fleury.com.br/exames/tomomielografia-ate-3-ou-4-segmentos',
'https://www.fleury.com.br/exames/tonometria-olhos-ambos',
'https://www.fleury.com.br/exames/topiramato-dosagem-soro',
'https://www.fleury.com.br/exames/toracocentese-guiada-por-ultrassom',
'https://www.fleury.com.br/exames/toxocara-canis-anticorpo-igg-soro',
'https://www.fleury.com.br/exames/toxoplasma-gondii-anticorpo-igg-soro',
'https://www.fleury.com.br/exames/toxoplasma-gondii-anticorpo-igm-soro',
'https://www.fleury.com.br/exames/toxoplasma-gondii-anticorpos-igg-e-igm-liquor',
'https://www.fleury.com.br/exames/toxoplasma-gondii-anticorpos-igg-e-igm-soro',
'https://www.fleury.com.br/exames/toxoplasma-gondii-avidez-de-igg-soro',
'https://www.fleury.com.br/exames/toxoplasma-gondii-deteccao-por-pcr-varios-materiais',
'https://www.fleury.com.br/exames/transaminase-glutamico-oxalacetica-varios-materiais',
'https://www.fleury.com.br/exames/transaminase-glutamico-oxalacetica-soro',
'https://www.fleury.com.br/exames/transaminase-glutamico-piruvica-varios-materiais',
'https://www.fleury.com.br/exames/transaminase-glutamico-piruvica-soro',
'https://www.fleury.com.br/exames/transferrina-focalizacao-isoeletrica-soro',
'https://www.fleury.com.br/exames/transglutaminase-tecidual-anticorpos-iga-soro',
'https://www.fleury.com.br/exames/transglutaminase-tecidual-anticorpos-igg-soro',
'https://www.fleury.com.br/exames/tratamento-com-cloreto-de-radio-223',
'https://www.fleury.com.br/exames/tratamento-do-hipertiroidismo-doenca-de-graves',
'https://www.fleury.com.br/exames/tratamento-do-hipertiroidismo-doenca-de-plummer',
'https://www.fleury.com.br/exames/treponema-pallidum-anticorpos-totais-soro',
'https://www.fleury.com.br/exames/treponema-pallidum-anticorpos-liquor',
'https://www.fleury.com.br/exames/treponema-pallidum-floculacao-por-rpr-soro',
'https://www.fleury.com.br/exames/treponema-pallidum-pesquisa-varios-materiais',
'https://www.fleury.com.br/exames/triagem-neonatal-ampliada-sangue-periferico',
'https://www.fleury.com.br/exames/triagem-genomica-portadores-de-multiplas-malformacoes-congenita-sangue-total',
'https://www.fleury.com.br/exames/trichomonas-pesquisa-varios-materiais',
'https://www.fleury.com.br/exames/trichomonas-pesquisa-secrecao-uretral',
'https://www.fleury.com.br/exames/trichomonas-pesquisa-secrecao-vaginal',
'https://www.fleury.com.br/exames/trichomonas-pesquisa-urina-primeiro-jato',
'https://www.fleury.com.br/exames/triclorocompostos-totais-urina',
'https://www.fleury.com.br/exames/triglicerides-varios-materiais',
'https://www.fleury.com.br/exames/triglicerides-pos-prandial-soro',
'https://www.fleury.com.br/exames/triglicerides-soro',
'https://www.fleury.com.br/exames/trimipramina-dosagem-soro',
'https://www.fleury.com.br/exames/tripsina-pesquisa-fezes',
'https://www.fleury.com.br/exames/triptase-dosagem-soro',
'https://www.fleury.com.br/exames/tromboelastograma-curva-com-ap-tem-sangue',
'https://www.fleury.com.br/exames/tromboelastograma-curva-com-ex-tem-sangue',
'https://www.fleury.com.br/exames/tromboelastograma-curva-com-fib-tem-sangue',
'https://www.fleury.com.br/exames/tromboelastograma-curva-com-hep-tem-sangue',
'https://www.fleury.com.br/exames/tromboelastograma-curva-com-in-tem-sangue',
'https://www.fleury.com.br/exames/tromboelastograma-sangue',
'https://www.fleury.com.br/exames/troponina-i-plasma',
'https://www.fleury.com.br/exames/trypanosoma-cruzi-anticorpo-igg-soro',
'https://www.fleury.com.br/exames/trypanosoma-cruzi-anticorpos-igg-por-imunofluorescencia-ifi-soro',
'https://www.fleury.com.br/exames/transito-intestinal-e-morfologia-do-delgado',
'https://www.fleury.com.br/exames/tetano-anticorpo-igg-soro',
'https://www.fleury.com.br/exames/ultra-som-obstetrica-com-doppler-gestacao-multipla',
'https://www.fleury.com.br/exames/ultra-som-obstetrica-com-doppler-terceiro-trimestre',
'https://www.fleury.com.br/exames/ultra-som-obstetrico-transvaginal-gestacao-multipla',
'https://www.fleury.com.br/exames/ultra-som-morfologico-1o-trim-c/-doppler-e-perfil-bioquimico-gestmult',
'https://www.fleury.com.br/exames/ultra-som-morfologico-1o-trimestre-com-doppler-e-perfil-bioquimico',
'https://www.fleury.com.br/exames/ultra-som-morfologico-c/doppler-terceiro-trim-gest-multipla',
'https://www.fleury.com.br/exames/ultra-som-morfologico-c/doppler-primeiro-trimestre-gestacao-multipla',
'https://www.fleury.com.br/exames/ultra-som-morfologico-c/doppler-segundo-trimestre-gestacao-multipla',
'https://www.fleury.com.br/exames/ultra-sonografia-biomicroscopica-olho-direito',
'https://www.fleury.com.br/exames/ultra-sonografia-biomicroscopica-olho-esquerdo',
'https://www.fleury.com.br/exames/ultra-sonografia-biomicroscopica-olhos-ambos',
'https://www.fleury.com.br/exames/ultra-sonografia-diagnostica-do-globo-ocular-olho-direito',
'https://www.fleury.com.br/exames/ultra-sonografia-diagnostica-do-globo-ocular-olho-esquerdo',
'https://www.fleury.com.br/exames/ultra-sonografia-diagnostica-do-globo-ocular-olhos-ambos',
'https://www.fleury.com.br/exames/ultra-sonografia-gestacional-inicial-gestacao-multipla',
'https://www.fleury.com.br/exames/ultra-sonografia-morfologica-com-doppler-primeiro-trimestre',
'https://www.fleury.com.br/exames/ultra-sonografia-morfologica-com-doppler-segundo-trimestre',
'https://www.fleury.com.br/exames/ultra-sonografia-morfologica-com-doppler-terceiro-trimestre',
'https://www.fleury.com.br/exames/ultra-sonografia-gestacional-inicial',
'https://www.fleury.com.br/exames/ultra-sonografia-obstetrica',
'https://www.fleury.com.br/exames/ultra-sonografia-obstetrica-com-doppler',
'https://www.fleury.com.br/exames/ultra-sonografia-obstetrica-gestacao-multipla',
'https://www.fleury.com.br/exames/ultra-sonografia-obstetrica-transvaginal',
'https://www.fleury.com.br/exames/ultra-sonografia-abdomen-total-e-pelve-feminina',
'https://www.fleury.com.br/exames/ultra-sonografia-abdomen-total-e-pelve-feminina-de-0-a-2-anos-e-11-meses',
'https://www.fleury.com.br/exames/ultra-sonografia-com-doppler-transcraniano-com-pesquisa-de-shunt-de',
'https://www.fleury.com.br/exames/ultra-sonografia-com-doppler-transcraniano-com-vasoreatividade',
'https://www.fleury.com.br/exames/ultra-sonografia-com-doppler-transcraniano',
'https://www.fleury.com.br/exames/ultra-sonografia-pelvica-e-endovaginal',
'https://www.fleury.com.br/exames/ultrassonografia-transcraniana-de-substancia-negra',
'https://www.fleury.com.br/exames/ultrassonografia-de-coluna-infantil',
'https://www.fleury.com.br/exames/ultrassonografia-de-coluna-infantil-com-doppler',
'https://www.fleury.com.br/exames/ultrassonografia-de-0-a-2-anos-e-11-meses',
'https://www.fleury.com.br/exames/ultrassonografia-prova-boyden',
'https://www.fleury.com.br/exames/ultrassonografia-abdomen-superior',
'https://www.fleury.com.br/exames/ultrassonografia-abdomen-superior-de-0-a-2-anos-e-11-meses',
'https://www.fleury.com.br/exames/ultrassonografia-abdomen-total-e-pelve-feminina-endovaginal',
'https://www.fleury.com.br/exames/ultrassonografia-abdomen-total-e-pelve-masculina',
'https://www.fleury.com.br/exames/ultrassonografia-adrenal',
'https://www.fleury.com.br/exames/ultrassonografia-adrenal-de-3-anos-a-11-anos-e-11-meses',
'https://www.fleury.com.br/exames/ultrassonografia-adrenal-de-0-a-2-anos-e-11-meses',
'https://www.fleury.com.br/exames/ultrassonografia-antebraco',
'https://www.fleury.com.br/exames/ultrassonografia-antebracos',
'https://www.fleury.com.br/exames/ultrassonografia-aorta-e-iliacas',
'https://www.fleury.com.br/exames/ultrassonografia-aorta-e-iliacas-3d',
'https://www.fleury.com.br/exames/ultrassonografia-braco',
'https://www.fleury.com.br/exames/ultrassonografia-bracos',
'https://www.fleury.com.br/exames/ultrassonografia-com-doppler-abdomen-superior',
'https://www.fleury.com.br/exames/ultrassonografia-com-doppler-abdomen-superior-de-0-a-2-anos-e-11-meses',
'https://www.fleury.com.br/exames/ultrassonografia-com-doppler-abdomen-total-e-pelve-feminina',
'https://www.fleury.com.br/exames/ultrassonografia-com-doppler-abdomen-total-e-pelve-feminina-0-a-2-anos-11',
'https://www.fleury.com.br/exames/ultrassonografia-com-doppler-arterias-temporais',
'https://www.fleury.com.br/exames/ultrassonografia-com-doppler-carotidas',
'https://www.fleury.com.br/exames/ultrassonografia-com-doppler-cranio-crianca',
'https://www.fleury.com.br/exames/ultrassonografia-com-doppler-das-partes-moles',
'https://www.fleury.com.br/exames/ultrassonografia-com-doppler-das-partes-moles-de-0-a-2-anos-e-11-meses',
'https://www.fleury.com.br/exames/ultrassonografia-com-doppler-endovaginal',
'https://www.fleury.com.br/exames/ultrassonografia-com-doppler-escrotal',
'https://www.fleury.com.br/exames/ultrassonografia-com-doppler-escrotal-de-3-anos-a-11-anos-e-11-meses',
'https://www.fleury.com.br/exames/ultrassonografia-com-doppler-escrotal-de-0-a-2-anos-e-11-meses',
'https://www.fleury.com.br/exames/ultrassonografia-com-doppler-joelho',
'https://www.fleury.com.br/exames/ultrassonografia-com-doppler-mamas',
'https://www.fleury.com.br/exames/ultrassonografia-com-doppler-ombro',
'https://www.fleury.com.br/exames/ultrassonografia-com-doppler-pelve-feminina',
'https://www.fleury.com.br/exames/ultrassonografia-com-doppler-pelve-feminina-de-0-a-2-anos-e-11-meses',
'https://www.fleury.com.br/exames/ultrassonografia-com-doppler-pescoco',
'https://www.fleury.com.br/exames/ultrassonografia-com-doppler-pescoco-de-0-a-2-anos-e-11-meses',
'https://www.fleury.com.br/exames/ultrassonografia-com-doppler-punho',
'https://www.fleury.com.br/exames/ultrassonografia-com-doppler-pe',
'https://www.fleury.com.br/exames/ultrassonografia-com-doppler-penis',
'https://www.fleury.com.br/exames/ultrassonografia-com-doppler-quadril',
'https://www.fleury.com.br/exames/ultrassonografia-com-doppler-quadril-infantil',
'https://www.fleury.com.br/exames/ultrassonografia-com-doppler-tornozelo',
'https://www.fleury.com.br/exames/ultrassonografia-com-doppler-transretal',
'https://www.fleury.com.br/exames/ultrassonografia-com-doppler-trato-urinario-feminino',
'https://www.fleury.com.br/exames/ultrassonografia-com-doppler-trato-urinario-feminino-de-0-a-2-anos-e-11-meses',
'https://www.fleury.com.br/exames/ultrassonografia-com-doppler-trato-urinario-masculino',
'https://www.fleury.com.br/exames/ultrassonografia-com-doppler-trato-urinario-masculino-0-a-2-anos-e-11-meses',
'https://www.fleury.com.br/exames/ultrassonografia-com-doppler-veias-jugulares',
'https://www.fleury.com.br/exames/ultrassonografia-com-doppler-arterial-de-membro-inferior',
'https://www.fleury.com.br/exames/ultrassonografia-com-doppler-arterial-de-membro-superior',
'https://www.fleury.com.br/exames/ultrassonografia-com-doppler-arterial-de-membros-inferiores',
'https://www.fleury.com.br/exames/ultrassonografia-com-doppler-arterial-de-membros-superiores',
'https://www.fleury.com.br/exames/ultrassonografia-com-doppler-arterial-e-venoso-de-membro-inferior',
'https://www.fleury.com.br/exames/ultrassonografia-com-doppler-arterial-e-venoso-de-membro-superior',
'https://www.fleury.com.br/exames/ultrassonografia-com-doppler-arterial-e-venoso-de-membros-inferiores',
'https://www.fleury.com.br/exames/ultrassonografia-com-doppler-arterial-e-venoso-de-membros-superiores',
'https://www.fleury.com.br/exames/ultrassonografia-com-doppler-venoso-de-membro-inferior',
'https://www.fleury.com.br/exames/ultrassonografia-com-doppler-venoso-de-membro-superior',
'https://www.fleury.com.br/exames/ultrassonografia-com-doppler-venoso-de-membros-inferiores',
'https://www.fleury.com.br/exames/ultrassonografia-com-doppler-venoso-de-membros-superiores',
'https://www.fleury.com.br/exames/ultrassonografia-com-doppler-cotovelo',
'https://www.fleury.com.br/exames/ultrassonografia-com-doppler-masculina-de-mamas',
'https://www.fleury.com.br/exames/ultrassonografia-com-doppler-pelvica-e-endovaginal',
'https://www.fleury.com.br/exames/ultrassonografia-com-mapeamento-de-linfonodos-cervical',
'https://www.fleury.com.br/exames/ultrassonografia-com-preparo-intestinal-endovaginal',
'https://www.fleury.com.br/exames/ultrassonografia-cotovelo',
'https://www.fleury.com.br/exames/ultrassonografia-cotovelos',
'https://www.fleury.com.br/exames/ultrassonografia-coxa',
'https://www.fleury.com.br/exames/ultrassonografia-coxas',
'https://www.fleury.com.br/exames/ultrassonografia-de-0-a-2-anos-e-11-meses-abdomen-total-e-pelve-masculina',
'https://www.fleury.com.br/exames/ultrassonografia-de-partes-moles',
'https://www.fleury.com.br/exames/ultrassonografia-de-partes-moles-de-0-a-2-anos-e-11-meses',
'https://www.fleury.com.br/exames/ultrassonografia-endovaginal',
'https://www.fleury.com.br/exames/ultrassonografia-escrotal',
'https://www.fleury.com.br/exames/ultrassonografia-escrotal-de-0-a-2-anos-e-11-meses',
'https://www.fleury.com.br/exames/ultrassonografia-glandulas-salivares',
'https://www.fleury.com.br/exames/ultrassonografia-glandulas-salivares-de-0-a-2-anos-e-11-meses',
'https://www.fleury.com.br/exames/ultrassonografia-infantil-de-0-a-2-anos-e-11-meses-torax',
'https://www.fleury.com.br/exames/ultrassonografia-inguinal',
'https://www.fleury.com.br/exames/ultrassonografia-joelho',
'https://www.fleury.com.br/exames/ultrassonografia-joelhos',
'https://www.fleury.com.br/exames/ultrassonografia-mamas',
'https://www.fleury.com.br/exames/ultrassonografia-mamas-reconvocacao',
'https://www.fleury.com.br/exames/ultrassonografia-masculina-de-mamas',
'https://www.fleury.com.br/exames/ultrassonografia-mao',
'https://www.fleury.com.br/exames/ultrassonografia-maos',
'https://www.fleury.com.br/exames/ultrassonografia-ombro',
'https://www.fleury.com.br/exames/ultrassonografia-ombros',
'https://www.fleury.com.br/exames/ultrassonografia-para-crianca-cranio',
'https://www.fleury.com.br/exames/ultrassonografia-para-monitoracao-de-ovulacao-transvaginal',
'https://www.fleury.com.br/exames/ultrassonografia-para-puncao-de-glandulas-salivares',
'https://www.fleury.com.br/exames/ultrassonografia-para-puncao-de-linfonodos',
'https://www.fleury.com.br/exames/ultrassonografia-parede-abdominal',
'https://www.fleury.com.br/exames/ultrassonografia-parede-abdominal-de-0-a-2-anos-e-11-meses',
'https://www.fleury.com.br/exames/ultrassonografia-pele',
'https://www.fleury.com.br/exames/ultrassonografia-perna',
'https://www.fleury.com.br/exames/ultrassonografia-pernas',
'https://www.fleury.com.br/exames/ultrassonografia-pescoco',
'https://www.fleury.com.br/exames/ultrassonografia-pescoco-de-0-a-2-anos-e-11-meses',
'https://www.fleury.com.br/exames/ultrassonografia-por-via-abdominal-prostata-de-0-a-2-anos-e-11-meses',
'https://www.fleury.com.br/exames/ultrassonografia-prostata-por-via-abdominal',
'https://www.fleury.com.br/exames/ultrassonografia-punho',
'https://www.fleury.com.br/exames/ultrassonografia-punhos',
'https://www.fleury.com.br/exames/ultrassonografia-pe',
'https://www.fleury.com.br/exames/ultrassonografia-pelvica-feminina',
'https://www.fleury.com.br/exames/ultrassonografia-pelvica-feminina-de-0-a-2-anos-e-11-meses',
'https://www.fleury.com.br/exames/ultrassonografia-pes',
'https://www.fleury.com.br/exames/ultrassonografia-penis',
'https://www.fleury.com.br/exames/ultrassonografia-quadril',
'https://www.fleury.com.br/exames/ultrassonografia-quadril-infantil',
'https://www.fleury.com.br/exames/ultrassonografia-quadris',
'https://www.fleury.com.br/exames/ultrassonografia-tornozelo',
'https://www.fleury.com.br/exames/ultrassonografia-tornozelos',
'https://www.fleury.com.br/exames/ultrassonografia-transretal',
'https://www.fleury.com.br/exames/ultrassonografia-trato-urinario-feminino',
'https://www.fleury.com.br/exames/ultrassonografia-trato-urinario-feminino-3d',
'https://www.fleury.com.br/exames/ultrassonografia-trato-urinario-feminino-de-0-a-2-anos-e-11-meses',
'https://www.fleury.com.br/exames/ultrassonografia-trato-urinario-masculino',
'https://www.fleury.com.br/exames/ultrassonografia-trato-urinario-masculino-3d',
'https://www.fleury.com.br/exames/ultrassonografia-trato-urinario-masculino-de-0-a-2-anos-e-11-meses',
'https://www.fleury.com.br/exames/ultrassonografia-torax',
'https://www.fleury.com.br/exames/ultrassonografia-torax-de-3-anos-a-11-anos-e-11-meses',
'https://www.fleury.com.br/exames/ultrassonografia-veia-cava-e-iliacas',
'https://www.fleury.com.br/exames/urecistografia-retrograda',
'https://www.fleury.com.br/exames/ureia-varios-materiais',
'https://www.fleury.com.br/exames/uretrocistografia-infantil',
'https://www.fleury.com.br/exames/uretrocistografia-com-correntinha',
'https://www.fleury.com.br/exames/uretrocistografia-miccional',
'https://www.fleury.com.br/exames/urina-exame-de-varios-materiais',
'https://www.fleury.com.br/exames/urina-exame-de-apos-massagem-prostatica-urina-pos-massagem-prostatica',
'https://www.fleury.com.br/exames/urina-exame-de-sondagem-vesical-urina-colhida-por-sonda-vesical-de-alivio',
'https://www.fleury.com.br/exames/urina-exame-de-urina-jato-final',
'https://www.fleury.com.br/exames/urina-exame-de-urina-jato-medio',
'https://www.fleury.com.br/exames/urina-exame-de-urina-primeiro-jato',
'https://www.fleury.com.br/exames/urodinamica',
'https://www.fleury.com.br/exames/urografia-excretora',
'https://www.fleury.com.br/exames/uroressonancia-urografia-por-ressonancia-magnetica',
'https://www.fleury.com.br/exames/urotomografia-urografia-por-tomografia',
'https://www.fleury.com.br/exames/ureia-depuracao-soro-e-urina',
'https://www.fleury.com.br/exames/ureia-amostra-isolada-urina',
'https://www.fleury.com.br/exames/ureia-soro',
'https://www.fleury.com.br/exames/ureia-urina',
'https://www.fleury.com.br/exames/usom-obstetrico-com-avaliacao-da-translucencia-nucal-com-doppler',
'https://www.fleury.com.br/exames/ustekinumabe-stelara-260mg-medicamento-e-aplicacao-endovenoso',
'https://www.fleury.com.br/exames/ustekinumabe-stelara-390mg-medicamento-e-aplicacao-endovenoso',
'https://www.fleury.com.br/exames/ustekinumabe-stelara-45mg-medicamento-e-aplicacao-subcutaneo',
'https://www.fleury.com.br/exames/ustekinumabe-stelara-520mg-medicamento-e-aplicacao-endovenoso',
'https://www.fleury.com.br/exames/ustekinumabe-stelara-90mg-medicamento-e-aplicacao-subcutaneo',
'https://www.fleury.com.br/exames/ustekinumabe-stelara-apenas-aplicacao-endovenoso',
'https://www.fleury.com.br/exames/ustekinumabe-stelara-apenas-aplicacao-subcutaneo',
'https://www.fleury.com.br/exames/vldl-colesterol-soro',
'https://www.fleury.com.br/exames/vacina-bivalente-contra-papilomavirus-humano',
'https://www.fleury.com.br/exames/vacina-contra-virus-influenza-funcionarios-e-medicos',
'https://www.fleury.com.br/exames/vacina-hexavalente-combinada',
'https://www.fleury.com.br/exames/vacina-pentavalente-combinada',
'https://www.fleury.com.br/exames/vacina-quadrivalente-combinada',
'https://www.fleury.com.br/exames/vacina-tetra-viral',
'https://www.fleury.com.br/exames/vacina-triplice-bacteriana-acelular-adulto',
'https://www.fleury.com.br/exames/vacina-triplice-viral',
'https://www.fleury.com.br/exames/vacina-contra-haemophilus-influenzae',
'https://www.fleury.com.br/exames/vacina-contra-hepatite-a',
'https://www.fleury.com.br/exames/vacina-contra-hepatite-a-infantil',
'https://www.fleury.com.br/exames/vacina-contra-hepatite-b-adulto',
'https://www.fleury.com.br/exames/vacina-contra-hepatite-b-infantil',
'https://www.fleury.com.br/exames/vacina-contra-herpes-zoster',
'https://www.fleury.com.br/exames/vacina-contra-meningococo-a-c-y-e-w135-conjugada',
'https://www.fleury.com.br/exames/vacina-contra-meningococo-b-recombinante',
'https://www.fleury.com.br/exames/vacina-contra-papilomavirus-humano',
'https://www.fleury.com.br/exames/vacina-contra-pneumococo13-conjugada',
'https://www.fleury.com.br/exames/vacina-contra-rotavirus-5',
'https://www.fleury.com.br/exames/vacina-contra-varicela',
'https://www.fleury.com.br/exames/vacina-contra-virus-influenza-adulto',
'https://www.fleury.com.br/exames/vacina-contra-hepatites-a-e-b-combinadas',
'https://www.fleury.com.br/exames/vancomicina-soro',
'https://www.fleury.com.br/exames/varicella-zoster-virus-anticorpos-igg-e-igm-soro',
'https://www.fleury.com.br/exames/varicella-zoster-virus-por-pcr-qualitativo-varios-materiais',
'https://www.fleury.com.br/exames/varicella-zoster-virus-por-pcr-quantitativo-sangue-total',
'https://www.fleury.com.br/exames/vedolizumabe-entyvio-300mg-medicamento-e-aplicacao-endovenoso',
'https://www.fleury.com.br/exames/vedolizumabe-entyvio-apenas-aplicacao-endovenoso',
'https://www.fleury.com.br/exames/videoendoscopia-da-degluticao',
'https://www.fleury.com.br/exames/videoendoscopia-de-degluticao-infantil',
'https://www.fleury.com.br/exames/videolaringoscopia-ou-videolaringoestroboscopia',
'https://www.fleury.com.br/exames/videonasofibrolaringoscopia-infantil',
'https://www.fleury.com.br/exames/virus-febre-amarela-deteccao-do-rna-por-pcr-varios-materiais',
'https://www.fleury.com.br/exames/visao-de-cores-teste-de',
'https://www.fleury.com.br/exames/vitamina-a-soro',
'https://www.fleury.com.br/exames/vitamina-b1-sangue-total',
'https://www.fleury.com.br/exames/vitamina-b12-soro',
'https://www.fleury.com.br/exames/vitamina-b2-plasma',
'https://www.fleury.com.br/exames/vitamina-b3-niacina-plasma',
'https://www.fleury.com.br/exames/vitamina-b5-soro',
'https://www.fleury.com.br/exames/vitamina-b6-sangue',
'https://www.fleury.com.br/exames/vitamina-d-125-dihidroxi-soro',
'https://www.fleury.com.br/exames/vitamina-d-25-hidroxi-por-lc-ms/ms-soro',
'https://www.fleury.com.br/exames/vitamina-d-25-hidroxi-soro',
'https://www.fleury.com.br/exames/vitamina-e-soro',
'https://www.fleury.com.br/exames/vitamina-k-plasma',
'https://www.fleury.com.br/exames/volumetria-de-lesoes-desmielinizantes',
'https://www.fleury.com.br/exames/voriconazol-soro',
'https://www.fleury.com.br/exames/vulvoscopia',
'https://www.fleury.com.br/exames/varias-condicoes-varios-materiais',
'https://www.fleury.com.br/exames/video-head-impulse-test-v-hit',
'https://www.fleury.com.br/exames/virus-chikungunya-anticorpos-igg-e-igm-soro',
'https://www.fleury.com.br/exames/virus-chikungunya-deteccao-do-rna-por-pcr-em-tempo-real-plasma',
'https://www.fleury.com.br/exames/virus-respiratorio-pesquisa-varios-materiais',
'https://www.fleury.com.br/exames/virus-sincicial-respiratorio-antigeno-varios-materiais',
'https://www.fleury.com.br/exames/virus-influenza-a-teste-molecular-para-deteccao-varios-materiais',
'https://www.fleury.com.br/exames/virus-mayaro-pesquisa-de-anticorpos-igg-e-igm-soro-soro',
'https://www.fleury.com.br/exames/virus-oropouche-pesquisa-de-anticorpos-igg-e-igm-soro',
'https://www.fleury.com.br/exames/virus-isolamento-p/-herpes-simples-e-adenovirus-varios-materiais',
'https://www.fleury.com.br/exames/watson-for-genomics-from-quest-core-bloco-de-parafina',
'https://www.fleury.com.br/exames/x-fragil-analise-molecular-sangue-total',
'https://www.fleury.com.br/exames/yo-antinuclear-neuronal-anticorpos-soro',
'https://www.fleury.com.br/exames/zap-70-deteccao-por-citometria-de-fluxo-varios-materiais',
'https://www.fleury.com.br/exames/zika-virus-deteccao-do-rna-por-pcr-em-tempo-real-varios-materiais',
'https://www.fleury.com.br/exames/zikavirus-anticorpo-igg-soro',
'https://www.fleury.com.br/exames/zikavirus-anticorpo-igm-soro',
'https://www.fleury.com.br/exames/zikavirus-deteccao-por-pcr-liquido-amniotico',
'https://www.fleury.com.br/exames/zinco-plasma',
'https://www.fleury.com.br/exames/zinco-urina',
'https://www.fleury.com.br/exames/zonisamida-soro',
'https://www.fleury.com.br/exames/anticorpos-anti-adrenal-soro',
'https://www.fleury.com.br/exames/apos-massagem-prostatica-urina-pos-massagem-prostatica',
'https://www.fleury.com.br/exames/limpeza-do-material-vilo-corial-produto-de-abortam-processamento-de-amostra',
'https://www.fleury.com.br/exames/munoglobulina-endobulin-kiovig-apenas-aplicacao-endovenoso',
'https://www.fleury.com.br/exames/ph-varios-materiais',
'https://www.fleury.com.br/exames/ph-fezes',
'https://www.fleury.com.br/exames/ph-medido-em-phmetro-varios-materiais',
'https://www.fleury.com.br/exames/ph-secrecao-vaginal',
'https://www.fleury.com.br/exames/ph-urina',
'https://www.fleury.com.br/exames/von-willebrand-teste-do-ddavp-para-doenca-de-sangue-total',
'https://www.fleury.com.br/exames/acido-zoledronico-zometar-intravenoso',
'https://www.fleury.com.br/exames/acido-5-hidroxi-indol-acetico-urina',
'https://www.fleury.com.br/exames/acido-ascorbico-plasma',
'https://www.fleury.com.br/exames/acido-delta-amino-levulinico-amostra-isolada-urina',
'https://www.fleury.com.br/exames/acido-delta-amino-levulinico-urina',
'https://www.fleury.com.br/exames/acido-dietilamino-lisergico-dosagem-urina',
'https://www.fleury.com.br/exames/acido-folico-dosagem-soro',
'https://www.fleury.com.br/exames/acido-folico-soro',
'https://www.fleury.com.br/exames/acido-hipurico-dosagem-urina',
'https://www.fleury.com.br/exames/acido-homovanilico-urina-amostra-isolada',
'https://www.fleury.com.br/exames/acido-homovanilico-urina-24-horas',
'https://www.fleury.com.br/exames/acido-lactico-em-liquidos-biologicos-varios-materiais',
'https://www.fleury.com.br/exames/acido-lactico-plasma',
'https://www.fleury.com.br/exames/acido-lactico-sangue-arterial',
'https://www.fleury.com.br/exames/acido-mandelico-urina',
'https://www.fleury.com.br/exames/acido-metilipurico-urina',
'https://www.fleury.com.br/exames/acido-metilmalonico-soro',
'https://www.fleury.com.br/exames/acido-orotico-urina',
'https://www.fleury.com.br/exames/acido-oxalico-amostra-isolada-urina',
'https://www.fleury.com.br/exames/acido-oxalico-urina',
'https://www.fleury.com.br/exames/acido-valproico-soro',
'https://www.fleury.com.br/exames/acido-vanil-mandelico-amostra-isolada-urina',
'https://www.fleury.com.br/exames/acido-vanil-mandelico-urina',
'https://www.fleury.com.br/exames/acido-fenilglioxilico-dosagem-urina',
'https://www.fleury.com.br/exames/acido-trans-trans-muconico-dosagem-urina',
'https://www.fleury.com.br/exames/acido-tricloroacetico-dosagem-urina',
'https://www.fleury.com.br/exames/acido-urico-depuracao-soro-e-urina',
'https://www.fleury.com.br/exames/acido-urico-varios-materiais',
'https://www.fleury.com.br/exames/acido-urico-amostra-isolada-urina',
'https://www.fleury.com.br/exames/acido-urico-soro',
'https://www.fleury.com.br/exames/acido-urico-urina',
'https://www.fleury.com.br/exames/acidos-biliares-totais-dosagem-soro',
'https://www.fleury.com.br/exames/acidos-graxos-livres-totais-soro',
'https://www.fleury.com.br/exames/acidos-graxos-perfil-soro',
'https://www.fleury.com.br/exames/acidos-graxos-de-cadeia-muito-longa-soro',
'https://www.fleury.com.br/exames/acidos-organicos-urina']

In [9]:
import pandas as pd
from pandas import json_normalize

def valores(lista):
    valores = ''
    for value in lista:
        valores += value +'\n'
    return valores

exames = {}
paginas = {'Não encontrada' : 0}

for url in lista:
    new_html = requests.get(url).content
    seletor = Selector (text = new_html)
    titulo_exame = seletor.xpath('//h1[@class = "typographycomponentstyle__H1-sc-1oox73n-1 lomAMa"]/text()').extract_first()
    if titulo_exame == 'Página não encontrada':
        print('Pagina Não Encontrada')
        paginas['Não encontrada'] += 1
        pass
    else:
        outros_nomes = seletor.xpath('//div[@class = "sc-htpNat gridcomponent__ColStyled-sc-8zg10d-2 dsZazA"]/p/text()').extract()
        outros_nomes_ext = [outros for outros in outros_nomes]
        exames[titulo_exame] =  valores(outros_nomes)

print(paginas)
lista_exames = pd.json_normalize(exames)
lista_exames.to_excel('/content/excel.xlsx', index=False)
   


Pagina Não Encontrada
Pagina Não Encontrada
Pagina Não Encontrada
Pagina Não Encontrada
Pagina Não Encontrada
Pagina Não Encontrada
Pagina Não Encontrada
Pagina Não Encontrada
Pagina Não Encontrada
Pagina Não Encontrada
Pagina Não Encontrada
Pagina Não Encontrada
Pagina Não Encontrada
Pagina Não Encontrada
Pagina Não Encontrada
Pagina Não Encontrada
Pagina Não Encontrada
Pagina Não Encontrada
Pagina Não Encontrada
Pagina Não Encontrada
Pagina Não Encontrada
Pagina Não Encontrada
Pagina Não Encontrada
Pagina Não Encontrada
Pagina Não Encontrada
Pagina Não Encontrada
Pagina Não Encontrada
Pagina Não Encontrada
Pagina Não Encontrada


KeyboardInterrupt: ignored

In [10]:
paginas


{'Não encontrada': 29}